# 导入模块

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import gc
import json
import random
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 导入订单数据

In [3]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money
,om.status, om.user_id, om.overdue, om.alipay_order_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money, tod.product_id
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name, om.has_actual, om.all_pledge, om.day 租赁天数
,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
,om.buy_service_product
,tso.status as service_status 
,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
, tor.update_time, tprm.purchase_amount, tprm.liquidated_damages 违约金
, tomt.reason
, om.order_type, om.union_rent_tag, tprm.rent_received 已收租金 , tprm.paid_buyout_balance_payment 已付买断尾款, tprm.relet_days
, tolog.status 物流状态, tojo.app_type

, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.rejected_ato_risk') end,'"','') as rejected_ato_risk  # 联合拒量

, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.union_rent_rejected') end,'"','') as union_rent_rejected   # 联合运营

, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_241011') end,'"','') as strategy_241011                    # 拒量
, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_241205_rejected') end,'"','') as strategy_241205_rejected  # 拒量
, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_241212_rejected') end,'"','') as strategy_241212_rejected  # 拒量
, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_250120_rejected') end,'"','') as strategy_250120_rejected  # 拒量
, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_250330_rejected') end,'"','') as strategy_250330_rejected  # 拒量

-- , ymos.money, ymos.part_payment, ymos.refund_date, ymos.reality_refund_date, ymos.sort, ymos.status 分期租金支付状态
from  db_digua_business.t_order  om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 

where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
-- 京东外部订单关联表
left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id
-- 订单物流表
left join db_digua_business.t_order_logistics tolog on tolog.order_id=om.id
-- 租后分期表
-- left join db_rent.ya_merchant_order_stages ymos on ymos.order_id=om.id
left join db_credit.risk risk on risk.order_id=om.id


where om.user_mobile is not null 
-- 曙光计划订单会在去重逻辑中剔除，这里不做过滤，否则存在非线下小店订单被剔除的情况
-- and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
-- ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and pa.type!=4 号卡活动，已过期，不统计
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )               -- 近20天数据
-- and  om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -0 day )
-- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-11-14'
-- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-11-20'
-- and TIME(om.create_time) <= '17:00:00'
-- and order_type = 'ZFB_ORDER'



'''

df_order = clean.query(sql1)
df_order.shape
# (175508, 63)

(261231, 65)

In [4]:
sql_modelname = '''
-- 商品表
select
tp.id, tp.classify_id, tp.id 商品ID, tpmn.name phone_name
from db_digua_business.t_product tp
left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
'''
df_modelname = clean.query(sql_modelname)
df_order = df_order.merge(df_modelname, left_on='product_id', right_on='id',how='left')

In [5]:
sql_risk = ''' -- risk等级
select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
-- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_tag') end,'"','') as union_rent_tag
-- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_rejected') end,'"','') as union_rent_rejected

from db_credit.risk
'''
df_risk = clean.query(sql_risk)
# df_risk.loc[:, 'time_r'] = pd.to_datetime(df_risk.time.dt.date,errors="coerce")
# df_risk = df_risk.sort_values(by='time', ascending=False).groupby('id_card_r').head(1)

In [6]:
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = clean.query(sql3)
df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)
# df_re = df_re.sort_values(by='time', ascending=False).groupby('id_card_re').head(1)

In [7]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = clean.query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

In [8]:
# sql_name = '''
# SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
# FROM db_digua_business.t_user_verify_order_record tuvor
# left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
# where tuvor.del_flag = 0 ORDER BY tuvor.update_time
# '''
# df_name = clean.query(sql_name)

In [9]:
# # 首逾
# sql_stages = '''
# select 
# tos.order_id, tos.sort, tos.refund_date, tos.reality_refund_date
# from db_digua_business.t_order_stages tos
# where tos.sort = 2
# '''
# df_stages = clean.query(sql_stages)

In [10]:
sql_wl = '''
select
date_format(om.create_time, '%Y-%m-%d') 下单日期, tort.order_num order_number, tort.task_status
from db_digua_business.t_order_retention_task tort
left join db_digua_business.t_order om on om.id=tort.order_id
'''
df_wl = clean.query(sql_wl)
df_wl.shape

(4353, 3)

In [11]:
sql_ywl = '''
select

from db_digua_business.t_order om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
'''

In [12]:
# df_order[df_order.order_number=='A20250806150830177']

# 提取订单风控数据

# 合并数据

In [13]:
df = df_order.copy()
df = df[df.type!=4]

# df = df[df.phone_name=='Apple watch SE']
# df.phone_name.unique()
# df.order_id.notnull().sum()
# df = df[df['product_name'].str.contains(r'Apple Watch SE')]
# df.phone_name.unique()
df.shape

(261231, 69)

# 处理日期

In [14]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
df['年份'] = df["下单日期"].dt.year
df['hour'] = df['create_time'].dt.hour


In [15]:
# 机审强拒策略
# 提取拒绝理由字段并清理特殊字符
# df['拒绝理由'] = df["rejected"].fillna('').str.replace(r'[\[\]"]', '', regex=True)
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')

df['拒绝理由_risk']= df['rejected'].fillna('')+df['strategy_241205_rejected'].fillna('')+df['strategy_241212_rejected'].fillna('')+df['strategy_241011'].fillna('')+df['strategy_250120_rejected'].fillna('')+df['strategy_250330_rejected'].fillna('')+df['union_rent_rejected'].fillna('')+df['rejected_ato_risk'].fillna('')

df.loc[:, '拒绝理由'] = np.where(df['拒绝理由'] == '', df['拒绝理由_risk'], df['拒绝理由'])
df['拒绝理由'] = df["拒绝理由"].str.replace("[","").str.replace("]","").str.replace('"','')

In [16]:
df["下单日期"].unique()
# df[df['拒绝理由']=='[]']#.value_counts()

<DatetimeArray>
['2025-11-09 00:00:00', '2025-11-10 00:00:00', '2025-11-11 00:00:00',
 '2025-11-12 00:00:00', '2025-11-13 00:00:00', '2025-11-14 00:00:00',
 '2025-11-15 00:00:00', '2025-11-16 00:00:00', '2025-11-17 00:00:00',
 '2025-11-18 00:00:00', '2025-11-19 00:00:00', '2025-11-20 00:00:00',
 '2025-11-21 00:00:00', '2025-11-22 00:00:00', '2025-11-23 00:00:00',
 '2025-11-24 00:00:00']
Length: 16, dtype: datetime64[ns]

# 渠道补充

In [17]:
# qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
# qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)
# qudao_name_df.rename(columns={'订单号': '订单号1'}, inplace=True)
# df =  df.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号1',how = 'left')
# df["channel_name"] = np.where(df.渠道名称.notnull(),df.渠道名称,df["channel_name"])

# 处理备注信息

In [18]:
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip().str.split('"').str[0]
# # 修复后的代码：增强字符串类型转换的鲁棒性,total_describes为空时使用下面这个
# df["取消原因"] = df["total_describes"].fillna('').astype(str).apply(
#     lambda x: x.split("客户申请取消：")[1].split("$")[0].strip().split('"')[0] 
#     if "客户申请取消：" in x else ""
# )
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip().str.split('"').str[0]
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
# df["无法联系原因"] = df["total_describes"].fillna('').astype(str).apply(
#     lambda x: x.split("用户无法联系：")[1].split("$")[0].strip().split('"')[0] 
#     if "用户无法联系：" in x else ""
# )

df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")
df.loc[:,"颜色"]=df.apply(lambda x:clean.getcolor(x["sku_attributes"]),axis=1)
df.loc[:,"内存"]=df.apply(lambda x: clean.getneicun(x["sku_attributes"]),axis=1)
df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
df.loc[:, '商品ID'] = df.商品ID.astype(str)+'_'+df.phone_name
# 物流状态=5，退货拒签
df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
df.loc[:,'机型内存'] = df['phone_name'] + df['内存']
df.loc[:, '租赁类型'] = np.where(df.租赁天数 >= 90, '长租', '短租')


# 渠道归属

In [19]:

df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)


In [20]:
# df[df['来源渠道'].isin(['搜索下挂-商品', '搜索-vivo'])][['order_number','来源渠道', '归属渠道', 'channel_type_id', 'order_type']].drop_duplicates()
# df[(df.channel_type_id.isin ([48, 81])) & (df.order_type=='ZFB_ORDER') & (df['来源渠道'] != '灯火联投测试')].来源渠道.value_counts()
# df.归属渠道.value_counts()


# 订单去重

In [21]:
# 保留不去重的数据
df_no_drop = df.copy()
# 订单去重
# to_do 京东创建订单数不去重
df = clean.order_drop_duplicates(df)
df.shape


(106154, 93)

In [22]:
# df[df.user_mobile=='15104246892']

# 定义状态

In [23]:
# note 2025-02-24添加
df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')
# , 'union_rent_tag', 'union_rent_rejected'
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
# note 2025-02-24修改
# 可用om.refuse_type 判断'拒绝类型：1、前置拦截；2、后置拦截（机审强拒）；3、人审拒绝  4.用户申请取消 5.自动取消  6.出库前强拒',暂不修改逻辑
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)

In [24]:
# df[df.status2.isna()][['order_number', 'status', 'status2']]
# df.dropna(subset=["status2"], axis=0, inplace=True)

In [25]:

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)




# 保留商家的数据

In [26]:
df_contain = df.copy()
df_contain.shape


(106154, 101)

# 剔除商家数据

In [27]:
# 调用Data_Clean类中的drop_merchant函数
df = clean.drop_merchant(df)
df.shape

(101339, 101)

# 去重订单明细导出

In [28]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-11-24'

# 导入服务订单

In [29]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

df_j = df[df["进件"]=="进件"]
df_j.shape

(25203, 102)

In [30]:
df = clean.status_node(df)
# 判定app出库单2025.11.11
df.loc[:, '搜索app出库'] = np.where((df.channel_type_id.isin([48, 81])) & (df.order_type=='DIGUA_APP') & (df.是否出库==1)&(df.alipay_order_id.notna()), 1, 0)
df.loc[:, '芝麻app出库'] = np.where((df.channel_type_id.isin([2, 3, 80])) & (df.order_type=='DIGUA_APP') & (df.是否出库==1)&(df.alipay_order_id.notna()), 1, 0)
print(df.搜索app出库.value_counts())
df.芝麻app出库.value_counts()


搜索app出库
0    101335
1         4
Name: count, dtype: int64


芝麻app出库
0    101336
1         3
Name: count, dtype: int64

# 导入台账数据

In [31]:
#读取导入出库台账数据
# f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
# df_ck = pd.read_excel(f_path_ck,sheet_name="2023")  

f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2025") 
df_ck.shape

(45094, 18)

In [32]:
dfck = pd.merge(df_ck,df,left_on="订单号", right_on="order_number")
dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape

(2557, 137)

In [33]:
dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)
# 删除 露营设备 出库 
try:
    dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
except:
    dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)
# 将序列号转换为日期
base_date = pd.to_datetime('1899-12-30')  # Excel的基准日期
dfck['日期'] = base_date + pd.to_timedelta(dfck['日期'], unit='D')


In [34]:
# dfck.日期.value_counts().sort_index(ascending=False)

# 每日各渠道出库单数

In [35]:
# 按转化日期看每日各渠道出库单数
df_weekday_zh = pd.crosstab(dfck["日期"],dfck["归属渠道"],margins=True)
# 按下单日期看每日各渠道出库单数
df_weekday = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)
# 按下单日期看每日各渠道出库单数（纯租物）
df_ly_ck = pd.crosstab(dfck["下单日期"],dfck["来源渠道"],margins=True)
# 按下单日期看每日支付宝直播商品出库单数
df_zfb_ck = pd.crosstab(dfck["下单日期"],dfck["activity_name"],margins=True)
# 按下单日期看 免押 出库单数
df_my = pd.crosstab(dfck["下单日期"],dfck["押金类型"],margins=True)

In [36]:
# dfck['归属渠道'].unique()
# df_weekday_zh

# 剔除拒量订单

In [37]:
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)
df2 = df2[df2.是否拒量==0]
day_name_cn_mapping = {
    'Monday': '星期一',
    'Tuesday': '星期二',
    'Wednesday': '星期三',
    'Thursday': '星期四',
    'Friday': '星期五',
    'Saturday': '星期六',
    'Sunday': '星期日'
}

# 临时需求

## 自然月支付宝长租发货量(出库)+GMV(new_actual_money)金额是多少

In [38]:
df_cz = df[(df.租赁类型=='长租')&(df.是否出库==1)]
df_cz_g = df_cz.groupby(["下单日期"]).agg({"是否出库":'sum',"new_actual_money":'sum'}).rename(columns={"是否出库":"出库","new_actual_money":"金额"})
df_cz_g.loc[:, '平均签约价'] = df_cz_g.金额/df_cz_g.出库
df_cz_g.loc['日均值',:] = df_cz_g.mean()
df_cz_g.loc['总计',:] = df_cz_g.sum()
df_cz_g.to_excel("F:/需求/瑞彤需求/近一周自然月支付宝长租发货量+GMV金额.xlsx")

In [39]:
# df_contain["是否进件"] = np.where(df_contain["进件"]=='进件',1,0)
# df_contain["是否出库"] = np.where(df_contain["status"].isin([2,3,4,5,6,8,15]),1,0)
# df_es = df_contain[df_contain.商品类型=='二手']
# df_es.loc[:, '订单渠道'] = np.where(df_es['order_type']=='ZFB_ORDER', '支付宝', df_es['归属渠道'])
# print(df_es.订单渠道.value_counts())
# df_es_zfb = df_es[df_es.订单渠道=='支付宝']
# df_es_jd = df_es[df_es.订单渠道=='京东渠道']
# df_es_group = df_es.groupby('下单日期').agg({'order_number':'count', '是否进件': 'sum', '是否出库': 'sum'}).rename(columns={'order_number':'去重订单数', '是否进件': '进件数', '是否出库': '出库数'})
# df_es_zfb_group = df_es_zfb.groupby('下单日期').agg({'order_number':'count', '是否进件': 'sum', '是否出库': 'sum'}).rename(columns={'order_number':'去重订单数', '是否进件': '进件数', '是否出库': '出库数'})
# df_es_jd_group = df_es_jd.groupby('下单日期').agg({'order_number':'count', '是否进件': 'sum', '是否出库': 'sum'}).rename(columns={'order_number':'去重订单数', '是否进件': '进件数', '是否出库': '出库数'})

# with pd.ExcelWriter('F:\需求\瑞彤需求\二手近十五天数据_2025.11.21.xlsx') as writer:
#     df_es_group.to_excel(writer, sheet_name='全渠道', index=True)
#     df_es_zfb_group.to_excel(writer, sheet_name='支付宝', index=True)
#     df_es_jd_group.to_excel(writer, sheet_name='京东渠道', index=True)  

## 租安盾风险最大等级

In [40]:
# # 获取出库订单order_number list
# df_ck_order_list = df[df.是否出库==1].order_number.tolist()

# # 获取租安盾风险等级
# # 下单时
# # 租安盾的表用 订单号关联，11.7 后 不要用trace_id关联。因为11.7之后租安盾是实时的，不走缓存
# tmp5 = clean.query(""" select * from  db_credit.risk_alipay_rent_risk rarr where date_format(rarr.time, '%Y-%m-%d') >= '2025-11-03' """) 
# tmp5['zad_risk_level'] = tmp5['data'].apply(lambda x:json.loads(x).get('comprehensive_risk_models',{}).get('risk_level') if pd.notna(x) else None) 

# tmp5['ship_goods_risk_models'] = tmp5['data'].apply(lambda x:json.loads(x).get('ship_goods_risk_models',{}) if pd.notna(x) else None)
# tmp5['is_bf'] = tmp5['ship_goods_risk_models'].apply(lambda x:x[0].get('can_ship_flag') if x!={} else None)

# tmp5 = tmp5[['out_biz_no','zad_risk_level','orderConcurrentRentRisk','is_bf','time']]
# tmp5 = tmp5.rename(columns = {'out_biz_no':'order_number'})

# print(tmp5.shape)
# # 筛选出出库的订单
# tmp5 = tmp5[tmp5.order_number.isin(df_ck_order_list)]
# tmp5.shape
# # tmp5[:2]

In [41]:
# # 将重复的order_number记录合并，time最大记录的zad_risk_level	orderConcurrentRentRisk	is_bf，拼接到time最小的记录中，列名改为last_zad_risk_level	last_orderConcurrentRentRisk	last_is_bf
# # 首先找到每个order_number中time最小的记录作为主记录
# main_records = tmp5.loc[tmp5.groupby('order_number')['time'].idxmin()]

# # 然后找到每个order_number中time最大的记录索引，用于获取需要拼接的字段
# last_records = tmp5.loc[tmp5.groupby('order_number')['time'].idxmax(), 
#                     ['order_number', 'zad_risk_level', 'orderConcurrentRentRisk', 'is_bf']]

# # 重命名需要拼接的字段
# last_records.rename(columns={
#     'zad_risk_level': 'last_zad_risk_level',
#     'orderConcurrentRentRisk': 'last_orderConcurrentRentRisk',
#     'is_bf': 'last_is_bf'
# }, inplace=True)

# # 将主记录与最后记录合并
# tmp5 = main_records.merge(last_records, on='order_number', how='left')
# tmp5

In [42]:


# # 定时任务 3/6/12
# zad_current = clean.query("""select out_biz_no,data_3_h,data_6_h,data_12_h from db_credit.risk_alipay_rent_risk_cron""")
# zad_current['risk_level_3h'] = zad_current['data_3_h'].apply(lambda x:json.loads(x).get('comprehensive_risk_models',{}).get('risk_level') if pd.notna(x) else None)
# zad_current['risk_level_6h'] = zad_current['data_6_h'].apply(lambda x:json.loads(x).get('comprehensive_risk_models',{}).get('risk_level') if pd.notna(x) else None)
# zad_current['risk_level_12h'] = zad_current['data_12_h'].apply(lambda x:json.loads(x).get('comprehensive_risk_models',{}).get('risk_level') if pd.notna(x) else None)


# zad_current['orderConcurrentRentRisk_3h'] = zad_current['data_3_h'].apply(lambda x: json.loads(x).get('risk_infos', [{}])[0].get('risk_item_list', [{}])[0].get('risk_level', '') if pd.notna(x) and x else None)
# zad_current['orderConcurrentRentRisk_6h'] = zad_current['data_6_h'].apply(lambda x: json.loads(x).get('risk_infos', [{}])[0].get('risk_item_list', [{}])[0].get('risk_level', '') if pd.notna(x) and x else None)
# zad_current['orderConcurrentRentRisk_12h'] = zad_current['data_12_h'].apply(lambda x: json.loads(x).get('risk_infos', [{}])[0].get('risk_item_list', [{}])[0].get('risk_level', '') if pd.notna(x) and x else None)

# zad_current['ship_goods_risk_models'] = zad_current['data_3_h'].apply(lambda x:json.loads(x).get('ship_goods_risk_models',{}) if pd.notna(x) else None)
# zad_current['is_bf_3h'] = zad_current['ship_goods_risk_models'].apply(lambda x:x[0].get('can_ship_flag') if x!={} else None)

# zad_current['ship_goods_risk_models'] = zad_current['data_6_h'].apply(lambda x:json.loads(x).get('ship_goods_risk_models',{}) if pd.notna(x) else None)
# zad_current['is_bf_6h'] = zad_current['ship_goods_risk_models'].apply(lambda x:x[0].get('can_ship_flag') if x!={} and pd.notna(x) else None)

# zad_current['ship_goods_risk_models'] = zad_current['data_12_h'].apply(lambda x:json.loads(x).get('ship_goods_risk_models',{}) if pd.notna(x) else None)
# zad_current['is_bf_12h'] = zad_current['ship_goods_risk_models'].apply(lambda x:x[0].get('can_ship_flag') if x!={} and pd.notna(x) else None)


# var_1 = ['out_biz_no','risk_level_3h','risk_level_6h','risk_level_12h','orderConcurrentRentRisk_3h','orderConcurrentRentRisk_6h','orderConcurrentRentRisk_12h'
#          ,'is_bf_3h','is_bf_6h','is_bf_12h']
# zad_current = zad_current[var_1]
# zad_current = zad_current.rename(columns = {'out_biz_no':'order_number'})
# zad_current.shape
# # zad_current[:1]


# tmp6 = tmp5.merge(zad_current, on = 'order_number', how = 'left')
# tmp6.shape

In [43]:
# # 取出订单中指定列的值作为映射字典的键
# # 空值映射为-1，其他值映射为对应的字典值
# # 将每条记录的最大值对应键取出作为tmp6的max_risk_level列
# level_dic1 = {'nor':0,'low':1,'mid':2,'high':3,'sup':4}
# reverse_dic1 = {v: k for k, v in level_dic1.items()}
# level_dic2 = {'T1':0,'T2':1,'T3':2,'T4':3,'T5':4,'T6':5,'T7':6,'T8':7,'T9':8,'T10':9}
# reverse_dic2 = {v: k for k, v in level_dic2.items()}

# # 定义需要处理的列
# risk_columns1 = ['orderConcurrentRentRisk', 'last_orderConcurrentRentRisk','orderConcurrentRentRisk_3h','orderConcurrentRentRisk_6h','orderConcurrentRentRisk_12h']
# risk_columns2 = ['zad_risk_level','is_bf', 
#                 'last_zad_risk_level','last_is_bf',
#                 'risk_level_3h', 'risk_level_6h', 'risk_level_12h'
#                 ]
# # 创建映射函数
# def map_risk_level1(value):
#     if pd.isna(value) or value not in level_dic1:
#         return -1
#     return level_dic1[value]

# def map_risk_level(value):
#     if pd.isna(value) or value not in level_dic2:
#         return -1
#     return level_dic2[value]


# mapped_columns1 = []
# for col in risk_columns1:
#     if col in tmp6.columns:
#         # 创建新的映射列
#         mapped_col_name1 = f'{col}_mapped'
#         tmp6[mapped_col_name1] = tmp6[col].apply(map_risk_level1)
#         mapped_columns1.append(mapped_col_name1)
# # 计算每行中映射值的最大值
# if mapped_columns1:
#     tmp6['max_RentRisk_value'] = tmp6[mapped_columns1].max(axis=1)
    
#     # 将最大值映射回对应的键名
#     def map_value_to_key(value):
#         if value == -1:
#             return 'Unknown'  # 或者可以返回None或其他标记
#         return reverse_dic1.get(value, 'Unknown')
    
#     tmp6['max_RentRisk'] = tmp6['max_RentRisk_value'].apply(map_value_to_key)


# # 对每个风险等级列进行映射
# mapped_columns = []
# for col in risk_columns2:
#     if col in tmp6.columns:
#         # 创建新的映射列
#         mapped_col_name = f'{col}_mapped'
#         tmp6[mapped_col_name] = tmp6[col].apply(map_risk_level)
#         mapped_columns.append(mapped_col_name)

# # 计算每行中映射值的最大值
# if mapped_columns:
#     tmp6['max_risk_level_value'] = tmp6[mapped_columns].max(axis=1)
    
#     # 将最大值映射回对应的键名
#     def map_value_to_key(value):
#         if value == -1:
#             return 'Unknown'  # 或者可以返回None或其他标记
#         return reverse_dic2.get(value, 'Unknown')
    
#     tmp6['max_risk_level'] = tmp6['max_risk_level_value'].apply(map_value_to_key)
    
    
# print(tmp6.max_risk_level.value_counts())
# tmp6.max_RentRisk.value_counts()

In [44]:
# # 筛选max_risk_level为T10且max_RentRisk=sup
# tmp6 = tmp6[(tmp6.max_risk_level=='T10')&(tmp6.max_RentRisk=='sup')]

In [45]:
# # 分割时间段，time分成11.03-11.09、11.10-11.13、11.14-11.19三个时间段，再分组聚合max_risk_level
# tmp7 = tmp6.copy()
# # 使用pd.cut进行时间段分段

# # 确保time列是datetime类型
# tmp7['time'] = pd.to_datetime(tmp7['time'])

# # 定义时间分割点
# bins = [
#     pd.to_datetime('2025-11-02 23:59:59'),  # 小于这个时间
#     pd.to_datetime('2025-11-09 23:59:59'),  # 11.03-11.09
#     pd.to_datetime('2025-11-13 23:59:59'),  # 11.10-11.13
#     pd.to_datetime('2025-11-19 23:59:59')   # 11.14-11.19
# ]

# labels = ['11.03-11.09', '11.10-11.13', '11.14-11.19']

# # 创建时间段分组
# tmp7['time_period'] = pd.cut(tmp7['time'], bins=bins, labels=labels, include_lowest=False)

# # 按时间段分组聚合max_risk_level，.unstack将第二索引转为列
# risk_summary = tmp7.groupby(['time_period', 'max_risk_level']).size().unstack(fill_value=0)
# # 计算各列各行总数
# risk_summary['total'] = risk_summary.sum(axis=1)
# risk_summary.loc['total',:] = risk_summary.sum(axis=0)
# risk_summary


In [46]:
# df.phone_name.unique()
# df[(df.phone_name=='iPhone13')&(df.status==4)]#.status2.value_counts()

## 获取前5的spu机型、前10的sku机型内存转化

In [47]:
# df_spu = df.groupby('phone_name').agg({'是否出库':'sum'}).rename(columns={'是否出库':'出库'})

# df_spu.loc[:, '出库占比'] = df_spu['出库'] / df_spu['出库'].sum()
# df_spu.loc[:, '出库占比'] = df_spu['出库占比'].apply(lambda x:format(x,".2%"))
# # 出库数降序排序
# df_spu = df_spu.sort_values(by='出库', ascending=False)

# df_spu_new = df_spu.reset_index()
# df_spu_new[:10]

In [48]:
# df_sku = df.groupby('机型内存').agg({'是否出库':'sum'}).rename(columns={'是否出库':'出库'})
# df_sku.loc[:, '出库占比'] = df_sku['出库'] / df_sku['出库'].sum()
# df_sku.loc[:, '出库占比'] = df_sku['出库占比'].apply(lambda x:format(x,".2%"))
# # 出库数降序排序
# df_sku = df_sku.sort_values(by='出库', ascending=False)
# df_sku_new = df_sku.reset_index()
# df_sku_new[:15]


In [49]:
# with pd.ExcelWriter(f'F:/需求/七月需求/2025年10月01spu、sku出库占比.xlsx', engine='xlsxwriter') as writer:
#     df_spu_new[:10].to_excel(writer, sheet_name='spu')
#     df_sku_new[:15].to_excel(writer, sheet_name='sku')

## 迪瓜租手机芝麻租赁频道异常交易订单匹配

In [50]:
# df_12 =  df[df.order_number == 'A20250807124004660'][['order_number','拒绝理由','电审拒绝原因','审核状态','tips','是否前置拦截','是否机审强拒','是否出库前风控强拒']]
# df_12
# df.下单日期.unique()


In [51]:
# df_ali_order = pd.read_excel('F:/需求/瑞彤需求/迪瓜租手机芝麻租赁频道异常交易核对.xlsx')
# df_ali_orderid_list = df_ali_order['order_id'].astype(str).tolist()
# df_ali_new = df[df['alipay_order_id'].isin(df_ali_orderid_list)]

# df_ali_merge = df_ali_order.merge(df_ali_new[['order_number','alipay_order_id','下单日期', '来源渠道', '归属渠道', 'status2', 'all_money']], left_on='order_id', right_on='alipay_order_id', how='left')
# df_ali_merge.rename(columns={'order_number': '订单号', 'status2': '订单状态', 'all_money': '总租金', 'alipay_order_id': '支付宝订单号'}, inplace=True)
# df_ali_merge.to_excel('F:/需求/瑞彤需求/迪瓜租手机芝麻租赁频道异常交易核对结果.xlsx', index=False)
# 宏生京东iPhone17系列出库订单明细
# df_ck_jd = df[(df.是否出库==1)&(df.phone_name.str.contains(r'iPhone 17|iPhone Air'))]
# print(df_ck_jd.phone_name.unique())
# df_result = df_ck_jd[['下单日期', 'order_number', 'all_money', 'new_actual_money', 'all_pledge', '租赁方案', '机型内存']]
# df_result.rename(columns={'order_number': '订单号', 'all_money': '总租金', 'new_actual_money': '签约价', 'all_pledge': '押金'}, inplace=True)
# df_result.to_excel('F:/宏生/2025年京东iPhone17系列出库订单明细.xlsx', index=False)


A20250301093707114 正常被拒订单  拒绝理由:命中自有模型强拒, 命中融安分低于680强拒, 评分等级低于D,命中拒绝  审核状态：机审强拒  是否机审强拒：1  是否出库前风控强拒: 0

A20250821195141539 拒量订单（回捞241205） 拒绝理由:null  审核状态：人审拒绝  是否机审强拒：0, 是否出库前风控强拒: 0   日报会回加到机审强拒中

A20250916223149153 联合拒量订单 拒绝理由:null  审核状态：人审拒绝  是否机审强拒：0  是否出库前风控强拒：0

A20250807124004660 联合运营被拒订单(支付宝联合运营订单) 拒绝理由:null  审核状态：人审拒绝  是否机审强拒：0  是否出库前风控强拒：0  [命中银联模型及Fico联合规则强拒(青云分豁免), 已有在租, 支付宝联合运营订单, 用户下单次数:2]   日报不会回加到机审强拒中


In [52]:

# df_lhyy = df[(df.tips.str.contains(r'支付宝联合运营', regex=True, na=False)==True)&(df['是否出库'] == 1)&(df['下单日期'] >= '2025-10-01')&(df['下单日期'] <= '2025-10-16')][['order_number', '下单日期']]#.shape
# df_lhyy_g = df_lhyy.groupby(['下单日期']).agg({"order_number":np.size}).reset_index()
# df_lhyy_g.loc['合计'] = df_lhyy_g.select_dtypes(include=[np.number]).sum()
# df_lhyy_g

## 统计联合拒量出库订单明细；

In [53]:
# # df.drop_duplicates(subset=["id_card_num", "下单日期"], keep="last", inplace=True)# 75
# df[(df.tips.str.contains(r'联合拒量', regex=True, na=False)==True)&(df['是否出库'] == 1)&(df['下单月份'] == '2025-11')][['order_number', '下单日期']].to_excel('F:/需求/倩姐需求/联合拒量_2025-11.xlsx', index=True)

### 联合拒量转化

In [54]:
# def group_data(df0, classify):
#     df_group = df0.groupby(classify).agg({'order_id':'size','是否进件':'sum','进件前取消':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum','出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
#     df_group.rename(columns={'order_id':'去重订单数', '是否进件': '进件数', '是否出库': '出库'},inplace=True )

#     df_group["拦截率"] = df_group["前置拦截"]/df_group["去重订单数"]
#     df_group["拦截率"] = df_group["拦截率"].apply(lambda x:format(x,".2%"))

#     df_group["进件前取消率"] = df_group["进件前取消"]/df_group["去重订单数"]
#     df_group["进件前取消率"] = df_group["进件前取消率"].apply(lambda x:format(x,".2%"))

#     df_group['预授权通过率'] = df_group["进件数"]/df_group["去重订单数"]
#     df_group["预授权通过率"] = df_group["预授权通过率"].apply(lambda x:format(x,".2%"))

#     df_group["强拒比例"] = df_group["机审强拒"]/df_group['进件数']
#     df_group["强拒比例"] = df_group["强拒比例"].apply(lambda x:format(x,".2%"))

#     df_group["机审通过件"] = df_group["进件数"]-df_group["机审强拒"]

#     df_group["风控通过件"] = df_group["进件数"]-df_group["机审强拒"]-df_group["人审拒绝"]
#     df_group["风控通过率"] = df_group["风控通过件"]/df_group['进件数']
#     df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x:format(x,".2%"))

#     df_group["取消率"] = df_group["客户取消"]/df_group['进件数']
#     df_group["取消率"] = df_group["取消率"].apply(lambda x:format(x,".2%"))

#     df_group["无法联系占比"]=df_group["无法联系"]/df_group['进件数']
#     df_group["无法联系占比"]=df_group["无法联系占比"].apply(lambda x:format(x,".2%"))

#     df_group["人审拒绝率"]=df_group["人审拒绝"]/df_group['进件数']
#     df_group["人审拒绝率"]=df_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

#     df_group["出库前强拒比例"]=df_group["出库前风控强拒"]/df_group['进件数']
#     df_group["出库前强拒比例"]=df_group["出库前强拒比例"].apply(lambda x:format(x,".2%"))

#     df_group["进件出库率"]=df_group["出库"]/df_group['进件数']
#     df_group["进件出库率"]=df_group["进件出库率"].apply(lambda x:format(x,".2%"))

#     df_group["订单出库率"]=df_group["出库"]/df_group["去重订单数"]
#     df_group["订单出库率"]=df_group["订单出库率"].apply(lambda x:format(x,".2%"))

#     return df_group

In [55]:
# df_lhjl = df[(df.tips.str.contains(r'联合拒量', regex=True, na=False)==True)]


# df_lhjl_g = group_data(df_lhjl, '下单日期')
# # df_lhjl_g = df_lhjl_g[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
# #                     "出库前风控强拒","待审核",'出库', '拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_lhjl_g.loc['汇总',:] = df_lhjl_g.select_dtypes(include=[np.number]).sum()
# df_lhjl_g.to_excel('F:/需求/倩姐需求/联合拒量转化.xlsx', index=True)

In [56]:
# df_lhjl.loc[:, '是否逾期'] = np.where(df_lhjl['overdue'] == 0, 0, 1)
# df_lhjl.groupby('下单月份').agg({ '是否逾期': 'sum' })

In [57]:
# def year_sum(df, year):
#     df2 = df[df.年份==year]176
#     df2_ck = df2[df2.是否出库==1]
#     df2_group = df2_ck.groupby('下单月份').agg({'是否出库': 'sum', 'purchase_amount': 'sum', 'new_actual_money': 'sum'}).rename(columns={'是否出库': '出库数', 'purchase_amount': '采购金额', 'new_actual_money': '签约价'})
#     return df2_group
# df_group_2023 = year_sum(df, 2023)
# df_group_2024 = year_sum(df, 2024)
# df_group_2025 = year_sum(df, 2025)
# df_group_2025
# with pd.ExcelWriter(f'F:/需求/七月需求/23-25年出库数据1.xlsx', engine='xlsxwriter') as writer:
#     df_group_2023.to_excel(writer, sheet_name='2023年出库数据')
#     df_group_2024.to_excel(writer, sheet_name='2024年出库数据')
#     df_group_2025.to_excel(writer, sheet_name='2025年出库数据')

# df_new = df[~df.merchant_name.isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机'])]
# df_new_g = df_new[df_new.是否出库==1].groupby('下单月份').agg({'是否出库': 'sum', 'new_actual_money': 'sum'}).rename(columns={'new_actual_money': '签约价', '是否出库': '出库'})
# df_g = df.groupby('下单月份').agg({'是否进件': 'sum'}).rename(columns={'是否进件': '进件'})
# df_group = df_g.merge(df_new_g, on='下单月份', how='left').reset_index()
# df_group[['进件', '出库', '签约价']].set_index('进件')

In [58]:
# # 我们业务开始每个月的出库数据进件数据，进件出库率（出库/进件），每个月的采购价purchase_amount汇总，以及签约价new_actual_money汇总吧
# # 未完成订单的签约价-已收new_actual_money-rent_received
# # 剔除商家拒量的订单
# df = df[~df.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机","喜卓灵新租机"])]
# df_copy = df[['下单月份', 'status', '是否进件' ,'是否出库' ,'purchase_amount' ,'new_actual_money', '已收租金']]
# df_copy
# # 应收未收金额：未完成订单（status in [4,5]）的签约价-已收(new_actual_money-rent_received)
# df_copy.loc[:,'应收未收金额' ] = np.where(df_copy['status'].isin([4,5]), df_copy['new_actual_money']-df_copy['已收租金'], 0)
# # 按下单月份分组选取出库、进件、采购价、签约价、应收未收金额的汇总
# df_res = df_copy.groupby('下单月份').agg({'是否进件': 'sum', '是否出库': 'sum', 'purchase_amount': 'sum', 'new_actual_money': 'sum', '应收未收金额': 'sum'}).rename(columns={'是否进件': '进件', '是否出库': '出库', 'purchase_amount': '采购价','new_actual_money': '签约价'})
# # 计算进件出库率（出库/进件）
# df_res['进件出库率'] = df_res['出库']/df_res['进件']
# # 进件出库率百分比
# df_res['进件出库率'] = df_res['进件出库率'].apply(lambda x: '{:.2%}'.format(x))
# # 写入excel
# # with pd.ExcelWriter(f'F:/需求/倩姐需求/业务量数据.xlsx', engine='openpyxl') as writer:
# #     df_res.to_excel(writer, sheet_name='业务量数据')

## 按小时分组聚合转化

In [59]:
# # 将create_time一天24个小时从0点开始，每2个小时为一组，生成一个新列，表示小时段：0-1，2-3，4-5，...，22-23
# # 1. 确保create_time是datetime类型（若不是，先转换）
# df['create_time'] = pd.to_datetime(df['create_time'])  # 转换为datetime格式
# # 2. 提取小时数（0-23）
# hours = df['create_time'].dt.hour  # 得到每个时间的小时部分
# # 3. 计算时间段（如0-1、2-3...）
# start_hour = (hours // 2) * 2  # 计算每组起始小时（0,2,4,...,22）
# end_hour = start_hour + 1      # 计算每组结束小时（1,3,5,...,23）
# time_periods = start_hour.astype(str) + '-' + end_hour.astype(str)  # 格式化为字符串
# # 4. 将时间段添加到原数据框（核心步骤）
# df['下单时间段'] = time_periods  # 直接赋值，新列会自动加入df

# df2['create_time'] = pd.to_datetime(df2['create_time'])  # 转换为datetime格式
# # 2. 提取小时数（0-23）
# hours = df2['create_time'].dt.hour  # 得到每个时间的小时部分
# # 3. 计算时间段（如0-1、2-3...）
# start_hour = (hours // 2) * 2  # 计算每组起始小时（0,2,4,...,22）
# end_hour = start_hour + 1      # 计算每组结束小时（1,3,5,...,23）
# time_periods = start_hour.astype(str) + '-' + end_hour.astype(str)  # 格式化为字符串
# # 4. 将时间段添加到原数据框（核心步骤）
# df2['下单时间段'] = time_periods  # 直接赋值，新列会自动加入df


# df_hour = all_models.data_group(df, df2, df_risk_examine, '下单时间段')
# df_hour['出库（按下单时间）'] = df_hour['出库']

# df_hour = df_hour[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库（按下单时间）', '拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_hour_new = df_hour.reset_index()
# df_hour_new#.to_excel(f'F:/需求/瑞彤需求/小时分组转化数据_{t_date}.xlsx')

## 搜索渠道、归属渠道细分

In [60]:
df[df.归属渠道=='搜索渠道'].来源渠道.unique()


array(['搜索-租手机', '顶部搜索框的搜索结果页', '搜索下挂-商品', '搜索-租苹果手机', '公域搜索带券',
       '邦道-小程序-手机', '搜索直达banner', '邦道-搜索-苹果17', '搜索-租手表', '搜索-租手机苹果',
       'AC月活10', '搜索下挂券', '租iPhone17', '搜索-租苹果耳机', '搜索-租小米手机', '搜索-租真我手机',
       '搜索-租机', '手机分期购0首付', '搜索-租iQOO手机', '搜索-租vivo手机'], dtype=object)

In [61]:
# df[(df.归属渠道=='顶部搜索框的搜索结果页')&(df.是否出库==1)][['order_number', 'channel_type_id', 'order_type']].dtypes

In [62]:
df.归属渠道.unique()

array(['单人聊天会话中的小程序消息卡片（分享）', '支付宝客户端首页', '搜索渠道', '芝麻租物', '京东渠道', '八派信息',
       '其他渠道场景渠道。', '未知渠道', '我的小程序入口', '支付宝直播', '小程序商家消息（服务提醒）',
       '生活号-J-苹果旗舰', '支付宝社群', '生活号', '生活号-商品组件', '派金花', '京东超值租图标', '宜品花',
       '芝麻信用', '顶部搜索框的搜索结果页DIGUA_APP', '社群-每日信息首页', '扫描二维码', '抖音渠道',
       '首页十二宫格及更多', '订阅消息', '系统桌面图标', '生活号-J-电脑设备', '搜索侠客行-首页DIGUA_APP',
       '赛租机', '社群-专享商品', '支群-社群专享', '第三方 APP（如钉钉）打开，在跳转链接中传入访问来源参数：',
       '社群-群福利', '支群2-社群专属福利', '搜索下挂-商品VIVO_ORDER',
       '顶部搜索框的搜索结果页VIVO_ORDER', '租机宝', '芝麻信用频道',
       '顶部搜索框的搜索结果页XX_SHOP_ORDER', '支付宝 push 消息（同1014）',
       '付费流量(通过商家数字推广平台，灯火等投放的广告)DIGUA_APP', '卡包\r\n', '搜索下挂-商品DIGUA_APP',
       '顶部搜索框的搜索结果页ASUS_ORDER', '曙光计划', '新浪微博', '支付宝客服入口', '支付成功页',
       '邦道-搜索-苹果17VIVO_ORDER', '生活号-J-摄影', '京享租右卡'], dtype=object)

In [63]:
df[df.归属渠道=='新浪微博'][['order_number', 'channel_type_id', 'order_type']]

,order_number,channel_type_id,order_type
63185,A202511171655311043,4,WEIBO_ORDER
63612,A20251117175437905,48,WEIBO_ORDER
66140,A20251117170816134,0,WEIBO_ORDER
66200,A20251117173818632,4,WEIBO_ORDER
66305,A20251117182639293,48,WEIBO_ORDER
72610,A2025111810080192,6,WEIBO_ORDER
73076,A20251118141544298,81,WEIBO_ORDER
73121,A20251118143356662,48,WEIBO_ORDER
85583,A20251120174221691,4,WEIBO_ORDER
85839,A20251120182341309,48,WEIBO_ORDER


In [64]:
# df.loc[:, '机型'] = np.where(df["phone_name"]=='iPhone 17', 'iPhone17'
#                                 ,np.where(df["phone_name"]=='iPhone 17 Pro', "iPhone17Pro"
#                                         ,np.where(df["phone_name"].str.contains(r'iPhone 17 Pro Max'), "iPhone17ProMax"
#                                                 ,np.where(df["phone_name"].str.contains(r'iPhone 16|iPhone16'), "iPhone16"
#                                                         ,np.where(df["phone_name"].str.contains(r'iPhone 15|iPhone15'), "iPhone15"
#                                                                 ,"iPhone Others")))))
# df[df.phone_name.fillna('').str.contains('iPhone')].phone_name.unique()
# df[df.order_number=='A20251108111314190'][['来源渠道','归属渠道', 'channel_type_id', 'order_type']]

# df[(df.来源渠道=='邦道-小程序-手机')&(df.是否进件==1)][['下单日期','order_number','来源渠道','归属渠道', 'channel_type_id', 'order_type']]

## 龙哥需求

In [65]:
# # 11月10日：出库、订单号、归属渠道、机型、phone_name、租赁方案
# df_ck_10 = df[(df.下单日期=='2025-11-11')&(df.是否出库==1)]
# df_ck_10 = df_ck_10[['order_id', '归属渠道', 'phone_name', '租赁方案']]
# df_ck_10.to_excel('F:\需求\总需求/11月11日出库.xlsx', index=False)
# order_list = df_ck_10.order_id.to_list()
# df[(~df.order_id.isin(order_list))&(df.下单日期=='2025-11-10')&(df.是否出库==1)][['order_id', '是否拒量','merchant_name', '归属渠道', 'tips']]

# 总体转化

In [66]:
# 总体数据

df_all = all_models.data_group(df, df2, df_risk_examine, '下单日期')
df_all['出库（按下单时间）'] = df_all['出库']
df_all['出库（按转化时间）'] = df_weekday_zh['All'][:-1]
df_all = df_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）', '拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_all_new = df_all.reset_index()
df_all_new.insert(0, '星期', df_all_new['下单日期'].apply(lambda x: day_name_cn_mapping[x.day_name()]))
df_all_new.insert(1, '月份', df_all_new['下单日期'].astype(str).str.split('-').str[0]+'-'+df_all_new['下单日期'].astype(str).str.split('-').str[1])
# df_all_new.tail()
# df_all_new.loc['汇总', :] = df_all_new.select_dtypes(include=[np.number]).sum()
df_all_new.tail()

,星期,月份,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,拒量出库,进件出库率,总体进件出库率（含拒量）,拒量进件出库率增加,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
11,星期四,2025-11,2025-11-20,6996,2778,39.71%,1687,24.11%,1178,69.83%,510.0,242,268.0,15.89%,37,35,6.0,6,184,196.0,0,10.91%,10.91%,0.00%,2.19%,14.34%,0.36%,2.07%,2.63%
12,星期五,2025-11,2025-11-21,5651,2117,37.46%,1551,27.45%,1066,68.73%,486.0,218,268.0,17.28%,43,39,3.0,16,167,164.0,0,10.77%,10.77%,0.00%,2.77%,14.06%,0.19%,2.51%,2.96%
13,星期六,2025-11,2025-11-22,5132,1906,37.14%,1485,28.94%,1015,68.35%,470.0,206,264.0,17.78%,26,25,9.0,51,153,156.0,0,10.30%,10.30%,0.00%,1.75%,13.87%,0.61%,1.68%,2.98%
14,星期日,2025-11,2025-11-23,5025,1939,38.59%,1433,28.52%,1047,73.06%,386.0,162,224.0,15.63%,18,12,4.0,79,111,130.0,0,7.75%,7.75%,0.00%,1.26%,11.30%,0.28%,0.84%,2.21%
15,星期一,2025-11,2025-11-24,736,348,47.28%,160,21.74%,121,75.62%,39.0,5,34.0,21.25%,0,0,0.0,34,0,0.0,0,0.00%,0.00%,0.00%,0.00%,3.12%,0.00%,0.00%,0.00%


## 月度转化

In [67]:
# df_all.to_excel(f'F:/需求/瑞彤需求/支付宝数据_{t_date}.xlsx')
# df2[((df2['客户取消']==1)|(df2['已退款']==1))&(df2['下单日期']=='2025-09-09')][['order_id','客户取消','取消原因']].shape 
# df[(df['下单日期']=='2025-09-15')&(df['待审核']==1)]
# df_all = all_models.data_group_contain_hl(df, df2, df_risk_examine, '下单月份')
# # df_all['出库（按下单时间）'] = df_all['出库']
# # df_all['出库（按转化时间）'] = df_weekday_zh['All'][:-1]
# df_all = df_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_all_new = df_all.reset_index()
# # df_all_new
# df_all_new.to_excel('F:\需求\总需求\迪瓜APP订单转化.xlsx')

# df_all = all_models.data_group_contain_hl(df, df2, df_risk_examine, 'phone_name')
# df_all = df_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# # 按出库数降序排序
# df_all = df_all.sort_values(by='出库', ascending=False)
# df_all.loc['汇总', : ] = df_all.select_dtypes(include=[np.number]).sum()
# df_all.to_excel('F:\需求\总需求\京东渠道机型订单转化.xlsx')

In [68]:

# print(df[df['activity_name'].isin(["1000单曙光计划"])].shape)
# reject_merchants = ["小蚂蚁租机", "兴鑫兴通讯", "人人享租", "崇胜数码", "喜卓灵租机", "喜卓灵新租机", "云启德曜"]
# print(df[df['merchant_name'].isin(reject_merchants)].shape)
# print(df[df.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略|回捞策略250330命中|联合拒量订单|支付宝联合运营', regex=True)==True].shape)



# 搜索渠道转化

In [69]:
# 统计搜索渠道转app出库数
df_ss_app_g = df.groupby('下单日期').agg({'搜索app出库': 'sum'}).rename(columns={'搜索app出库': 'app出库'})
df_ss_app_g

df_ss = df[df['归属渠道']=='搜索渠道']
df_ss2 = df2[df2['归属渠道']=='搜索渠道']
df_ss_group = all_models.data_group(df_ss, df_ss2, df_risk_examine, '下单日期')
# 基于下单日期拼接df_ss_group与df_ss_app_g
df_ss_group = df_ss_group.merge(df_ss_app_g, on='下单日期', how='left')


# df_ss_group['出库（按下单时间）'] = df_weekday['搜索渠道'][:-1]
df_ss_group['出库（按下单时间）'] = df_ss_group['出库']
df_ss_group['出库（按转化时间）'] = df_weekday_zh['搜索渠道'][:-1]
df_ss_group = df_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','app出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_ss_group_new = df_ss_group.reset_index()
df_ss_group.tail()


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,app出库,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
下单日期,,,,,,,,,,,,,,,,,,,,,,,,
2025-11-20,1660,171,10.30%,420,25.30%,271.0,64.52%,150.0,72.0,78.0,18.57%,6,3,0.0,5,64,70.0,0,15.24%,1.43%,17.14%,0.00%,0.71%,3.86%
2025-11-21,1486,151,10.16%,416,27.99%,269.0,64.66%,147.0,65.0,82.0,19.71%,9,16,0.0,4,53,48.0,1,12.74%,2.16%,15.62%,0.00%,3.85%,3.57%
2025-11-22,1231,108,8.77%,329,26.73%,209.0,63.53%,120.0,48.0,72.0,21.88%,2,9,0.0,13,48,52.0,1,14.59%,0.61%,14.59%,0.00%,2.74%,3.90%
2025-11-23,1217,121,9.94%,338,27.77%,241.0,71.30%,97.0,44.0,53.0,15.68%,0,1,0.0,16,36,42.0,0,10.65%,0.00%,13.02%,0.00%,0.30%,2.96%
2025-11-24,136,18,13.24%,26,19.12%,20.0,76.92%,6.0,0.0,6.0,23.08%,0,0,0.0,6,0,0.0,0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%


In [70]:
# df_ss_29 = df_ss[(df_ss['下单日期']=='2025-10-29')]
# df_qj_ss = pd.read_excel('F:/需求/倩姐需求/搜索20251029进件.xlsx')
# df_qj_order_list = df_qj_ss['id_card_num'].tolist()
# print(len(df_qj_order_list))
# df_ss_29_list = df_ss_29[df_ss_29['是否进件']==1].id_card_num.tolist()
# print(len(df_ss_29_list))
df_ss_app_g

,app出库
下单日期,
2025-11-09,0
2025-11-10,1
2025-11-11,0
2025-11-12,0
2025-11-13,0
2025-11-14,0
2025-11-15,0
2025-11-16,0
2025-11-17,0


In [71]:
# # 筛选出df_ss_29中不在df_qj_order_list中的元素
# df_ss_29_not_in = df_ss_29[(~df_ss_29['id_card_num'].isin(df_qj_order_list))&(df_ss_29['是否进件']==1)]
# df_ss_29_not_in = df_ss_29_not_in[['order_number', 'id_card_num', '归属渠道','merchant_name','channel_type_id','order_type', 'status2']]
# df_ss_29_not_in

In [72]:
# df_qj_ss_not_in = df_qj_ss[~df_qj_ss['id_card_num'].isin(df_ss_29_list)]
# df_qj_ss_not_in = df_qj_ss_not_in[['order_number', 'id_card_num']]
# df_qj_ss_not_in

## 按周分组，搜索渠道非机审拒绝的订单

In [73]:
# df_ss = df[df['归属渠道']=='搜索渠道']
# # 将下单周转换为时间范围格式，例如"2025.8.20-2025.8.24"
# def get_week_range(date):
#     # 获取该日期所在周的周一
#     monday = date - pd.Timedelta(days=date.weekday())
#     # 获取该日期所在周的周日
#     sunday = monday + pd.Timedelta(days=6)
#     # 格式化为"年.月.日-年.月.日"格式
#     return f"{monday.year}.{monday.month}.{monday.day}-{sunday.year}.{sunday.month}.{sunday.day}"

# df_ss_quyy = df_ss[((df_ss['机审强拒'] == 0)&((df_ss['人审拒绝']==1)|(df_ss['客户取消']==1)|(df_ss['无法联系']==1)|(df_ss['出库前风控强拒']==1)))]
# # 按星期设置列：下单周，周一至周日为一周，未满一周按一周算，eg:8月20到8月24，9月1日到9月4日都算一周
# df_ss_quyy['下单周范围'] = df_ss_quyy['下单日期'].apply(get_week_range)

# df_ss_quyy['取消原因2'] = df_ss_quyy['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_ss_quyy['取消原因2'].drop_duplicates()
# df_ss_quyy_group = df_ss_quyy.groupby(['下单周范围', '取消原因2']).agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_ss_quyy_group = df_ss_quyy_group.sort_values(['下单周范围', '取消数'], ascending=[True, False])
# df_ss_quyy_group['取消率'] = df_ss_quyy_group['取消数']/(df_ss_quyy['是否进件'].sum())
# df_ss_quyy_group['取消率'] = df_ss_quyy_group['取消率'].apply(lambda x: format(x, '.2%'))
# # df_ss_quyy_group.to_excel('F:/需求/瑞彤需求/搜索非机审拒绝的取消原因.xlsx', sheet_name='订单取消原因')
# df_ss_quyy_group

In [74]:
# df_ss_jg = df_ss_quyy[df_ss_quyy.取消原因2=='不接受监管设备'][['下单日期','order_number','机型内存']]
# df_ss_jg.to_excel('F:/需求/瑞彤需求/搜索不接受监管设备.xlsx', sheet_name='订单明细',index=False)

# 单人聊天 转化

In [75]:
df_dr = df[df['来源渠道']=='单人聊天会话中的小程序消息卡片（分享）']
df_dr2 = df2[df2['来源渠道']=='单人聊天会话中的小程序消息卡片（分享）']
df_dr_group = all_models.data_group(df_dr, df_dr2, df_risk_examine,  '下单日期')

# df_dr_group['出库（按下单时间）'] = df_weekday['单人聊天会话中的小程序消息卡片（分享）'][:-1]
df_dr_group['出库（按下单时间）'] = df_dr_group['出库']
try:
    df_dr_group['出库（按转化时间）'] = df_weekday_zh['单人聊天会话中的小程序消息卡片（分享）'][:-1]
except:
    df_dr_group['出库（按转化时间）'] = 0
df_dr_group = df_dr_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例",'出库（按下单时间）','出库（按转化时间）','进件出库率','订单出库率']].fillna(0)
df_dr_group_new = df_dr_group.reset_index()

In [76]:
df_dr_group_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,出库（按下单时间）,出库（按转化时间）,进件出库率,订单出库率
11,2025-11-20,213,31,14.55%,47,22.07%,33.0,70.21%,0,0.0,0.00%,0.00%
12,2025-11-21,162,29,17.90%,29,17.90%,17.0,58.62%,1,1.0,3.45%,0.62%
13,2025-11-22,167,20,11.98%,44,26.35%,31.0,70.45%,0,0.0,0.00%,0.00%
14,2025-11-23,152,23,15.13%,36,23.68%,30.0,83.33%,0,0.0,0.00%,0.00%
15,2025-11-24,10,2,20.00%,1,10.00%,0.0,0.00%,0,0.0,0.00%,0.00%


# 芝麻渠道转化

In [77]:
# df_fzm = df[df['归属渠道']!='芝麻租物']
# df_fzm2 = df2[df2['归属渠道']!='芝麻租物']
# df_fzm_group = all_models.data_group(df_fzm, df_fzm2, df_risk_examine, '下单月份')

# # df_fzm_group['出库（按下单时间）'] = df_weekday['芝麻租物'][:-1]
# df_fzm_group['出库（按下单时间）'] = df_fzm_group['出库']
# # df_fzm_group['出库（按转化时间）'] = df_weekday_zh['芝麻租物'][:-1],'出库（按转化时间）'
# df_fzm_group = df_fzm_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_fzm_group_new = df_fzm_group.reset_index()
# df_fzm_group#.tail()


In [78]:
# # 统计租物渠道转app出库数
df_zm_app_g = df.groupby('下单日期').agg({'芝麻app出库': 'sum'}).rename(columns={'芝麻app出库': 'app出库'})
df_zm_app_g

df_zm = df[df['归属渠道']=='芝麻租物']
df_zm2 = df2[df2['归属渠道']=='芝麻租物']
df_zm_group = all_models.data_group(df_zm, df_zm2, df_risk_examine, '下单日期')
# 拼接df_zm_group和df_zm_app_g
df_zm_group = df_zm_group.merge(df_zm_app_g, on='下单日期', how='left')

# df_zm_group['出库（按下单时间）'] = df_weekday['芝麻租物'][:-1]
df_zm_group['出库（按下单时间）'] = df_zm_group['出库']
df_zm_group['出库（按转化时间）'] = df_weekday_zh['芝麻租物'][:-1]
df_zm_group = df_zm_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）', 'app出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_zm_group_new = df_zm_group.reset_index()
df_zm_group.tail()

,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,app出库,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
下单日期,,,,,,,,,,,,,,,,,,,,,,,,
2025-11-20,3252,1743,53.60%,546,16.79%,356,65.20%,190.0,72,118.0,21.61%,22,21,1.0,1,73,81.0,0,13.37%,4.03%,13.19%,0.18%,3.85%,2.24%
2025-11-21,2197,1102,50.16%,502,22.85%,316,62.95%,186.0,65,121.0,24.10%,25,16,0.0,9,71,69.0,0,14.14%,4.98%,12.95%,0.00%,3.19%,3.23%
2025-11-22,2061,1016,49.30%,518,25.13%,320,61.78%,198.0,75,123.0,23.75%,16,12,0.0,26,69,72.0,0,13.32%,3.09%,14.48%,0.00%,2.32%,3.35%
2025-11-23,1911,954,49.92%,489,25.59%,323,66.05%,166.0,52,114.0,23.31%,14,8,0.0,36,56,64.0,0,11.45%,2.86%,10.63%,0.00%,1.64%,2.93%
2025-11-24,332,198,59.64%,61,18.37%,36,59.02%,25.0,4,21.0,34.43%,0,0,0.0,21,0,0.0,0,0.00%,0.00%,6.56%,0.00%,0.00%,0.00%


In [79]:
# df_zm = df[(df['归属渠道']=='芝麻租物')&(df['是否出库']==1)]
# df_zm.loc[:, '已买断'] = np.where((df_zm.status==8)&(df_zm.has_actual==1)&(df_zm.relet_days==0), 1, 0)
# df_zm.loc[:, '已归还'] = np.where((df_zm.status.isin([8, 15, 16]))&(df_zm.has_actual==0)&(df_zm.relet_days==0), 1, 0)
# df_zm.loc[:, '已续租'] = np.where(df_zm.relet_days>0, 1, 0)
# df_zm_group = df_zm.groupby(['下单月份']).agg({'已买断': 'sum', '已归还': 'sum', '已续租': 'sum','是否出库': 'sum'})
# df_zm_group['买断率'] = (df_zm_group['已买断']/df_zm_group['是否出库']).apply(lambda x: format(x, ".2%"))
# df_zm_group['归还率'] = (df_zm_group['已归还']/df_zm_group['是否出库']).apply(lambda x: format(x, ".2%"))
# df_zm_group['续租率'] = (df_zm_group['已续租']/df_zm_group['是否出库']).apply(lambda x: format(x, ".2%"))
# df_zm_group

In [80]:
# df_zm_29 = df_zm[(df_zm['下单日期']=='2025-10-29')]
# df_qj_zm = pd.read_excel('F:/需求/倩姐需求/租物20251029进件.xlsx')
# df_qj_order_list = df_qj_zm['id_card_num'].tolist()
# print(len(df_qj_order_list))
# df_zm_29_list = df_zm_29[df_zm_29['是否进件']==1].id_card_num.tolist()
# print(len(df_zm_29_list))
# df_zm[(df_zm['机审强拒']==1)&(df_zm['下单日期']=='2025-10-23')].shape

In [81]:
# 筛选出df_zm_29中不在df_qj_order_list中的元素
# df_zm_29_not_in = df_zm_29[df_zm_29['是否进件']==1][~df_zm_29['id_card_num'].isin(df_qj_order_list)]
# df_zm_29_not_in[['order_number', 'id_card_num']]
# 筛选出df_qj_zm中不在df_zm_29_list中的元素
# df_qj_zm_not_in = df_qj_zm[~df_qj_zm['id_card_num'].isin(df_zm_29_list)]
# df_qj_zm_not_in[['id_card_num','order_number']].shape

# 纯租物 转化

In [82]:
#逻辑是否需要优化   
df_zw = df[df['来源渠道']=='芝麻信用']
df_zw2 = df2[df2['来源渠道']=='芝麻信用']
df_zw_group = all_models.data_group(df_zw, df_zw2, df_risk_examine, '下单日期')

# df_zw_group['出库（按下单时间）'] = df_ly_ck['芝麻租物'][:-1]
df_zw_group['出库（按下单时间）'] = df_zw_group['出库']
df_zw_group = df_zw_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例",'出库（按下单时间）','进件出库率','订单出库率']]
df_zw_group_new = df_zw_group.reset_index()
df_zw_group_new#.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,出库（按下单时间）,进件出库率,订单出库率
0,2025-11-09,3614,2023,55.98%,547,15.14%,325,59.41%,72,13.16%,1.99%
1,2025-11-10,3990,2174,54.49%,668,16.74%,402,60.18%,94,14.07%,2.36%
2,2025-11-11,3442,1788,51.95%,563,16.36%,351,62.34%,83,14.74%,2.41%
3,2025-11-12,3113,1726,55.44%,430,13.81%,278,64.65%,54,12.56%,1.73%
4,2025-11-13,3167,1735,54.78%,413,13.04%,244,59.08%,67,16.22%,2.12%
5,2025-11-14,3153,1724,54.68%,463,14.68%,280,60.48%,91,19.65%,2.89%
6,2025-11-15,2826,1607,56.86%,401,14.19%,245,61.10%,56,13.97%,1.98%
7,2025-11-16,2818,1576,55.93%,423,15.01%,251,59.34%,68,16.08%,2.41%
8,2025-11-17,3377,1881,55.70%,523,15.49%,317,60.61%,77,14.72%,2.28%
9,2025-11-18,3295,1818,55.17%,488,14.81%,306,62.70%,96,19.67%,2.91%


In [83]:
# df_zm.是否进件.sum()

# 抖音转化

In [84]:
df_dy = df[df['归属渠道']=='抖音渠道']
df_dy2 = df2[df2['归属渠道']=='抖音渠道']
# df_dy_group = all_models.data_group(df_dy, df_dy2, df_risk_examine, '下单日期')
# df_dy_group = clean.continuous_dates(df_dy_group)
# df_dy_group['出库（按下单时间）'] = df_dy_group['出库']
# try:
#     df_dy_group['出库（按转化时间）'] = df_weekday_zh['抖音渠道'][:-1]
# except:
#     df_dy_group['出库（按转化时间）'] = 0
# df_dy_group = df_dy_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","客户取消","取消率","人审拒绝","人审拒绝率","待审核","出库前风控强拒",'出库（按下单时间）','出库（按转化时间）','进件出库率','订单出库率']].fillna(0) 
# df_dy_group_new = df_dy_group.reset_index()


In [85]:
classify_id_list = [125,126,127,185]
dfck.loc[:, '抖音电脑'] = np.where((dfck.归属渠道=='抖音渠道')&(dfck.classify_id.isin(classify_id_list)), 1, 0)
# 按转化日期看抖音电脑品类出库
dfck_dy_pc = pd.crosstab(index=dfck['日期'], columns=dfck['抖音电脑'],margins=True)
# 电脑
df_dy_classify = df_dy[df_dy.classify_id.isin(classify_id_list)]
df_d2y_classify = df_dy2[df_dy2.classify_id.isin(classify_id_list)]
df_dy_group = all_models.data_group(df_dy_classify, df_d2y_classify, df_risk_examine, '下单日期')
df_dy_group = clean.continuous_dates(df_dy_group)
df_dy_group['出库（按下单时间）'] = df_dy_group['出库']
try:
    df_dy_group['出库（按转化时间）'] = dfck_dy_pc[1][:-1]
except:
    df_dy_group['出库（按转化时间）'] = 0
df_dy_group = df_dy_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","客户取消","取消率","人审拒绝","人审拒绝率","待审核","出库前风控强拒",'出库（按下单时间）','出库（按转化时间）', '进件出库率','订单出库率']].fillna(0) 
df_dy_group_new = df_dy_group.reset_index()

In [86]:
# df_dy_group_new = all_models.data_group(df_dy_classify, df_d2y_classify, df_risk_examine, '下单月份')
df_dy_group_new#.tail()

,index,去重订单数,进件前取消,进件前取消率,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,客户取消,取消率,人审拒绝,人审拒绝率,待审核,出库前风控强拒,出库（按下单时间）,出库（按转化时间）,进件出库率,订单出库率
0,2025-11-09,3.0,1.0,33.33%,0.0,0.00%,2.0,66.67%,1.0,50.00%,1.0,0.0,0.00%,1.0,50.00%,0.0,0.0,0.0,0,0.00%,0.00%
1,2025-11-10,8.0,3.0,37.50%,1.0,12.50%,3.0,37.50%,3.0,100.00%,0.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,0,0.00%,0.00%
2,2025-11-11,3.0,1.0,33.33%,2.0,66.67%,0.0,0.00%,0.0,nan%,0.0,0.0,nan%,0.0,nan%,0.0,0.0,0.0,0,nan%,0.00%
3,2025-11-12,4.0,3.0,75.00%,1.0,25.00%,0.0,0.00%,0.0,nan%,0.0,0.0,nan%,0.0,nan%,0.0,0.0,0.0,0,nan%,0.00%
4,2025-11-13,7.0,4.0,57.14%,0.0,0.00%,3.0,42.86%,2.0,66.67%,1.0,1.0,33.33%,0.0,0.00%,0.0,0.0,0.0,0,0.00%,0.00%
5,2025-11-14,19.0,7.0,36.84%,4.0,21.05%,8.0,42.11%,7.0,87.50%,1.0,0.0,0.00%,1.0,12.50%,0.0,0.0,0.0,0,0.00%,0.00%
6,2025-11-15,5.0,0.0,0.00%,1.0,20.00%,4.0,80.00%,4.0,100.00%,0.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,0,0.00%,0.00%
7,2025-11-16,10.0,3.0,30.00%,1.0,10.00%,5.0,50.00%,4.0,80.00%,1.0,1.0,20.00%,0.0,0.00%,0.0,0.0,0.0,0,0.00%,0.00%
8,2025-11-17,8.0,7.0,87.50%,0.0,0.00%,1.0,12.50%,1.0,100.00%,0.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,0,0.00%,0.00%
9,2025-11-18,4.0,2.0,50.00%,0.0,0.00%,2.0,50.00%,2.0,100.00%,0.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,0,0.00%,0.00%


# 支付宝直播转化

In [87]:
# df_zfb2[(df_zfb2['下单日期']=='2025-07-19')&(df_zfb2['是否出库']==1)][['下单日期','order_number','activity_name','是否拒量','是否出库']]
# 先被拒了，然后进入回捞再出库的，不是原先的出库

In [88]:
# df_zfb[(df_zfb['下单日期']=='2025-07-19')&(df_zfb['是否出库']==1)][['下单日期','order_number','activity_name','是否拒量','是否出库']]

In [89]:
# # 支付宝直播
df_zfb = df[df['activity_name'].str.contains('支付宝直播商品')==True]
df_zfb2 = df2[df2['activity_name'].str.contains('支付宝直播商品')==True]
df_zfb_group = all_models.data_group(df_zfb, df_zfb2, df_risk_examine, '下单日期')
df_zfb_group = clean.continuous_dates(df_zfb_group)

# df_zfb_group['出库（按下单时间）'] = df_zfb_ck['支付宝直播商品'][:-1]
df_zfb_group['出库（按下单时间）'] = df_zfb_group['出库']
df_zfb_group['出库（按转化时间）'] = df_zfb_group['出库']
df_zfb_group = df_zfb_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","人审拒绝率","风控通过件","客户取消","取消率",'无法联系',"出库前风控强拒","待审核",'出库（按下单时间）', '出库（按转化时间）','进件出库率','订单出库率']]
df_zfb_group1 = df_zfb_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","人审拒绝率","风控通过件","客户取消","取消率",'无法联系',"出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','订单出库率']].reset_index(names=['下单日期'])
df_zfb_group1.tail()

,下单日期,去重订单数,进件前取消,进件前取消率,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,客户取消,取消率,无法联系,出库前风控强拒,待审核,出库（按下单时间）,进件出库率,订单出库率
11,2025-11-20,23,11,47.83%,10,43.48%,2,8.70%,2.0,100.00%,0.0,0.0,0.00%,0.0,0,0.00%,0,0.0,0,0,0.00%,0.00%
12,2025-11-21,21,11,52.38%,5,23.81%,4,19.05%,3.0,75.00%,1.0,1.0,25.00%,0.0,0,0.00%,0,0.0,0,0,0.00%,0.00%
13,2025-11-22,21,16,76.19%,4,19.05%,1,4.76%,1.0,100.00%,0.0,0.0,0.00%,0.0,0,0.00%,0,0.0,0,0,0.00%,0.00%
14,2025-11-23,14,7,50.00%,4,28.57%,2,14.29%,2.0,100.00%,0.0,0.0,0.00%,0.0,0,0.00%,0,0.0,0,0,0.00%,0.00%
15,2025-11-24,7,6,85.71%,0,0.00%,1,14.29%,1.0,100.00%,0.0,0.0,0.00%,0.0,0,0.00%,0,0.0,0,0,0.00%,0.00%


In [90]:
# df_zfb.下单日期.unique()
df_zfb.来源渠道.unique()

array(['支付宝直播', '未知渠道', '社群-每日信息首页', '顶部搜索框的搜索结果页', '支群-社群专享',
       '其他渠道场景渠道。', '邦道-小程序-手机', '芝麻信用频道', '支付宝客户端首页', '芝麻信用', '社群-群福利',
       '八派信息', '订阅消息', '生活号-J-苹果旗舰', '邦道-搜索-苹果17', '支群2-社群专属福利',
       '搜索下挂-商品', '社群-专享商品', '我的小程序入口', '手机分期购0首付'], dtype=object)

## 支付宝直播进件前取消原因

In [91]:

# df_zfb_jjqqx = df_zfb[(df_zfb['进件前取消']==1)]
# df_zfb_jjqqx['取消原因2'] = df_zfb_jjqqx['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_zfb_jjqqx['取消原因2'].drop_duplicates()
# df_zfb_jjqqx_group = df_zfb_jjqqx.groupby(['取消原因2']).agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_zfb_jjqqx_group['取消率'] = df_zfb_jjqqx_group['取消数']/(df_zfb_jjqqx_group['取消数'].sum())
# df_zfb_jjqqx_group['取消率'] = df_zfb_jjqqx_group['取消率'].apply(lambda x: format(x, '.2%'))
# df_zfb_jjqqx_group.to_excel('F:/需求/崔婷玉需求/支付宝直播进件前订单取消原因2.xlsx', sheet_name='进件前订单取消原因')



## 支付宝直播取消原因、拒绝理由及占比

In [92]:
# df_zfb['取消原因2'] = df_zfb['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_zfb['取消原因2'].drop_duplicates()
# df_zfb_qx_group = df_zfb.groupby(['取消原因2']).agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_zfb_qx_group['取消率'] = df_zfb_qx_group['取消数']/(df_zfb_qx_group['取消数'].sum())
# df_zfb_qx_group['取消率'] = df_zfb_qx_group['取消率'].apply(lambda x: format(x, '.2%'))
# # 按取消数降序排序
# df_zfb_qx_group = df_zfb_qx_group.sort_values(by='取消数', ascending=False)
# df_zfb_jj_group = df_zfb.groupby(['拒绝理由']).agg({'order_number': 'count'}).rename(columns={'order_number': '拒绝数'})
# df_zfb_jj_group['拒绝率'] = (df_zfb_jj_group['拒绝数']/(df_zfb_jj_group['拒绝数'].sum())).apply(lambda x: format(x, '.2%'))
# df_zfb_jj_group = df_zfb_jj_group.sort_values(by='拒绝数', ascending=False)
# with pd.ExcelWriter('F:/需求/崔婷玉需求/支付宝直播取消拒绝理由占比_{t_date}.xlsx') as writer:
#     df_zfb_group1.to_excel(writer, sheet_name='支付宝直播')
#     df_zfb_qx_group.to_excel(writer, sheet_name='取消原因')
#     df_zfb_jj_group.to_excel(writer, sheet_name='拒绝理由')
# df_zfb_qx_group

In [93]:
df1_ck = df_zfb[df_zfb['是否出库']==1]
Today_zfb = datetime.now().strftime('%Y%m%d%H')

# 总体剔除直播数据

In [94]:
df_tc = df_all[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_dr_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_dy_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_zfb_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]
df_tc.fillna(df_all[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_dr_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_zfb_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']], inplace=True)
df_tc['预授权通过率'] = df_tc['进件数'] / df_tc['去重订单数']
df_tc['预授权通过率'] = df_tc['预授权通过率'].apply(lambda x: format(x, '.2%'))
df_tc['强拒比例'] = df_tc['机审强拒'] / df_tc['进件数']
df_tc['强拒比例'] = df_tc['强拒比例'].apply(lambda x: format(x, '.2%'))
df_tc['进件出库率'] = df_tc['出库（按下单时间）'] / df_tc['进件数']
df_tc['进件出库率'] = df_tc['进件出库率'].apply(lambda x: format(x, '.2%'))
df_tc['订单出库率'] = df_tc['出库（按下单时间）'] / df_tc['去重订单数']
df_tc['订单出库率'] = df_tc['订单出库率'].apply(lambda x: format(x, '.2%'))
df_tc = df_tc[['去重订单数', '进件数', '预授权通过率', '机审强拒', '强拒比例', '出库（按下单时间）', '出库（按转化时间）', '进件出库率', '订单出库率']].reset_index()
df_tc.tail()

,下单日期,去重订单数,进件数,预授权通过率,机审强拒,强拒比例,出库（按下单时间）,出库（按转化时间）,进件出库率,订单出库率
11,2025-11-20,6755.0,1635.0,24.20%,1141.0,69.79%,184.0,196.0,11.25%,2.72%
12,2025-11-21,5461.0,1515.0,27.74%,1043.0,68.84%,166.0,163.0,10.96%,3.04%
13,2025-11-22,4941.0,1439.0,29.12%,982.0,68.24%,153.0,156.0,10.63%,3.10%
14,2025-11-23,4858.0,1395.0,28.72%,1015.0,72.76%,111.0,130.0,7.96%,2.28%
15,2025-11-24,719.0,158.0,21.97%,120.0,75.95%,0.0,0.0,0.00%,0.00%


# 京享租 2025-05-20正式上线

In [95]:
df_no_drop_jd = df_no_drop[df_no_drop.归属渠道=='京东渠道']
df_no_drop_jd = df_no_drop_jd[df_no_drop_jd.下单日期>='2025-05-20']
df_no_drop_jd.loc[:, '创建进件数'] = np.where(df_no_drop_jd.进件=='进件', 1, 0)
df_no_drop_jd_g = df_no_drop_jd.groupby('下单日期').agg({'order_id': 'count', '创建进件数': 'sum'}).rename(columns={'order_id': '创建订单数'})
df_no_drop_jdsc_g = df_no_drop_jd[df_no_drop_jd.app_type=='JD_APP'].groupby('下单日期').agg({'order_id': 'count'}).rename(columns={'order_id': '创建订单数'})
# df_no_drop_jd_g.head()


In [96]:
df_jd = df[df['归属渠道']=='京东渠道']
df_jd2 = df2[df2['归属渠道']=='京东渠道']
df_jd = df_jd[df_jd.下单日期>='2025-05-20']
df_jd2 = df_jd2[df_jd2.下单日期>='2025-05-20']
df_jd_group = all_models.data_group(df_jd, df_jd2, df_risk_examine, '下单日期')


df_jd_group['出库（按下单时间）'] = df_jd_group['出库']
try:
    df_jd_group['出库（按转化时间）'] = df_weekday_zh['京东渠道'][:-1]
except:
    df_jd_group['出库（按转化时间）'] = 0
df_jd_group = df_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_jd_group_new = df_jd_group.reset_index()
df_jd_group_new = df_no_drop_jd_g.merge(df_jd_group_new, on='下单日期', how='inner')
df_jd_group_new.tail()

,下单日期,创建订单数,创建进件数,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
11,2025-11-20,1858,675,1350,728,53.93%,578,42.81%,441.0,76.30%,137.0,81.0,56.0,9.69%,8,10,0.0,0,38,39.0,6.57%,1.38%,14.01%,0.00%,1.73%,2.81%
12,2025-11-21,1765,621,1287,733,56.95%,514,39.94%,394.0,76.65%,120.0,68.0,52.0,10.12%,7,6,0.0,3,36,37.0,7.00%,1.36%,13.23%,0.00%,1.17%,2.80%
13,2025-11-22,1641,567,1208,687,56.87%,486,40.23%,381.0,78.40%,105.0,59.0,46.0,9.47%,6,4,0.0,11,25,25.0,5.14%,1.23%,12.14%,0.00%,0.82%,2.07%
14,2025-11-23,1788,603,1310,763,58.24%,504,38.47%,406.0,80.56%,98.0,53.0,45.0,8.93%,3,3,0.0,23,16,20.0,3.17%,0.60%,10.52%,0.00%,0.60%,1.22%
15,2025-11-24,260,82,182,115,63.19%,61,33.52%,56.0,91.80%,5.0,1.0,4.0,6.56%,0,0,0.0,4,0,0.0,0.00%,0.00%,1.64%,0.00%,0.00%,0.00%


In [97]:

# df_jd.机型内存.value_counts()
# 按去重订单数降序
# df_jd_group_new = df_jd_group_new.sort_values(by='去重订单数', ascending=False)

# df_jd_group_new.to_excel('F:/需求/崔婷玉需求/京东所有机型转化数据.xlsx')


## 临时需求

### 京东二手商品转化数据及取消原因，含商家

In [98]:
# df_jd_es = df[(df['归属渠道']=='京东渠道')&(df['商品类型']=='二手')]
# df_jd_es.shape
# df_jd_es2 = df2[(df2['归属渠道']=='京东渠道')&(df2['商品类型']=='二手')&(df2['商品ID'].isin([7979, 7975]))]
# # df_jd_es.loc[:, '日期范围'] = '0809-0934'
# # df_jd_es2.loc[:, '日期范围'] = '0809-0934'
# df_jdes_group = all_models.data_group(df_jd_es, df_jd_es2, df_risk_examine, '下单日期')

# df_jdes_group = df_jdes_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_jdes_group.loc['汇总', :] = df_jdes_group.select_dtypes(include=[np.number]).sum()
# df_jdes_group_new = df_jdes_group.reset_index()
# # 添加汇总行，仅计算数值型

# df_jdes_group_new.tail()#.to_excel('F:/需求/崔婷玉需求/京东二手商品总体转化数据.xlsx')


In [99]:
# df[(df['归属渠道']=='京东渠道')&(df['商品类型']=='二手')].商品ID.unique()

In [100]:
# df_jdes_jj = df_jd_es[(df_jd_es['是否进件']==1)]
# df_jdes_jj['取消原因2'] = df_jdes_jj['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_jdes_jj['取消原因2'].drop_duplicates()
# df_jdes_qx = df_jdes_jj.groupby('取消原因2').agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_jdes_qx['取消率'] = df_jdes_qx['取消数']/(df_jdes_jj['是否进件'].sum())
# df_jdes_qx['取消率'] = df_jdes_qx['取消率'].apply(lambda x: format(x, '.2%'))
# df_jdes_qx#.to_excel('F:/需求/崔婷玉需求/京东二手商品总体机审强拒理由.xlsx')
# with pd.ExcelWriter('F:/需求/崔婷玉需求/京东二手特定商品ID转化数据及取消原因.xlsx', engine='xlsxwriter') as writer:
#     df_jdes_group_new.to_excel(writer, sheet_name='京东二手商品转化数据', index=False)
#     df_jdes_qx.to_excel(writer, sheet_name='京东二手商品取消原因')

In [101]:
# df_jd_qs = df[(df['归属渠道']=='京东渠道')&(df['是否出库']==1)]
# df_jd_qs.shape
# df_jd_qs.loc[:, '订单状态'] = np.where(df_jd_qs['退货拒签']==1, '已退货',np.where((df_jd_qs['status']==4)&(df_jd_qs.overdue==1), '逾期', df_jd_qs['status2']))
# # df_jd_qs.loc[:, '是否逾期'] = np.where((df_jd_qs['status']==4)&(df_jd_qs.overdue==1), '逾期', '未逾期')#(df['退货拒签']==0)&
# # df_jd_qs.loc[:, '是否买断'] = np.where(df_jd_qs.has_actual==1, '买断', '未买断')
# df_jd_qs[['order_number', '订单状态']].to_excel('F:/需求/崔婷玉需求/京东渠道8月出库订单号.xlsx', index=False)

In [102]:
# df_no_drop_jd = df_no_drop[df_no_drop.来源渠道 == '京东618活动']
# df_no_drop_jd.loc[:, '创建进件数'] = np.where(df_no_drop_jd.进件 == '进件', 1, 0)
# df_no_drop_jd_g = df_no_drop_jd.groupby('机型内存').agg({'order_id': 'count', '创建进件数': 'sum'}).rename(columns={'order_id': '创建订单数'})
# df_jd_yk = df[df['来源渠道'] == '京东618活动']
# df_jd2_yk = df2[df2['来源渠道'] == '京东618活动']
# df_jd_group = all_models.data_group(df_jd_yk, df_jd2_yk, df_risk_examine, '机型内存')
# df_jd_group = df_jd_group[
#     ["去重订单数", "前置拦截", "拦截率", "进件数", "预授权通过率", "机审强拒", "强拒比例", "机审通过件",
#         "人审拒绝", "风控通过件", "风控通过率", "客户取消", "无法联系", "出库前风控强拒", "待审核",
#         '出库', '进件出库率', '取消率', '人审拒绝率', '出库前强拒比例',
#         '无法联系占比', '订单出库率']].fillna(0)
# df_jd_group_new = df_jd_group.reset_index()
# df_jd_group_new2 = df_no_drop_jd_g.merge(df_jd_group_new, on='机型内存', how='inner')

# df_jd_group_new2.to_excel('F:\需求\瑞彤需求\京东618活动机型出库数据.xlsx', index=False)


### 统计10.14-10.20京东商城和京东租赁的客户取消的具体原因以及占比

### 统计京东订单，苹果17系列所有机型转化数据

In [103]:
# # df_jd17 = df_jd[df_jd['product_name'].str.contains(r'iPhone 17|iPhone Air')]
# df_jd17 = all_models.data_group(df_jd, df_jd2, df_risk_examine, '机型内存')

# df_jd17 = df_jd17[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_jd17

## 按下单周拉取转化数据

In [104]:
# # 按下单日期所在本周的周四到下周三为一周，eg:，8月7日到8月13日为一周，8月7日到8月13日的下单周范围都为2025.8.7-2025.8.13，由此逻辑设置列：下单周范围
# def get_week_range_v2(date):
#     # 获取该日期所在周的周四
#     # weekday(): 周一=0, 周二=1, ..., 周四=3, ..., 周日=6
#     days_since_thursday = (date.weekday() - 3) % 7
#     thursday = date - pd.Timedelta(days=days_since_thursday)
#     # 获取下周的周三（周四+6天）
#     next_wednesday = thursday + pd.Timedelta(days=6)
#     # 格式化为"年.月.日-年.月.日"格式
#     return f"{thursday.year}.{thursday.month}.{thursday.day}-{next_wednesday.year}.{next_wednesday.month}.{next_wednesday.day}"

# df_no_drop_jd = df_no_drop[df_no_drop.归属渠道=='京东渠道']
# df_jd = df[df['归属渠道']=='京东渠道']
# df_jd2 = df2[df2['归属渠道']=='京东渠道']

# df_no_drop_jd.loc[:, '创建进件数'] = np.where(df_no_drop_jd.进件=='进件', 1, 0)

# df_no_drop_jd.loc[:, '下单周范围'] = df_no_drop_jd['下单日期'].apply(get_week_range_v2)
# df_jd.loc[:, '下单周范围'] = df_jd['下单日期'].apply(get_week_range_v2)
# df_jd2.loc[:, '下单周范围'] = df_jd2['下单日期'].apply(get_week_range_v2)

# # 京东商城
# df_no_drop_jdsc_g = df_no_drop_jd[df_no_drop_jd.app_type=='JD_APP'].groupby('下单日期').agg({'order_id': 'count','创建进件数': 'sum'}).rename(columns={'order_id': '创建订单数'})
# df_jdsc = df_jd[df_jd.app_type=='JD_APP']
# df_jdsc2 = df_jd2[df_jd2.app_type=='JD_APP']
# df_jdsc_group = all_models.data_group(df_jdsc, df_jdsc2, df_risk_examine, '下单日期')
# df_jdsc_group['出库（按下单时间）'] = df_jdsc_group['出库']

# df_jdsc_group = df_jdsc_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_jdsc_group_new = df_jdsc_group.reset_index()
# df_jdsc_group_new = df_no_drop_jdsc_g.merge(df_jdsc_group_new, on='下单日期', how='left')

# # 京东金融
# df_no_drop_jdjr_g = df_no_drop_jd[df_no_drop_jd.app_type=='JR_APP'].groupby('下单日期').agg({'order_id': 'count','创建进件数': 'sum'}).rename(columns={'order_id': '创建订单数'})
# df_jdjr = df_jd[df_jd.app_type=='JR_APP']
# df_jdjr2 = df_jd2[df_jd2.app_type=='JR_APP']
# df_jdjr_group = all_models.data_group(df_jdjr, df_jdjr2, df_risk_examine, '下单日期')
# df_jdjr_group['出库（按下单时间）'] = df_jdjr_group['出库']

# df_jdjr_group = df_jdjr_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_jdjr_group_new = df_jdjr_group.reset_index()
# df_jdjr_group_new = df_no_drop_jdjr_g.merge(df_jdjr_group_new, on='下单日期', how='left')
# df_jdsc_group_new

In [105]:
# path = 'F:\需求\崔婷玉需求/'
# with pd.ExcelWriter(path+f'京东渠道各子渠道出库数据_{t_date}.xlsx', engine='openpyxl') as writer:
#     df_jdsc_group_new.to_excel(writer, sheet_name='京东商城', index=False)
#     df_jdjr_group_new.to_excel(writer, sheet_name='京东金融', index=False)

## 客户取消

In [106]:
# df_jd.columns

### 取消原因1：创建订单后处于订单取消状态

In [107]:
# df_no_drop_jd = df_no_drop[(df_no_drop.归属渠道=='京东渠道')&(df_no_drop['product_name'].str.contains(r'iPhone 17|iPhone Air'))]
# df_no_drop_jd.loc[:, '取消1'] = np.where((df_no_drop_jd['status2'] == '订单取消')&(df_no_drop_jd['order_id'].notna()), 1, 0)
# df_no_drop_jd_group = df_no_drop_jd.groupby('下单日期').agg({'取消1':'sum', 'order_id':'count'}).rename(columns={'取消1':'取消数', 'order_id':'创建订单数'})
# df_no_drop_jd_group['取消率'] = (df_no_drop_jd_group['取消数']/df_no_drop_jd_group['创建订单数']).apply(lambda x: format(x, '.2%'))
# df_no_drop_jd_group

### 取消原因2：进件后，存在机审强拒、人审拒绝、客户取消、无法联系、出库前风控强拒中至少有一个为1

In [108]:
# df_jd = df[(df['归属渠道']=='京东渠道')&(df['product_name'].str.contains(r'iPhone 17|iPhone Air'))]
# df_jd.loc[:, '取消2'] = np.where(((df_jd['机审强拒'] == 1)|(df_jd['人审拒绝']==1)|(df_jd['客户取消']==1)|(df_jd['无法联系']==1)|(df_jd['出库前风控强拒']==1)), 1, 0)
# df_jd_group = df_jd.groupby('下单日期').agg({'取消2':'sum', '是否进件':'sum'}).rename(columns={'取消2':'取消数', '是否进件':'进件数'})
# df_jd_group['取消率'] = (df_jd_group['取消数']/df_jd_group['进件数']).apply(lambda x: format(x, '.2%'))
# df_jd_group

### 取消原因4：审核通过后或有支付流水且状态为已退款

In [109]:
# df_jd.loc[:, '审核通过'] = np.where(df_jd['total_describes'].str.contains('msg=审核通过'), 1, 0)
# df_jd.loc[:, '取消4'] = np.where((df_jd['审核通过']==1)&(df_jd['status2']=='已退款'), 1, 0)
# df_jd_group_sh = df_jd.groupby('下单日期').agg({'取消4':'sum', '审核通过':'sum'}).rename(columns={'取消4':'取消数', '审核通过':'审核通过数'})
# df_jd_group_sh.loc[:, '取消率'] = (df_jd_group_sh['取消数']/df_jd_group_sh['审核通过数']).apply(lambda x: format(x, '.2%'))
# df_jd_group_sh

In [110]:
# # 写入Excel
# with pd.ExcelWriter('F:/需求/瑞彤需求/京东取消原因订单_月份.xlsx', engine='openpyxl') as writer:
#     df_no_drop_jd_group.to_excel(writer, sheet_name='取消原因1')
#     df_jd_group.to_excel(writer, sheet_name='取消原因2')
#     df_jd_group_sh.to_excel(writer, sheet_name='取消原因4')

### 发货前订单取消原因

In [111]:
# df_jd_shtg = df_jd#[(df_jd['风控通过件']==1)&(df_jd['status2']=='已退款')]
# df_jd_shtg.loc[:,'发货前取消'] = np.where((df_jd['status2']=='已退款')&(df_jd['风控通过件']==1)&(df_jd['出库前风控强拒']==0),1,0)
# # 物流状态=5，退货拒签
# # df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
# df_jd_shtg.loc[:,'发货拒签'] = np.where((df_jd['物流状态']==5)&(df_jd['是否出库']==1),1,0)
# df_jd_shtg_group = df_jd_shtg.groupby('下单月份').agg({'发货前取消':'sum', '发货拒签':'sum', '是否进件':'sum'})#.rename(columns={'发货前取消':'发货前取消数', '发货拒签':'发货拒签数'})
# df_jd_shtg_group['发货前取消率'] = (df_jd_shtg_group['发货前取消']/df_jd_shtg_group['是否进件']).apply(lambda x: format(x, ".2%"))
# df_jd_shtg_group['发货拒签率'] = (df_jd_shtg_group['发货拒签']/df_jd_shtg_group['是否进件']).apply(lambda x: format(x, ".2%"))
# df_jd_shtg_group

In [112]:
# df_xxx = df_jd_shtg[['下单日期', '取消原因2', 'order_number']]
# df_xxx#.取消原因2.

In [113]:
# # [df_jd['product_name'].str.contains(r'iPhone 17|iPhone Air')]
# df_jd = df[df['归属渠道']=='京东渠道']
# df_jd = df_jd[df_jd['product_name'].str.contains(r'iPhone 17|iPhone Air')]
# # df_jd_shtg = df_jd[(df_jd['风控通过件']==1)&(df_jd['status2']=='已退款')&(df_jd['出库前风控强拒']==0)]
# # df_jd_shtg = df_jd[(df_jd['是否进件'] == 1) & (df_jd['机审强拒'] == 0) & (df_jd['人审拒绝'] == 0)&(df_jd['status2']=='已退款')&(df_jd['出库前风控强拒']==0)]
# df_jd_shtg = df_jd[(df_jd['是否进件'] == 1)]
# df_jd_shtg['取消原因2'] = df_jd_shtg['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_jd_shtg['取消原因2'].drop_duplicates()
# df_jd_shtg_group = df_jd_shtg.groupby(['下单日期', 'rejected']).agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_jd_shtg_group['取消率'] = df_jd_shtg_group['取消数']/(df_jd_shtg['是否进件'].sum())
# df_jd_shtg_group['取消率'] = df_jd_shtg_group['取消率'].apply(lambda x: format(x, '.2%'))
# df_jd_shtg_group.shape#.to_excel('F:/需求/崔婷玉需求/iPhone17取消原因.xlsx', sheet_name='取消原因')

In [114]:
# # 写入Excel
# with pd.ExcelWriter('F:/需求/瑞彤需求/京东取消订单_月份.xlsx', engine='openpyxl') as writer:
#     df_tkjd_kh_group.to_excel(writer, sheet_name='客户取消')
#     df_jdqj_group.to_excel(writer, sheet_name='机审人审拒绝')
#     df_jd_shtg_group.to_excel(writer, sheet_name='审核通过取消订单')

## 京东商城

In [115]:
df_jdsc = df_jd[df_jd.app_type=='JD_APP']
df_jdsc2 = df_jd2[df_jd2.app_type=='JD_APP']
df_jdsc_group = all_models.data_group(df_jdsc, df_jdsc2, df_risk_examine, '下单日期')
df_jdsc_group['出库（按下单时间）'] = df_jdsc_group['出库']

df_jdsc_group = df_jdsc_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_jdsc_group_new = df_jdsc_group.reset_index()
df_jdsc_group_new = df_no_drop_jdsc_g.merge(df_jdsc_group_new, on='下单日期', how='left')
df_jdsc_group_new.tail()

,下单日期,创建订单数,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
11,2025-11-20,451,378,0,0.00%,368,97.35%,285.0,77.45%,83.0,50.0,33.0,8.97%,4,5,0.0,0,24,6.52%,1.09%,13.59%,0.00%,1.36%,6.35%
12,2025-11-21,438,362,0,0.00%,347,95.86%,261.0,75.22%,86.0,46.0,40.0,11.53%,5,5,0.0,3,27,7.78%,1.44%,13.26%,0.00%,1.44%,7.46%
13,2025-11-22,361,303,0,0.00%,295,97.36%,235.0,79.66%,60.0,33.0,27.0,9.15%,6,1,0.0,7,13,4.41%,2.03%,11.19%,0.00%,0.34%,4.29%
14,2025-11-23,305,242,0,0.00%,231,95.45%,185.0,80.09%,46.0,20.0,26.0,11.26%,2,1,0.0,16,7,3.03%,0.87%,8.66%,0.00%,0.43%,2.89%
15,2025-11-24,47,32,0,0.00%,31,96.88%,28.0,90.32%,3.0,1.0,2.0,6.45%,0,0,0.0,2,0,0.00%,0.00%,3.23%,0.00%,0.00%,0.00%


# 新浪微博

In [116]:
df_no_drop_wb = df_no_drop[df_no_drop.归属渠道=='新浪微博']
df_no_drop_wb = df_no_drop_wb[df_no_drop_wb.下单日期>='2025-11-20']
df_no_drop_wb.loc[:, '创建进件数'] = np.where(df_no_drop_wb.进件=='进件', 1, 0)
df_no_drop_wb_g = df_no_drop_wb.groupby('下单日期').agg({'order_id': 'count', '创建进件数': 'sum'}).rename(columns={'order_id': '创建订单数'})



In [117]:
df_wb = df[df['归属渠道']=='新浪微博']
df_wb2 = df2[df2['归属渠道']=='新浪微博']
df_wb = df_wb[df_wb.下单日期>='2025-11-20']
df_wb2 = df_wb2[df_wb2.下单日期>='2025-11-20']
df_wb_group = all_models.data_group(df_wb, df_wb2, df_risk_examine, '下单日期')


df_wb_group['出库（按下单时间）'] = df_wb_group['出库']
try:
    df_wb_group['出库（按转化时间）'] = df_weekday_zh['新浪微博'][:-1]
except:
    df_wb_group['出库（按转化时间）'] = 0
df_wb_group = df_wb_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_wb_group_new = df_wb_group.reset_index()
df_wb_group_new = df_no_drop_wb_g.merge(df_wb_group_new, on='下单日期', how='inner')
df_wb_group_new#.tail()

,下单日期,创建订单数,创建进件数,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
0,2025-11-20,8,1,7,0,0.00%,1,14.29%,0.0,0.00%,1.0,0.0,1.0,100.00%,0,1,0.0,0,0,0,0.00%,0.00%,0.00%,0.00%,100.00%,0.00%
1,2025-11-21,11,3,9,0,0.00%,3,33.33%,0.0,0.00%,3.0,3.0,0.0,0.00%,0,0,0.0,0,0,0,0.00%,0.00%,100.00%,0.00%,0.00%,0.00%
2,2025-11-22,14,3,9,0,0.00%,3,33.33%,3.0,100.00%,0.0,0.0,0.0,0.00%,0,0,0.0,0,0,0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
3,2025-11-23,15,2,14,3,21.43%,2,14.29%,1.0,50.00%,1.0,0.0,1.0,50.00%,0,0,0.0,1,0,0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
4,2025-11-24,4,2,4,0,0.00%,2,50.00%,1.0,50.00%,1.0,0.0,1.0,50.00%,0,0,0.0,1,0,0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%


# APP订单

In [214]:
df_app = df[df['order_type']=='DIGUA_APP']
# app订单不含拒量商家、二级量、三级量商家
reject_merchants = ["小蚂蚁租机", "兴鑫兴通讯", "人人享租", "崇胜数码", "喜卓灵租机", "喜卓灵新租机", "云启德曜"]
df_app.drop(df_app[df_app['merchant_name'].isin(reject_merchants)].index, inplace=True)

df_app2 = df2[df2['order_type']=='DIGUA_APP']

df_app_g = all_models.data_group(df_app, df_app2, df_risk_examine, '下单日期')
df_app_g = clean.continuous_dates(df_app_g)
df_app_g = df_app_g[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库', '拒量出库', '进件出库率','总体进件出库率（含拒量）','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_app_g_new = df_app_g.reset_index(names=['下单日期'])
df_app_g_new.tail()#[df_app_g_new['下单日期']>='2025-09-01']

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,拒量出库,进件出库率,总体进件出库率（含拒量）,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
11,2025-11-20,28,14,50.00%,9,32.14%,4.0,44.44%,5.0,1.0,4.0,44.44%,0,0,0.0,0,4,0.0,44.44%,44.44%,0.00%,11.11%,0.00%,0.00%,14.29%
12,2025-11-21,24,12,50.00%,10,41.67%,5.0,50.00%,5.0,0.0,5.0,50.00%,0,0,0.0,0,5,0.0,50.00%,50.00%,0.00%,0.00%,0.00%,0.00%,20.83%
13,2025-11-22,15,5,33.33%,5,33.33%,0.0,0.00%,5.0,2.0,3.0,60.00%,1,0,0.0,0,2,0.0,40.00%,40.00%,20.00%,40.00%,0.00%,0.00%,13.33%
14,2025-11-23,8,3,37.50%,3,37.50%,1.0,33.33%,2.0,0.0,2.0,66.67%,0,0,0.0,1,1,0.0,33.33%,33.33%,0.00%,0.00%,0.00%,0.00%,12.50%
15,2025-11-24,2,2,100.00%,0,0.00%,0.0,nan%,0.0,0.0,0.0,nan%,0,0,0.0,0,0,0.0,nan%,nan%,nan%,nan%,nan%,nan%,0.00%


In [120]:
# df_app[df_app['是否出库']==1][['order_number', 'status2', '归属渠道', '是否拒量', 'reason', '是否进件','机审通过件','机审强拒','人审拒绝','风控通过件','merchant_name','order_type','来源渠道']]

# 渠道的退货订单数

In [121]:
# 输出df归属渠道的种类
# df['归属渠道'].unique()
# 获取df中所以退货拒签的订单
df_qudao = df2[df2['是否出库']==1]
# df_qudao = df_qudao[df_qudao.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]#无用
# df_qudao = df_qudao[~df_qudao.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]

# 将归属渠道中的渠道数据依次取出芝麻租物， 搜索渠道，单人会话，S量，抖音渠道，，京东渠道（且前面六个渠道都需排除拒量），拒量(已存在是否拒量)的退货订单数
df_qudao['芝麻租物'] = np.where((df_qudao.归属渠道=='芝麻租物')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
df_qudao['搜索渠道'] = np.where((df_qudao.归属渠道=='搜索渠道')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
df_qudao['单人会话'] = np.where((df_qudao.归属渠道=='单人会话')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
conditions_s = ['八派信息', 'CPS直播', '派金花', '九州信息', '勉丫租', '哈银', '租瓜直播', '美仑美奂',
                        '分期乐1', '分期乐2', '98租超', '推一推', '硬派抖音']
df_qudao['S量'] = np.where((df_qudao.归属渠道.isin(conditions_s))&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
df_qudao['抖音渠道'] = np.where((df_qudao.归属渠道=='抖音渠道')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)

df_qudao['拒量'] = np.where((df_qudao['是否拒量']==1)&(df_qudao.退货拒签==1), 1, 0)

df_qudao['京东渠道'] = np.where((df_qudao.归属渠道=='京东渠道')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
# 将各个渠道的订单数相加
df_qudao_group = df_qudao.groupby('下单日期').agg({'是否出库':'sum','退货拒签':'sum','芝麻租物': 'sum', '搜索渠道': 'sum', '单人会话': 'sum', 'S量': 'sum', '抖音渠道': 'sum', '拒量': 'sum', '京东渠道': 'sum'}).rename(columns={'是否出库': '总体出库订单数', '退货拒签': '拒收订单数'})
df_qudao_group = df_qudao_group.reset_index(names=['下单日期'])

# df_qudao_group#.tail()#.to_excel('F:\日报\日报\拒收.xlsx', index=False, header=True)
df_qudao_group
# df_qudao['下单日期'].unique()

,下单日期,总体出库订单数,拒收订单数,芝麻租物,搜索渠道,单人会话,S量,抖音渠道,拒量,京东渠道
0,2025-11-09,162,8,6,1,0,0,0,0,1
1,2025-11-10,217,12,10,1,0,0,0,0,1
2,2025-11-11,190,3,1,0,0,0,0,0,2
3,2025-11-12,164,4,1,3,0,0,0,0,0
4,2025-11-13,145,4,3,1,0,0,0,0,0
5,2025-11-14,186,4,2,1,0,0,0,0,1
6,2025-11-15,134,4,3,1,0,0,0,0,0
7,2025-11-16,135,2,0,2,0,0,0,0,0
8,2025-11-17,172,3,3,0,0,0,0,0,0
9,2025-11-18,191,4,3,0,0,0,0,0,1


## 验证退货拒签的订单数是否正确

In [122]:
# 计算df_qudao_group所有下单日期的拒收订单数
# df_qudao_group['拒收订单数'].sum()


In [123]:
# sql_th = """
# select om.order_number,om.create_time
# from db_digua_business.t_order om
# left join db_digua_business.t_order_logistics tolog on om.id = tolog.order_id
# where tolog.status =  5
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-06-01'
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-06-30'
# """
# df_th = clean.query(sql_th)
# # 统计订单总数
# df_th


# 免审转化

判断免人审的单可以用t_order_risk表中的risk json串中的is_vip=1和status=0

In [124]:
df_ms = df[(df.is_vip=='1')&(df.status_result=='0')]
df_ms2 = df2[(df2.is_vip=='1')&(df2.status_result=='0')]
df_ms_group = all_models.data_group(df_ms, df_ms2, df_risk_examine,  '下单日期')

df_ms_group = df_ms_group[["进件数","人审拒绝", "客户取消","出库","待审核","进件出库率","取消率","人审拒绝率"]] 
# df_ms_group = df_ms_group[["进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    # "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
df_ms_group_new = df_ms_group.reset_index()
df_ms_group.tail()


,进件数,人审拒绝,客户取消,出库,待审核,进件出库率,取消率,人审拒绝率
下单日期,,,,,,,,
2025-11-20,461,203.0,31,181,5,39.26%,6.72%,44.03%
2025-11-21,452,184.0,34,162,15,35.84%,7.52%,40.71%
2025-11-22,442,182.0,26,143,48,32.35%,5.88%,41.18%
2025-11-23,359,145.0,14,106,72,29.53%,3.90%,40.39%
2025-11-24,40,5.0,0,0,33,0.00%,0.00%,12.50%


## 信审需求（倩姐）

In [125]:

# # df_ms_list = df_ms[(df_ms['下单日期']=='2025-08-18')&(df_ms['机审通过件']==1)]['order_id'].tolist()
# # df2[df2['机审通过件']==1].groupby('下单日期').agg({'order_id':'count'})
# # 免审订单号列表
# df_ms_list = df[(df['下单日期']=='2025-08-18')&(df['机审通过件']==1)&(df.is_vip=='1')&(df.status_result=='0')]['order_id'].tolist()
# len(df_ms_list)
# # df_ms_list
# # 机审通过件中缺少拒量的通过件，信审订单数
# # df2[(df2['下单日期']=='2025-08-18')&(df2['机审通过件']==1)&(~df2['order_id'].isin(df_ms_list))&((df2['is_vip'] == '0'))]['order_id'].count()
# # 机审通过件，不含拒量的通过件
# df2[(df2['下单日期']=='2025-08-18')&(df2['机审通过件']==1)]['order_id'].count()

### 获取机审通过订单中拒量的订单

In [126]:
# def xmy(df, df_risk_examine):
#         '''
#         计算小蚂蚁租机的总体转移，机审强拒，人审拒绝，出库前风控强拒数据
#         :param df: 总体数据的df
#         :param df_risk_examine: 出库前风控强拒数据的df
#         :param model: 传入的分组参数
#         :return: 返回小蚂蚁租机的数据
#         '''
#         # 匹配小蚂蚁（拒绝）数据
#         df_xmy = df[df['merchant_name'].isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机', '喜卓灵新租机'])]
#         # 出库前强拒数据重命名
#         df_risk_examine.rename(columns={'time': 'time_risk_ex', 'status': 'status_risk_ex'}, inplace=True)
#         # 对小蚂蚁数据和出库前强拒数据进行拼接
#         df_risk_examine_all = pd.merge(df_xmy, df_risk_examine, left_on='id_card_num', right_on='id_card', how='inner')
#         # 计算出库前强拒的订单数
#         df_risk_examine_all2 = df_risk_examine_all[(df_risk_examine_all['time_risk_ex'] < df_risk_examine_all['update_time'])]
#         # 进行排序并取到最近的一个订单
#         df_risk_examine_all2 = df_risk_examine_all2.sort_values(['order_id', 'time_risk_ex'], ascending=[True, False]).groupby('order_id').head(1)
#         df_risk_examine_all2 = df_risk_examine_all2[df_risk_examine_all2['status_risk_ex'] == '1']
#         # df_risk_examine_all2_g = df_risk_examine_all2.groupby(model).agg(order_risk_ex=('order_id', 'size'))
#         # df_risk_examine_all2_g
#         # 排除出库前风控强拒的订单
#         df_xmy_new = df_xmy[~df_xmy['order_id'].isin(df_risk_examine_all2['order_id'].to_list())]
#         # 定义人审拒绝订单
#         df_xmy_new['小蚂蚁人审拒绝'] = np.where(df_xmy_new['reason'] != '系统风控拒绝转移', 1, 0)
#         print(" 小蚂蚁人审拒绝：",df_xmy_new[(df_xmy_new['下单日期']=='2025-08-18')&(df_xmy_new['小蚂蚁人审拒绝']==1)]['order_id'].to_list())
#         print(df_xmy_new[(df_xmy_new['下单日期']=='2025-08-18')&(df_xmy_new['小蚂蚁人审拒绝']==1)]['order_id'].count())
#         print("order_risk_ex:", df_risk_examine_all2[(df_risk_examine_all2['下单日期']=='2025-08-18')]['order_id'].to_list())
#         print(df_risk_examine_all2[(df_risk_examine_all2['下单日期']=='2025-08-18')]['order_id'].count())
        
# xmy(df, df_risk_examine)
# df_241205 = df[df.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略|回捞策略250330命中', regex=True)==True]
# # 拒绝
# df_241205 = df_241205[~df_241205.merchant_name.isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机', '喜卓灵新租机'])]
# df_241205.loc[:, '出库前强拒_拒量'] = np.where(((df_241205.qvt_risk == '0')|(df_241205.qvt_risk.isna()))&(df_241205.qvt_result == '1'), 1, 0)
# df_241205[(df_241205['出库前强拒_拒量']==1)&(df_241205['下单日期']=='2025-08-18')]['order_id'].count()


### 信审订单，按下单日期统计

In [127]:
# # 信审订单数
# df_credit = df2[(df2['机审通过件']==1)&((df2['is_vip'] == '0')&(df.status_result=='0'))]#  | (df['is_vip'].isna())
# # 筛选未命中发货需上锁的订单
# # df_credit_examine = df_credit_examine[~(df_credit_examine.tips.str.contains('发货需上锁', na=False))]
# # df_credit = df_credit[~(df_credit.tips.str.contains('发货需上锁', na=False))]
# # # 基于下单日期和id_card_num去重
# df_credit = df_credit.drop_duplicates(subset=['下单日期', 'id_card_num'],keep="first")
# df_credit_group = df_credit.groupby('下单日期').agg({'order_id':'count'})
# # df_creadit_list = df_credit[df_credit['下单日期']=='2025-08-18']['order_id'].tolist()
# # df_credit.tail()
# # len(df_creadit_list)
# df_credit_group

## 各渠道每日免审出库单数统计

In [128]:
# 剔除据量的数据
# dfck_ms = dfck[(~dfck.tips.str.contains(r'策略2412|命中自有模型回捞策略')==True)&(dfck.status!=12)]
dfck_ms = clean.drop_rejected_merchant(dfck)
dfdfck=dfck_ms[(dfck_ms.is_vip=='1')&(dfck_ms.status_result=='0')]

dfdfck_ck = pd.crosstab(dfdfck["下单日期"],dfdfck["归属渠道"],margins=True)[['芝麻租物','搜索渠道']][:-1]

# 删除原先索引
# dfdfck_ck = dfdfck_ck.reset_index(drop=True)
dfdfck_ck.rename(columns={'芝麻租物': '芝麻租物免审出库', '搜索渠道': '搜索免审出库'}, inplace=True)
dfdfck_ck = clean.continuous_dates(dfdfck_ck)
dfdfck_ck = dfdfck_ck.reset_index(names=['下单日期'])[['芝麻租物免审出库', '搜索免审出库']]#.set_index('下单日期')
dfdfck_ck.tail()


归属渠道,芝麻租物免审出库,搜索免审出库
11,71.0,64.0
12,67.0,53.0
13,63.0,47.0
14,47.0,36.0
15,0.0,0.0


## 租物  搜索的免审进件

In [129]:
dfms = df[(df.is_vip=='1')&(df.status_result=='0')]
dfms_jj = pd.crosstab(dfms["下单日期"],dfms["归属渠道"])
# 删除原先索引
dfms_jj = dfms_jj.reset_index(drop=True)
# 添加下单日期
dfms_jj['下单日期'] = df_ms_group_new['下单日期']
dfms_jj = dfms_jj[['下单日期', "芝麻租物","搜索渠道"]]
dfms_jj.rename(columns={'芝麻租物': '芝麻租物免审进件', '搜索渠道': '搜索渠道免审进件'}, inplace=True)
dfms_jj.tail()

归属渠道,下单日期,芝麻租物免审进件,搜索渠道免审进件
11,2025-11-20,186,151
12,2025-11-21,187,149
13,2025-11-22,200,122
14,2025-11-23,166,98
15,2025-11-24,27,6


In [130]:
# 整合
df_ms_new = dfms_jj.copy()
df_ms_new.insert(0, '月份', df_ms_new['下单日期'].astype(str).str.split('-').str[0]+'-'+df_ms_new['下单日期'].astype(str).str.split('-').str[1])
df_ms_new.insert(2, '总体进件', df_all_new['进件数'])
df_ms_new.insert(3, '芝麻租物进件', df_zm_group_new['进件数'])
df_ms_new.insert(4, '免审进件', df_ms_group_new['进件数'])
df_ms_new.loc[:, '免审进件占比'] = df_ms_new['免审进件'] / df_ms_new['总体进件']
df_ms_new.loc[:, '免审进件占比'] = df_ms_new['免审进件占比'].apply(lambda x: format(x, '.2%'))
df_ms_new.loc[:, '芝麻租物免审进件占比'] = df_ms_new['芝麻租物免审进件'] / df_ms_new['芝麻租物进件']
df_ms_new.loc[:, '芝麻租物免审进件占比'] = df_ms_new['芝麻租物免审进件占比'].apply(lambda x: format(x, '.2%'))
df_ms_new.loc[:, '搜索免审进件占比'] = df_ms_new['搜索渠道免审进件'] / df_ss_group_new['进件数']
df_ms_new.loc[:, '搜索免审进件占比'] = df_ms_new['搜索免审进件占比'].apply(lambda x: format(x, '.2%'))
df_ms_new.loc[:, '总体出库'] = df_all_new['出库（按下单时间）']
df_ms_new.loc[:, '芝麻租物出库'] = df_zm_group_new['出库（按下单时间）']
df_ms_new.loc[:, '免审出库'] = df_ms_group_new['出库']
df_ms_new2 = pd.concat([df_ms_new, dfdfck_ck], axis=1)
# df_ms_new2
df_ms_new2.loc[:, '免审转化率'] = df_ms_new2['免审出库'] / df_ms_new2['免审进件']
df_ms_new2.loc[:, '免审转化率'] = df_ms_new2['免审转化率'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '免审出库占比'] = df_ms_new2['免审出库'] / df_ms_new2['总体出库']
df_ms_new2.loc[:, '免审出库占比'] = df_ms_new2['免审出库占比'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '芝麻租物免审出库占比'] = df_ms_new2['芝麻租物免审出库'] / df_ms_new2['芝麻租物出库']
df_ms_new2.loc[:, '芝麻租物免审出库占比'] = df_ms_new2['芝麻租物免审出库占比'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '搜索免审出库占比'] = df_ms_new2['搜索免审出库'] / df_ss_group_new['出库（按下单时间）']
df_ms_new2.loc[:, '搜索免审出库占比'] = df_ms_new2['搜索免审出库占比'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '机审通过'] = df_all_new['机审通过件']
df_ms_new2.loc[:, '非免审进件'] = df_ms_new2['机审通过'] - df_ms_new2['免审进件']
df_ms_new2.loc[:, '非免审出库'] = df_ms_new2['总体出库'] - df_ms_new2['免审出库']
df_ms_new2.loc[:, '非免审转化率'] = df_ms_new2['非免审出库'] / df_ms_new2['非免审进件']
df_ms_new2.loc[:, '非免审转化率'] = df_ms_new2['非免审转化率'].apply(lambda x: format(x, '.2%'))
df_ms_new3 = pd.concat([df_ms_new2, df_ms_group_new], axis=1)
# df_ms_new3.set_index('月份')
df_ms_new3#.tail()

,月份,下单日期,总体进件,芝麻租物进件,免审进件,芝麻租物免审进件,搜索渠道免审进件,免审进件占比,芝麻租物免审进件占比,搜索免审进件占比,总体出库,芝麻租物出库,免审出库,芝麻租物免审出库,搜索免审出库,免审转化率,免审出库占比,芝麻租物免审出库占比,搜索免审出库占比,机审通过,非免审进件,非免审出库,非免审转化率,下单日期,进件数,人审拒绝,客户取消,出库,待审核,进件出库率,取消率,人审拒绝率
0,2025-11,2025-11-09,2042,549,524,235,134,25.66%,42.81%,29.71%,162,72,160,72.0,41.0,30.53%,98.77%,100.00%,100.00%,556.0,32.0,2,6.25%,2025-11-09,524,245.0,32,160,0,30.53%,6.11%,46.76%
1,2025-11,2025-11-10,2274,675,603,279,135,26.52%,41.33%,32.14%,217,98,213,97.0,39.0,35.32%,98.16%,98.98%,100.00%,625.0,22.0,4,18.18%,2025-11-10,603,274.0,44,213,0,35.32%,7.30%,45.44%
2,2025-11,2025-11-11,2076,564,512,229,107,24.66%,40.60%,28.84%,190,84,189,83.0,47.0,36.91%,99.47%,98.81%,100.00%,531.0,19.0,1,5.26%,2025-11-11,512,210.0,47,189,0,36.91%,9.18%,41.02%
3,2025-11,2025-11-12,1790,435,402,158,104,22.46%,36.32%,27.66%,164,56,158,56.0,49.0,39.30%,96.34%,100.00%,100.00%,438.0,36.0,6,16.67%,2025-11-12,402,176.0,35,158,0,39.30%,8.71%,43.78%
4,2025-11,2025-11-13,1530,415,417,164,128,27.25%,39.52%,35.56%,145,66,143,64.0,42.0,34.29%,98.62%,96.97%,100.00%,443.0,26.0,2,7.69%,2025-11-13,417,187.0,50,143,0,34.29%,11.99%,44.84%
5,2025-11,2025-11-14,1564,465,411,180,108,26.28%,38.71%,30.08%,186,92,179,88.0,44.0,43.55%,96.24%,95.65%,100.00%,451.0,40.0,7,17.50%,2025-11-14,411,161.0,27,179,0,43.55%,6.57%,39.17%
6,2025-11,2025-11-15,1425,404,371,150,121,26.04%,37.13%,31.93%,134,56,125,51.0,43.0,33.69%,93.28%,91.07%,100.00%,411.0,40.0,9,22.50%,2025-11-15,371,170.0,28,125,0,33.69%,7.55%,45.82%
7,2025-11,2025-11-16,1295,424,337,164,97,26.02%,38.68%,28.61%,135,68,126,60.0,41.0,37.39%,93.33%,88.24%,100.00%,384.0,47.0,9,19.15%,2025-11-16,337,149.0,35,126,0,37.39%,10.39%,44.21%
8,2025-11,2025-11-17,1658,525,457,206,110,27.56%,39.24%,29.89%,172,78,166,73.0,45.0,36.32%,96.51%,93.59%,100.00%,489.0,32.0,6,18.75%,2025-11-17,457,193.0,40,166,0,36.32%,8.75%,42.23%
9,2025-11,2025-11-18,1588,491,456,180,142,28.72%,36.66%,35.95%,191,97,185,94.0,45.0,40.57%,96.86%,96.91%,100.00%,491.0,35.0,6,17.14%,2025-11-18,456,190.0,32,185,0,40.57%,7.02%,41.67%


## 免审订单转化

In [131]:
# 免审
df_ms1_o = df[(df.is_vip=='1')&(df.status_result=='0')]
df_ms2_o = df2[(df2.is_vip=='1')&(df2.status_result=='0')]
df_ms_o_group = all_models.data_group(df_ms1_o, df_ms2_o, df_risk_examine, '下单日期')
df_ms_o_group = df_ms_o_group[["进件数","人审拒绝","客户取消","出库","待审核","进件出库率","取消率","人审拒绝率",'出库前风控强拒','出库前强拒比例','无法联系','无法联系占比']]
df_ms_o_group_new = df_ms_o_group.reset_index()
df_ms_o_group_new.insert(0, '月份', df_ms_o_group_new['下单日期'].astype(str).str.split('-').str[0]+'-'+df_ms_o_group_new['下单日期'].astype(str).str.split('-').str[1])
# df_ms_o_group_new#.set_index('月份')

#  免押订单转化率 全免押

In [132]:
# 预授权
sql_ysq = '''
    select om.create_time, tmu.id_card_num,top.order_id,om.status
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
    from db_digua_business.t_order_pay top
    left join db_digua_business.t_order om on top.order_id = om.id
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
    left join db_digua_business.t_order_risk tor on om.id = tor.order_id 
    where top.pay_type = 'ZFBYSQ' 
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-10-01'
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-11-03'
    and om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )
    and om.user_mobile is not null 
    # and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    # ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')   
    -- and top.total_freeze_fund_amount is not null
    -- and trade_no is not null
    '''
df_ysq = clean.query(sql_ysq)
df_ysq.shape

(155330, 5)

In [133]:
df_ysq_t = df_ysq.copy()
df_ysq_t["下单日期"]=df_ysq_t["create_time"].dt.date
df_ysq_t["下单日期"]=pd.to_datetime(df_ysq_t["下单日期"],errors="coerce")
df_ysq_t['是否前置拦截'] = np.where((df_ysq_t.result.str.contains('id_card不得为空') )|(df_ysq_t.result.str.contains('mobile校验不通过')) |(df_ysq_t.result.str.contains('name校验不通过'))
                        |(df_ysq_t.result.str.contains('年龄超过49岁或低于18岁'))|(df_ysq_t.result=='风控未通过')|(df_ysq_t.result=='黑名单用户'),1,0)
df_ysq_t.loc[:,"进件"]=np.where((df_ysq_t["status"]==1)|(df_ysq_t["status"]==13),"未进件","进件")
# 去除是否前置拦截=1且进件=未进件的行
df_ans = df_ysq_t[(df_ysq_t['是否前置拦截']==0)|(df_ysq_t['进件']=='进件')]

# 按下单日期与id_card_num去重
df_ans = df_ans.drop_duplicates(subset=["下单日期","id_card_num"],keep="first")
# 按下单日期分组计算订单数
df_ans = df_ans.groupby(["下单日期"]).agg({"order_id": 'count'}).rename(columns={"order_id": "预授权调起人数"}).reset_index()
df_ans

,下单日期,预授权调起人数
0,2025-11-09,3911
1,2025-11-10,4425
2,2025-11-11,4160
3,2025-11-12,4071
4,2025-11-13,3685
5,2025-11-14,3612
6,2025-11-15,3373
7,2025-11-16,3086
8,2025-11-17,3849
9,2025-11-18,3926


In [134]:
# 预授权免押
sql_ysqmy = '''
    select om.create_time, tmu.id_card_num,top.order_id
    from db_digua_business.t_order_pay top
    left join db_digua_business.t_order om on top.order_id = om.id
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id 
    where top.pay_type = 'ZFBYSQ' 
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-10-01'
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-09-26'
    and om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )
    and om.user_mobile is not null 
    # and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    # ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')   
    and top.total_freeze_fund_amount = 0
    and trade_no is not null
    and top.status in (2, 5)
    '''

df_ysqmy = clean.query(sql_ysqmy)
df_ysqmy["下单日期"]=df_ysqmy["create_time"].dt.date
df_ysqmy["下单日期"]=pd.to_datetime(df_ysqmy["下单日期"],errors="coerce")
df_ysqmy = df_ysqmy.drop_duplicates(subset=["下单日期","id_card_num"],keep="first")
df_ysqmy = df_ysqmy.groupby(["下单日期"]).agg({"order_id": 'count'}).rename(columns={"order_id": "全免押"}).reset_index()
# df_ysqmy

In [135]:
df_qmy = df[df['押金类型']=='全免押']
df_qmy2 = df2[df2['押金类型']=='全免押']
df_qmy_group = all_models.data_group(df_qmy, df_qmy2, df_risk_examine, '下单日期')
df_qmy_group['出库（按下单时间）'] = df_my['全免押'][:-1]
df_qmy_group = df_qmy_group[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","待审核", '出库（按下单时间）', '进件出库率', '取消率', '人审转化率', '人审拒绝率']].fillna(0)
df_qmy_group_new = df_qmy_group.reset_index()
df_qmy_group_new.insert(1, '总进件', df_all_new['进件数'])
# df_qmy_group_new.insert(2, '免押进件预授权占比', (df_ysqmy.全免押/df_ans.预授权调起人数).map(lambda x: format(x, '.2%')))
df_qmy_group_new.insert(2, '免押进件预授权占比', (df_ysqmy.全免押 / df_ans.预授权调起人数).apply(lambda x: format(x, '.2%') if pd.notna(x) else None))
# df_qmy_group_new.set_index('下单日期')
df_qmy_group_new#.tail()

,下单日期,总进件,免押进件预授权占比,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,待审核,出库（按下单时间）,进件出库率,取消率,人审转化率,人审拒绝率
0,2025-11-09,2042,28.74%,1993,1437,72.10%,556.0,285,271.0,13.60%,33,0,162.0,8.13%,1.66%,29.14%,14.30%
1,2025-11-10,2274,28.68%,2191,1568,71.57%,623.0,304,319.0,14.56%,49,0,218.0,9.81%,2.24%,34.51%,13.87%
2,2025-11-11,2076,25.22%,2005,1475,73.57%,530.0,239,291.0,14.51%,50,0,191.0,9.43%,2.49%,35.66%,11.92%
3,2025-11-12,1790,23.19%,1696,1258,74.17%,438.0,202,236.0,13.92%,41,0,165.0,9.67%,2.42%,37.44%,11.91%
4,2025-11-13,1530,25.07%,1477,1035,70.07%,442.0,209,233.0,15.78%,58,0,151.0,9.82%,3.93%,32.81%,14.15%
5,2025-11-14,1564,25.64%,1483,1032,69.59%,451.0,192,259.0,17.46%,35,0,187.0,12.54%,2.36%,41.24%,12.95%
6,2025-11-15,1425,25.14%,1342,931,69.37%,411.0,194,217.0,16.17%,35,0,136.0,9.99%,2.61%,32.60%,14.46%
7,2025-11-16,1295,26.51%,1210,826,68.26%,384.0,178,206.0,17.02%,43,0,136.0,11.16%,3.55%,35.16%,14.71%
8,2025-11-17,1658,26.16%,1561,1072,68.67%,489.0,225,264.0,16.91%,46,0,180.0,11.02%,2.95%,35.17%,14.41%
9,2025-11-18,1588,25.34%,1500,1010,67.33%,490.0,218,272.0,18.13%,38,0,192.0,12.73%,2.53%,38.98%,14.53%


# 非免押（部分免押）

In [136]:
# 预授权非免押
sql_ysqfmy = '''
    select om.create_time, tmu.id_card_num,top.order_id
    from db_digua_business.t_order_pay top
    left join db_digua_business.t_order om on top.order_id = om.id
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id 
    where top.pay_type = 'ZFBYSQ' 
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-10-01'
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-09-26'
    and om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )
    and om.user_mobile is not null 
    # and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    # ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')   
    and top.total_freeze_fund_amount > 0
    and trade_no is not null
    and top.status in (2, 5)
    '''
df_ysqfmy = clean.query(sql_ysqfmy)

df_ysqfmy["下单日期"]=df_ysqfmy["create_time"].dt.date
df_ysqfmy["下单日期"]=pd.to_datetime(df_ysqfmy["下单日期"],errors="coerce")
df_ysqfmy = df_ysqfmy.drop_duplicates(subset=["下单日期","id_card_num"],keep="first")
df_ysqfmy = df_ysqfmy.groupby(["下单日期"]).agg({"order_id": 'count'}).rename(columns={"order_id": "部分免押"}).reset_index()
# df_ysqfmy

In [137]:
df_fmy = df[df['押金类型']=='部分免押']
df_fmy2 = df2[df2['押金类型']=='部分免押']
df_fmy_group = all_models.data_group(df_fmy, df_fmy2, df_risk_examine, '下单日期')
df_fmy_group['待审核'] = 0
df_fmy_group = df_fmy_group[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消", '待审核']]
try:
    df_fmy_group.loc[:, '出库'] = df_my['部分免押'][:-1]
except:
    df_fmy_group.loc[:, '出库'] = 0
df_fmy_group_new = df_fmy_group.reset_index()
df_fmy_group_new.insert(1, '总进件', df_all_new['进件数'])
df_fmy_group_new.insert(2, '非免押进件预授权占比', (df_ysqfmy.部分免押/df_ans.预授权调起人数).map(lambda x: format(x, '.2%')))
df_fmy_group_new.loc[:, '进件出库率'] = (df_fmy_group_new.出库/df_fmy_group_new.进件数).map(lambda x: format(x, '.2%'))
df_fmy_group_new.loc[:, '取消比例'] = (df_fmy_group_new.客户取消/df_fmy_group_new.进件数).map(lambda x: format(x, '.2%'))
df_fmy_group_new.loc[:, '人审转化率'] = (df_fmy_group_new.出库/df_fmy_group_new.机审通过件).map(lambda x: format(x, '.2%'))
df_fmy_group_new.loc[:, '人审拒绝率'] = (df_fmy_group_new.人审拒绝/df_fmy_group_new.进件数).map(lambda x: format(x, '.2%'))
df_fmy_group_new#.tail()
# df_fmy_group_new#.set_index('下单日期')

,下单日期,总进件,非免押进件预授权占比,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,待审核,出库,进件出库率,取消比例,人审转化率,人审拒绝率
0,2025-11-09,2042,1.48%,49,49.0,100.00%,0.0,0.0,0.0,0.00%,0,0,0.0,0.00%,0.00%,nan%,0.00%
1,2025-11-10,2274,2.12%,83,81.0,97.59%,2.0,0.0,2.0,2.41%,0,0,2.0,2.41%,0.00%,100.00%,0.00%
2,2025-11-11,2076,2.12%,71,70.0,98.59%,1.0,0.0,1.0,1.41%,0,0,1.0,1.41%,0.00%,100.00%,0.00%
3,2025-11-12,1790,2.65%,94,94.0,100.00%,0.0,0.0,0.0,0.00%,0,0,0.0,0.00%,0.00%,nan%,0.00%
4,2025-11-13,1530,1.95%,53,52.0,98.11%,1.0,0.0,1.0,1.89%,1,0,0.0,0.00%,1.89%,0.00%,0.00%
5,2025-11-14,1564,2.85%,81,81.0,100.00%,0.0,0.0,0.0,0.00%,0,0,0.0,0.00%,0.00%,nan%,0.00%
6,2025-11-15,1425,2.99%,83,83.0,100.00%,0.0,0.0,0.0,0.00%,0,0,0.0,0.00%,0.00%,nan%,0.00%
7,2025-11-16,1295,3.43%,85,85.0,100.00%,0.0,0.0,0.0,0.00%,0,0,0.0,0.00%,0.00%,nan%,0.00%
8,2025-11-17,1658,3.04%,97,97.0,100.00%,0.0,0.0,0.0,0.00%,0,0,0.0,0.00%,0.00%,nan%,0.00%
9,2025-11-18,1588,2.78%,88,87.0,98.86%,1.0,0.0,1.0,1.14%,0,0,0.0,0.00%,0.00%,0.00%,0.00%


In [138]:
# 将df_qmy_group_new，df_fmy_group_new写入excel中
# path = 'F:/需求/倩姐需求/'
# with pd.ExcelWriter(path + '预授权.xlsx') as writer:
#     df_qmy_group_new.to_excel(writer, sheet_name='免押', startrow=0, index=True)
#     df_fmy_group_new.to_excel(writer, sheet_name='非免押', startrow=0, index=True)


# 取消拒绝原因

In [139]:
# # 惜仪需求
# path_result = 'F:/myfile/数据提取及分析/'
# excel_name = f'取消&拒绝原因_{t_date}.xlsx'
# dfms = df2
# # 取消原因 总体
# qx_zt_df = pd.crosstab(dfms["下单日期"],dfms["取消原因"],margins=True)

# # 拒绝原因总体
# jj_zt_df = pd.crosstab(dfms["下单日期"],dfms["电审拒绝原因"],margins=True)
# #免审进件的电审拒绝原因
# dfms=df2[(df2["decision_result"].str.contains(pat="免人审",regex=False)==True)&(df2.is_vip=='1')&(df2.status_result=='0')]
# jj_ms_df = pd.crosstab(dfms["下单日期"],dfms["电审拒绝原因"],margins=True)
# #免审进件的取消原因
# dfms=df2[(df2["decision_result"].str.contains(pat="免人审",regex=False)==True)&(df2.is_vip=='1')&(df2.status_result=='0')]
# qx_ms_df = pd.crosstab(dfms["下单日期"],dfms["取消原因"],margins=True)

# with pd.ExcelWriter(path_result + excel_name, engine='xlsxwriter') as writer:  
#     qx_zt_df.to_excel(writer, sheet_name='取消原因-总体', startrow=0, index=True)
#     jj_zt_df.to_excel(writer, sheet_name='拒绝原因-总体', startrow=0, index=True) 
#     jj_ms_df.to_excel(writer, sheet_name='拒绝原因-免审', startrow=0, index=True) 
#     qx_ms_df.to_excel(writer, sheet_name='取消原因-免审', startrow=0, index=True) 

In [140]:
df2.取消原因.value_counts()

取消原因
不同意多预收租金             23
租金过高/租金不合理           19
买断价过高/买断价不合理         10
不同意预收首期租金             5
客户不愿告知原因，申请工单取消       4
其他                    3
已购买手机                 2
不接受溢价费                2
对活动理解错误               2
预付租金余额不足/过段时间重新下单     2
Name: count, dtype: int64

## 取消原因2&拒绝理由

In [141]:
# from datetime import datetime, timedelta
# # 获取昨日时间并转化为字符串
# from datetime import datetime  

# df2['取消原因2'] = df2['cancel_reason'].str.split('：')
# df2['取消原因2'] = df2['取消原因2'].apply(lambda x: x[-1] if x is not None else x)
# y_date = datetime.today() - timedelta(days=1)

# y_date = y_date.strftime('%Y-%m-%d')
# df2_new = df2[(df2['下单日期'] == y_date)&(df2['拒绝理由'].notna())]
# # 切割拒绝理由内容，取第一个逗号前的内容
# # df2_new['拒绝理由'] = df2_new['拒绝理由'].str.split(',').str[0]
# df_quju = df2_new
# # 取消原因_总体
# qx_zt_df = pd.crosstab(df_quju["下单日期"],df_quju["取消原因2"],margins=False)
# # 拒绝理由_总体
# jj_zt_df = pd.crosstab(df_quju["下单日期"],df_quju["拒绝理由"],margins=False)

# # jj_zt_df[jj_zt_df['下单日期'] == '2025-07-15']

# qx_zt_df
# df_quju.groupby('取消原因2').agg({'order_id': 'count'}).sort_values(by='order_id', ascending=False).rename(columns={'order_id': '订单数'})


# 商家转化数据

In [142]:
df_contain["待审核"] = np.where(df_contain["审核状态"]=='待审核',1,0)
df_contain["前置拦截"] = np.where(df_contain["审核状态"]=='前置拦截',1,0)
df_contain["机审强拒"] = np.where(df_contain["审核状态"]=='机审强拒',1,0)
df_contain["人审拒绝"] = np.where(df_contain["审核状态"]=='人审拒绝',1,0)
df_contain["客户取消"] = np.where(df_contain["审核状态"]=='客户取消',1,0)
df_contain["出库前风控强拒"] = np.where(df_contain["审核状态"]=='出库前风控强拒',1,0)
df_contain["无法联系"] = np.where(df_contain["审核状态"]=='无法联系',1,0)
df_contain["是否进件"] = np.where(df_contain["进件"]=='进件',1,0)
df_contain["是否出库"] = np.where(df_contain["status"].isin([2,3,4,5,6,8,15]),1,0)

df_contain["进件前取消"] = np.where(df_contain["审核状态"]=='进件前取消',1,0)

## 澄心优租

In [143]:
# 澄心优租
cxyz = all_models.merchant_names(df_contain, '澄心优租', '下单日期')
cxyz = cxyz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","订单出库率"]]
cxyz = clean.continuous_dates(cxyz)
cxyz_new = cxyz.reset_index(names=['下单日期'])

# try:
#     cxyz = all_models.merchant_names(df_contain, '澄心优租', '下单日期')
#     if cxyz.empty:
#         print("warn: 没有数据")
#         cxyz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','人审拒绝','人审拒绝率','待审核','出库','进件出库率','订单出库率'])
#     else:
#         cxyz = cxyz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","订单出库率"]]
#         cxyz = clean.continuous_dates(cxyz)
#         cxyz_new = cxyz.reset_index(names=['下单日期'])
# except:
#     cxyz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','人审拒绝','人审拒绝率','待审核','出库','进件出库率','订单出库率'])
cxyz_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,人审拒绝,人审拒绝率,待审核,出库,进件出库率,订单出库率
11,2025-11-20,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0,0
12,2025-11-21,1.0,0.0,0.00%,0.0,0.00%,0.0,0.0,nan%,0.0,0.0,nan%,0.00%
13,2025-11-22,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0,0
14,2025-11-23,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0,0
15,2025-11-24,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0,0


## 优优2店

In [144]:
# 优优2店
# yy2 = all_models.merchant_names(df_contain,'优优2店', '下单日期')
# yy2 = yy2[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# yy2


## 北京海鸟窝科技有限公司

In [145]:
# 北京海鸟窝科技有限公司
hnw = all_models.merchant_names(df_contain,'北京海鸟窝科技有限公司', '下单日期')
hnw = hnw[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]
hnw = clean.continuous_dates(hnw)
hnw_new = hnw.reset_index(names=['下单日期'])
# 修改后更具有健壮性
# try:
#     hnw = all_models.merchant_names(df_contain, '北京海鸟窝科技有限公司', '下单日期')
#     if hnw.empty:
#         print("warn: 没有数据")
#         hnw_new = pd.DataFrame(columns=['去重订单数','进件数','预授权通过率','出库','进件出库率'])
#     else:
#         hnw = hnw[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]
#         hnw = clean.continuous_dates(hnw)
#         hnw_new = hnw.reset_index(names=['下单日期'])
# except:
#     hnw_new = pd.DataFrame(columns=['去重订单数','进件数','预授权通过率','出库','进件出库率'])
hnw_new.tail()


,下单日期,去重订单数,进件数,预授权通过率,出库,进件出库率
11,2025-11-20,0.0,0.0,0,0.0,0
12,2025-11-21,0.0,0.0,0,0.0,0
13,2025-11-22,1.0,0.0,0.00%,0.0,nan%
14,2025-11-23,0.0,0.0,0,0.0,0
15,2025-11-24,0.0,0.0,0,0.0,0


## 租着用电脑数码

In [146]:
# 租着用电脑数码
# zzy = all_models.merchant_names(df_contain,'租着用电脑数码', '下单日期')
# zzy = zzy[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]
# zzy = clean.continuous_dates(zzy)
# zzy_new = zzy.reset_index(names=['下单日期'])

try:
    zzy = all_models.merchant_names(df_contain, '租着用电脑数码', '下单日期')
    if zzy.empty:
        print("warn: 没有数据")
        zzy_new = pd.DataFrame(columns=['去重订单数','进件数','预授权通过率','出库','进件出库率'])
    else:
        zzy = zzy[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]
        zzy = clean.continuous_dates(zzy)
        zzy_new = zzy.reset_index(names=['下单日期'])
except:
    zzy_new = pd.DataFrame(columns=['去重订单数','进件数','预授权通过率','出库','进件出库率'])
zzy_new.tail()

,下单日期,去重订单数,进件数,预授权通过率,出库,进件出库率
11,2025-11-20,9,4,44.44%,0,0.00%
12,2025-11-21,7,3,42.86%,0,0.00%
13,2025-11-22,4,3,75.00%,0,0.00%
14,2025-11-23,4,2,50.00%,0,0.00%
15,2025-11-24,4,2,50.00%,0,0.00%


## 格木木二奢名品

In [147]:
# 格木木二奢名品
gmm = all_models.merchant_names(df_contain,'格木木二奢名品', '下单日期')
gmm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
下单日期,,,,,,,,,,,,,,


## 广州康基贸易有限公司

In [148]:
# 广州康基贸易有限公司
kjmy = all_models.merchant_names(df_contain,'广州康基贸易有限公司', '下单日期')
kjmy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
下单日期,,,,,,,,,,,,,,


## 趣智数码

In [149]:
# 趣智数码
# qzsm = all_models.merchant_names(df_contain,'趣智数码', '下单日期')
# qzsm = qzsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# qzsm = clean.continuous_dates(qzsm)
# qzsm_new = qzsm.reset_index(names=['下单日期'])

try:
    qzsm = all_models.merchant_names(df_contain, '趣智数码', '下单日期')
    if qzsm.empty:
        print("warn: 没有数据")
        qzsm_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
    else:
        qzsm = qzsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        qzsm = clean.continuous_dates(qzsm)
        qzsm_new = qzsm.reset_index(names=['下单日期'])
except:
    qzsm_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
qzsm_new.tail()


,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
11,2025-11-20,1.0,0.0,0.00%,1.0,100.00%,1.0,0.0,0.0,0.00%,0.0,0.0,0.00%,0.00%,0.00%
12,2025-11-21,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
13,2025-11-22,1.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
14,2025-11-23,2.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
15,2025-11-24,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0


## 汇客好租

In [150]:
# # 汇客好租
# hkhz = all_models.merchant_names(df_contain,'汇客好租', '下单日期')
# hkhz = hkhz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# hkhz = clean.continuous_dates(hkhz)
# hkhz_new = hkhz.reset_index(names=['下单日期'])
# # 电脑
# df_contain_dy = df_contain[(df_contain.归属渠道=='抖音渠道')&(df_contain.classify_id.isin([125,126,127,185]))]
# hkhz_dy = all_models.merchant_names(df_contain_dy,'汇客好租', '下单日期')
# hkhz_dy = hkhz_dy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# hkhz_dy = clean.continuous_dates(hkhz_dy)
# hkhz_dy_new = hkhz_dy.reset_index(names=['下单日期'])


try:
    # 汇客好租
    hkhz = all_models.merchant_names(df_contain, '汇客好租', '下单日期')
    if hkhz.empty:
        print("warn: 没有数据")
        hkhz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
    else:
        hkhz = hkhz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        hkhz = clean.continuous_dates(hkhz)
        hkhz_new = hkhz.reset_index(names=['下单日期'])
except:
    hkhz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])

try:
    # 电脑
    df_contain_dy = df_contain[(df_contain.归属渠道=='抖音渠道')&(df_contain.classify_id.isin([125,126,127,185]))]
    hkhz_dy = all_models.merchant_names(df_contain_dy, '汇客好租', '下单日期')
    if hkhz_dy.empty:
        print("warn: 没有数据")
        hkhz_dy_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
    else:
        hkhz_dy = hkhz_dy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        hkhz_dy = clean.continuous_dates(hkhz_dy)
        hkhz_dy_new = hkhz_dy.reset_index(names=['下单日期'])
except:
    hkhz_dy_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])

hkhz_new.tail()

warn: 没有数据


,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
11,2025-11-20,4.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
12,2025-11-21,4.0,0.0,0.00%,3.0,75.00%,3.0,0.0,2.0,66.67%,0.0,0.0,0.00%,0.00%,0.00%
13,2025-11-22,3.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
14,2025-11-23,1.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
15,2025-11-24,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0


In [151]:
hkhz_dy_new

,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率


In [152]:
# df_contain.merchant_name.value_counts()
# hkhz_yf = all_models.merchant_names(df_contain, '汇客好租', '下单月份')
# hkhz_yf
# df_contain[df_contain.merchant_name=='汇客好租'].shape
# hkhz = all_models.merchant_names(df_contain, '汇客好租', '下单日期')
# hkhz

### 临时需求：获取抖音订单的已发货客户信息

In [153]:
# # 抖音订单：order_type = 'DY_ORDER'
# # 已发货 status notin (1,2,10,12,13)其中4为租赁中
# df_hkhz_dy = df_contain[(df_contain.merchant_name=='汇客好租')&(df_contain.order_type=='DY_ORDER')]
# # 剔除status为1,2,10,12,13的订单
# df_hkhz_dy = df_hkhz_dy[~df_hkhz_dy.status.isin([1,2,10,12,13])]
# df_hkhz_dy.loc[:, '订单状态'] = np.where(df_hkhz_dy.status==4, '租赁中', '已发货')
# df_hkhz_dy.loc[:, '是否逾期'] = np.where((df_hkhz_dy.overdue==1)&(df_hkhz_dy.status==4), '逾期', '未逾期')
# df_hkhz_dy.loc[:, '订单类型'] = np.where(df_hkhz_dy.order_type=='DY_ORDER', '抖音订单', '非抖音订单')
# # df_hkhz_dy_new = df_hkhz_dy[['merchant_name','订单类型','order_number', '订单状态', 'true_name', 'user_mobile', 'id_card_num', '是否逾期', '下单日期']].rename(columns={'merchant_name': '商户名称', 'order_number': '订单号', 'true_name': '姓名', 'user_mobile': '手机号', 'id_card_num': '身份证号'})
# # df_hkhz_dy_new.to_excel(f'F:/需求/倩姐需求/汇客好租抖音订单_{t_date}.xlsx', index=False)

In [154]:
# df_hkhz = df_contain[(df_contain.merchant_name=='汇客好租')]
# # 获取租赁中且逾期的订单
# df_hkhz_yq = df_hkhz[(df_hkhz.status==4)&(df_hkhz.overdue==1)]
# df_hkhz_yq.loc[:,'逾期金额'] = df_hkhz_yq.new_actual_money - df_hkhz_yq.已收租金 - df_hkhz_yq.已付买断尾款
# # 按下单月份分组统计逾期订单数和逾期金额
# df_hkhz_yq_group = df_hkhz_yq.groupby(df_hkhz_yq['下单月份']).agg({'order_number': 'count', '逾期金额': 'sum'}).rename(columns={'order_number': '逾期订单数'}).reset_index()
# df_hkhz_yq_group#.to_excel(f'F:/需求/倩姐需求/汇客好租逾期订单_{t_date}.xlsx', index=False)
# df_hkhz_yq[['order_number','下单日期','new_actual_money','已收租金','已付买断尾款','逾期金额']]
# df_hkhz_yq['new_actual_money'].mean()

## 小蚂蚁租机

In [155]:
# 小蚂蚁租机
xmy = all_models.merchant_names(df_contain,'小蚂蚁租机', '下单日期')
xmy = xmy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
if len(xmy)!=0:
    xmy = clean.continuous_dates(xmy)
xmy_new = xmy.reset_index(names=['下单日期'])


## 乙辉数码

In [156]:
# 乙辉数码
yhsm = all_models.merchant_names(df_contain,'乙辉数码', '下单日期')
yhsm = yhsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# yhsm = clean.continuous_dates(yhsm)
yhsm_new = yhsm.reset_index()
yhsm_new.dtypes


下单日期      datetime64[ns]
去重订单数              int64
前置拦截             float64
拦截率              float64
进件数              float64
预授权通过率           float64
机审通过件            float64
客户取消             float64
人审拒绝             float64
人审拒绝率            float64
待审核              float64
出库               float64
进件出库率            float64
取消率              float64
订单出库率            float64
dtype: object

## 兴鑫兴通讯

In [157]:
# 兴鑫兴通讯
xxx = all_models.merchant_names(df_contain,'兴鑫兴通讯', '下单日期')
xxx = xxx[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
xxx_new = xxx.reset_index()
# xxx_new

## 呱子笔记本电脑

In [158]:
# 呱子笔记本电脑
# gz = all_models.merchant_names(df_contain,'呱子笔记本电脑', '下单日期')
# gz = gz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# gz = clean.continuous_dates(gz)
# gz_new = gz.reset_index(names=['下单日期'])
# gz_new.set_index('下单日期')

## 南京聚格网络科技

In [159]:
jgwl = all_models.merchant_names(df_contain,'南京聚格网络科技', '下单日期')
jgwl = jgwl[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
try:
    jgwl = clean.continuous_dates(jgwl)
except:
    print("warn: 没有数据")
jgwl_new = jgwl.reset_index(names=['下单日期'])
# jgwl#_new#.set_index('下单日期')

warn: 没有数据


In [160]:
# df_contain['merchant_name'].unique()


## 人人享租

In [215]:
# rrxz = all_models.merchant_names(df_contain,'人人享租', '下单日期')
# rrxz = rrxz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# # rrxz = clean.continuous_dates(rrxz)
# rrxz_new = rrxz.reset_index(names=['下单日期'])
# # rrxz_new

try:
    rrxz = all_models.merchant_names(df_contain, '人人享租', '下单日期')
    if rrxz.empty:
        print("warn: 没有数据")
        rrxz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
    else:
        rrxz = rrxz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        rrxz = clean.continuous_dates(rrxz)
        rrxz_new = rrxz.reset_index(names=['下单日期'])
except:
    rrxz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
rrxz_new#.tail()

warn: 没有数据


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率


In [162]:
df_contain[df_contain['merchant_name']=='喜卓灵新租机'].下单日期.unique()

<DatetimeArray>
[]
Length: 0, dtype: datetime64[ns]

## 崇胜数码

In [163]:
# cssm = all_models.merchant_names(df_contain,'崇胜数码', '下单日期')
# cssm = cssm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# cssm_new = cssm.reset_index(names=['下单日期'])

## 喜卓灵租机

In [164]:

xzl = all_models.merchant_names(df_contain,'喜卓灵租机', '下单日期')
xzl = xzl[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
if len(xzl)!=0:
    xzl = clean.continuous_dates(xzl)
xzl_new = xzl.reset_index(names=['下单日期'])

# xzl_new

## 喜卓灵新租机

In [165]:
# xzl_news = all_models.merchant_names(df_contain,'喜卓灵新租机', '下单日期')
# xzl_news = xzl_news[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# # xzl_new = clean.continuous_dates(xzl_new)
# xzl_news = xzl_news.reset_index(names=['下单日期'])
# xzl_news

try:
    xzl_news = all_models.merchant_names(df_contain, '喜卓灵新租机', '下单日期')
    if xzl_news.empty:
        print("warn: 没有数据")
        xzl_news = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
    else:
        xzl_news = xzl_news[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        xzl_news = clean.continuous_dates(xzl_news)
        xzl_news = xzl_news.reset_index(names=['下单日期'])
except:
    xzl_news = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
xzl_news#.tail()

warn: 没有数据


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率


## 星晟数码

In [166]:
# # 入驻时间:2025-07-21 
# xcsm = all_models.merchant_names(df_contain,'星晟数码', '下单日期')
# xcsm = xcsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# xcsm = clean.continuous_dates(xcsm)
# xcsm_new = xcsm.reset_index(names=['下单日期'])
# # xcsm_new

try:
    xcsm = all_models.merchant_names(df_contain, '星晟数码', '下单日期')
    if xcsm.empty:
        print("warn: 没有数据")
        xcsm_new = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
    else:
        xcsm = xcsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        xcsm = clean.continuous_dates(xcsm)
        xcsm_new = xcsm.reset_index(names=['下单日期'])
except:
    xcsm_new = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
xcsm_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
11,2025-11-20,254,0,0.00%,86,33.86%,86,2,33,38.37%,0,3,3.49%,2.33%,1.18%
12,2025-11-21,215,0,0.00%,87,40.47%,87,0,49,56.32%,4,0,0.00%,0.00%,0.00%
13,2025-11-22,203,0,0.00%,72,35.47%,72,4,26,36.11%,16,4,5.56%,5.56%,1.97%
14,2025-11-23,181,0,0.00%,85,46.96%,85,0,20,23.53%,31,1,1.18%,0.00%,0.55%
15,2025-11-24,37,0,0.00%,20,54.05%,20,0,8,40.00%,8,0,0.00%,0.00%,0.00%


### 支付宝订单

In [167]:
# df_contain['order_type'].value_counts()
df_contain_zfb = df_contain[df_contain['order_type'] == 'ZFB_ORDER'] 
xssm_zfb = all_models.merchant_names(df_contain_zfb,'星晟数码', '下单日期')
xssm_zfb = xssm_zfb[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
xssm_zfb = clean.continuous_dates(xssm_zfb)
xcsm_zfb_new = xssm_zfb.reset_index(names=['下单日期'])
xcsm_zfb_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
11,2025-11-20,175,0,0.00%,24,13.71%,24,2,3,12.50%,0,2,8.33%,8.33%,1.14%
12,2025-11-21,132,0,0.00%,14,10.61%,14,0,2,14.29%,0,0,0.00%,0.00%,0.00%
13,2025-11-22,131,0,0.00%,15,11.45%,15,2,4,26.67%,1,1,6.67%,13.33%,0.76%
14,2025-11-23,104,0,0.00%,16,15.38%,16,0,2,12.50%,4,0,0.00%,0.00%,0.00%
15,2025-11-24,18,0,0.00%,2,11.11%,2,0,0,0.00%,1,0,0.00%,0.00%,0.00%


### 京东订单

In [168]:
# df_contain['order_type'].value_counts()
df_contain_jd = df_contain[df_contain['order_type'] == 'JD_ORDER'] 
xssm_jd = all_models.merchant_names(df_contain_jd,'星晟数码', '下单日期')
xssm_jd = xssm_jd[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
xssm_jd = clean.continuous_dates(xssm_jd)
xcsm_jd_new = xssm_jd.reset_index(names=['下单日期'])
xcsm_jd_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
10,2025-11-20,79,0,0.00%,62,78.48%,62,0,30,48.39%,0,1,1.61%,0.00%,1.27%
11,2025-11-21,83,0,0.00%,73,87.95%,73,0,47,64.38%,4,0,0.00%,0.00%,0.00%
12,2025-11-22,72,0,0.00%,57,79.17%,57,2,22,38.60%,15,3,5.26%,3.51%,4.17%
13,2025-11-23,77,0,0.00%,69,89.61%,69,0,18,26.09%,27,1,1.45%,0.00%,1.30%
14,2025-11-24,19,0,0.00%,18,94.74%,18,0,8,44.44%,7,0,0.00%,0.00%,0.00%


## 蘑菇时间  

In [169]:
# # 入驻时间:2025-07-08
# mgsj = all_models.merchant_names(df_contain,'蘑菇时间', '下单日期')
# mgsj = mgsj[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# mgsj = clean.continuous_dates(mgsj)
# mgsj_New = mgsj.reset_index(names=['下单日期'])
# # mgsj_New

try:
    mgsj = all_models.merchant_names(df_contain, '蘑菇时间', '下单日期')
    if mgsj.empty:
        print("warn: 没有数据")
        mgsj_New = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
    else:
        mgsj = mgsj[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        mgsj = clean.continuous_dates(mgsj)
        mgsj_New = mgsj.reset_index(names=['下单日期'])
except:
    mgsj_New = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
mgsj_New.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
11,2025-11-20,4.0,0.0,0.00%,2.0,50.00%,2.0,0.0,1.0,50.00%,0.0,0.0,0.00%,0.00%,0.00%
12,2025-11-21,1.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
13,2025-11-22,2.0,0.0,0.00%,1.0,50.00%,1.0,0.0,0.0,0.00%,0.0,0.0,0.00%,0.00%,0.00%
14,2025-11-23,1.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
15,2025-11-24,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0


## 云启德曜

In [170]:
try:
    yqdy = all_models.merchant_names(df_contain, '云启德曜', '下单日期')
    if yqdy.empty:
        print("warn: 没有数据")
        yqdy_New = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
    else:
        yqdy = yqdy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        yqdy = clean.continuous_dates(yqdy)
        yqdy_New = yqdy.reset_index(names=['下单日期'])
except:
    yqdy_New = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
yqdy_New#.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
0,2025-11-09,54,0,0.00%,54,100.00%,54,0,0,0.00%,0,0,0.00%,0.00%,0.00%
1,2025-11-10,68,0,0.00%,68,100.00%,68,0,0,0.00%,0,2,2.94%,0.00%,2.94%
2,2025-11-11,159,0,0.00%,159,100.00%,159,0,0,0.00%,0,1,0.63%,0.00%,0.63%
3,2025-11-12,188,0,0.00%,188,100.00%,188,0,0,0.00%,0,2,1.06%,0.00%,1.06%
4,2025-11-13,181,0,0.00%,181,100.00%,181,0,0,0.00%,0,6,3.31%,0.00%,3.31%
5,2025-11-14,189,0,0.00%,189,100.00%,189,0,0,0.00%,0,4,2.12%,0.00%,2.12%
6,2025-11-15,161,0,0.00%,161,100.00%,161,0,0,0.00%,0,2,1.24%,0.00%,1.24%
7,2025-11-16,177,0,0.00%,177,100.00%,177,0,0,0.00%,0,2,1.13%,0.00%,1.13%
8,2025-11-17,197,0,0.00%,197,100.00%,197,0,0,0.00%,0,3,1.52%,0.00%,1.52%
9,2025-11-18,199,0,0.00%,199,100.00%,199,0,0,0.00%,0,9,4.52%,0.00%,4.52%


In [171]:
df_yq = df_contain[(df_contain['merchant_name'] == '云启德曜')&(df_contain['下单日期']=='2025-11-10')]
df_yq[['order_number', 'merchant_name', '进件',"客户取消",'人审拒绝','待审核', '是否出库','商品ID','activity_name']].to_excel('F:\宏生\云启德曜_2025-11-10.xlsx')

## 艾欧尼亚

In [172]:
try:
    aony = all_models.merchant_names(df_contain, '艾欧尼亚数码', '下单日期')
    if aony.empty:
        print("warn: 没有数据")
        aony_New = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
    else:
        aony = aony[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        aony = clean.continuous_dates(aony)
        aony_New = aony.reset_index(names=['下单日期'])
except:
    aony_New = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
aony_New.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
11,2025-11-20,1.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
12,2025-11-21,1.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
13,2025-11-22,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
14,2025-11-23,1.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
15,2025-11-24,1.0,0.0,0.00%,1.0,100.00%,1.0,0.0,0.0,0.00%,1.0,0.0,0.00%,0.00%,0.00%


# 每日各渠道订单统计 导出

In [173]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-11-24'

In [174]:
path = r'\\digua\迪瓜租机\20.日报数据相关输出/'
# 每日各渠道去重订单统计结果表格导出
# df = df[df.归属渠道=='搜索渠道']
gsqd_qcdd = pd.crosstab(df["下单日期"],df["归属渠道"],margins=True)
lyqd_qcdd = pd.crosstab(df["下单日期"],df["来源渠道"],margins=True)
hdmc_qcdd = pd.crosstab(df["下单日期"],df["activity_name"],margins=True)
#每日各渠道进件统计结果表格导出
# df_j = df_j[df_j.归属渠道=='搜索渠道']
gsqd_jj = pd.crosstab(df_j["下单日期"],df_j["归属渠道"],margins=True)
lyqd_jj = pd.crosstab(df_j["下单日期"],df_j["来源渠道"],margins=True)
hdmc_jj = pd.crosstab(df_j["下单日期"],df_j["activity_name"],margins=True)
# 各归属渠道 出库 导出
# dfck = dfck[dfck.归属渠道=='搜索渠道']
gsqd_ck = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)
lyqd_ck = pd.crosstab(dfck["下单日期"],dfck["来源渠道"],margins=True)
hdmc_ck = pd.crosstab(dfck["下单日期"],dfck["activity_name"],margins=True)
# 商品ID
qd_list = ['搜索-租手机','搜索-租苹果手机','搜索下挂-商品','搜索直达banner','搜索下挂券','顶部搜索框的搜索结果页','搜索侠客行-核心词','搜索侠客行-首页','搜索邦道-销售线索','搜索邦道-小程序推广']
sp_jj = pd.pivot_table(df_j[df_j.来源渠道.isin(qd_list)],values='order_id', index=["下单日期", '商品ID'], columns='来源渠道', aggfunc='count')
sp_ck = pd.pivot_table(dfck[dfck.来源渠道.isin(qd_list)],values='order_id', index=["下单日期", '商品ID'], columns='来源渠道', aggfunc='count')
try:
    with pd.ExcelWriter(path + f'日报相关数据输出_{t_date}.xlsx', engine='xlsxwriter') as writer:
        # 去重订单
        gsqd_qcdd.to_excel(writer, sheet_name='归属渠道去重订单统计', index=True)  
        lyqd_qcdd.to_excel(writer, sheet_name='来源渠道去重订单统计', index=True)
        hdmc_qcdd.to_excel(writer, sheet_name='活动名称去重订单统计', index=True)
        # 进件
        gsqd_jj.to_excel(writer, sheet_name='归属渠道进件统计', index=True)
        lyqd_jj.to_excel(writer, sheet_name='来源渠道进件统计', index=True)
        hdmc_jj.to_excel(writer, sheet_name='活动名称进件统计', index=True)
        # 出库
        gsqd_ck.to_excel(writer, sheet_name='归属渠道出库统计', index=True)
        lyqd_ck.to_excel(writer, sheet_name='来源渠道出库统计', index=True)
        hdmc_ck.to_excel(writer, sheet_name='活动名称出库统计', index=True)
        # 商品ID
        sp_jj.to_excel(writer, sheet_name='来源渠道商品ID进件统计', index=True)
        sp_ck.to_excel(writer, sheet_name='来源渠道商品ID出库统计', index=True)
except PermissionError:
    print(PermissionError)

In [175]:
# sp_jj.tail()
df["下单日期"].unique()


<DatetimeArray>
['2025-11-09 00:00:00', '2025-11-10 00:00:00', '2025-11-11 00:00:00',
 '2025-11-12 00:00:00', '2025-11-13 00:00:00', '2025-11-14 00:00:00',
 '2025-11-15 00:00:00', '2025-11-16 00:00:00', '2025-11-17 00:00:00',
 '2025-11-18 00:00:00', '2025-11-19 00:00:00', '2025-11-20 00:00:00',
 '2025-11-21 00:00:00', '2025-11-22 00:00:00', '2025-11-23 00:00:00',
 '2025-11-24 00:00:00']
Length: 16, dtype: datetime64[ns]

In [176]:
# dfck[(dfck['来源渠道']=='搜索下挂-商品')&(dfck['退货拒签']==1)].shape

# 总体 "租完即送"占比

In [177]:
# 获取拒量的出库和非拒量的出库
dfck_fjl = dfck[~dfck.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
dfck_jl = dfck[(dfck.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True)&(dfck.tips.str.contains('联合拒量订单')==False)]
# 可能哪天会发现连续15天没有拒量出库的数据，可以将时间范围扩大21-30天临时解决先跑下日报

In [178]:
dfck_jl.shape#[dfck_jl['下单日期']=='2025-09-03']

(0, 138)

In [179]:
# #总体 "租完即送"占比（拒量）
# df_rent_jlck = pd.crosstab(dfck_jl["下单日期"],dfck_jl["租赁方案"],margins=True)
# # 填充不连续日期的数据

# df_rent_jlck = clean.continuous_dates(df_rent_jlck.iloc[:-1, :])
# df_rent_jlck = df_rent_jlck.reset_index(names=['下单日期'])
# df_rent_jlck['下单日期'] = pd.to_datetime(df_rent_jlck['下单日期'])
# df_rent_jlck = df_rent_jlck.set_index('下单日期')
# df_rent_jlck.rename(columns={'租完即送': '租完即送_拒量', '租完归还': '租完归还_拒量', 'All': 'All_拒量'}, inplace=True)
# try:
#     df_rent_jlck.loc[:, '租完即送出库占比_拒量'] = (df_rent_jlck.租完即送_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
# except:
#     df_rent_jlck.loc[:, '租完即送_拒量'] = 0
#     df_rent_jlck.loc[:, '租完即送出库占比_拒量'] = (df_rent_jlck.租完即送_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
# try:
#     df_rent_jlck.loc[:, '租完即归还库占比_拒量'] = (df_rent_jlck.租完归还_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
# except:
#     df_rent_jlck.loc[:, '租完归还_拒量'] = 0
#     df_rent_jlck.loc[:, '租完即归还库占比_拒量'] = (df_rent_jlck.租完归还_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))


In [180]:
import pandas as pd
from datetime import datetime, timedelta

df_rent_jlck = pd.crosstab(dfck_jl["下单日期"], dfck_jl["租赁方案"], margins=True)

# 如果 df_rent_jlck 为空，则进行处理
if df_rent_jlck.empty:
    # 获取当前日期
    current_date = datetime.now().date()
    
    # 计算15天前的日期
    start_date = current_date - timedelta(days=15)  # 15天不包括今天，所以是15天前
    
    # 生成最近15天的所有日期
    date_range = pd.date_range(start=start_date, end=current_date, freq='D')
    
    # 创建一个新的 DataFrame，列名与原始 df_rent_jlck 一致
    df_rent_jlck = pd.DataFrame(index=date_range, columns=['租完即送_拒量', '租完归还_拒量', 'All_拒量', '租完即送出库占比_拒量', '租完即归还库占比_拒量'])
    
    # 将所有数据填充为 0 或 '0%'
    df_rent_jlck.fillna(0, inplace=True)
    # df_rent_jlck['租完即送出库占比_拒量'] = 0
    # df_rent_jlck['租完即归还库占比_拒量'] = 0
    
    # 设置索引名称
    df_rent_jlck.index.name = '下单日期'
else:
    # 原有处理逻辑
    # df_rent_jlck_clean = df_rent_jlck.iloc[:-1, :]#.dropna(subset=['下单日期']) 
    df_rent_jlck = clean.continuous_dates(df_rent_jlck.iloc[:-1, :])
    df_rent_jlck = df_rent_jlck.reset_index(names=['下单日期'])
    df_rent_jlck['下单日期'] = pd.to_datetime(df_rent_jlck['下单日期'])
    df_rent_jlck = df_rent_jlck.set_index('下单日期')
    df_rent_jlck.rename(columns={'租完即送': '租完即送_拒量', '租完归还': '租完归还_拒量', 'All': 'All_拒量'}, inplace=True)
    
    try:
        df_rent_jlck.loc[:, '租完即送出库占比_拒量'] = (df_rent_jlck.租完即送_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
    except:
        df_rent_jlck.loc[:, '租完即送_拒量'] = 0
        df_rent_jlck.loc[:, '租完即送出库占比_拒量'] = (df_rent_jlck.租完即送_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
    
    try:
        df_rent_jlck.loc[:, '租完即归还库占比_拒量'] = (df_rent_jlck.租完归还_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
    except:
        df_rent_jlck.loc[:, '租完归还_拒量'] = 0
        df_rent_jlck.loc[:, '租完即归还库占比_拒量'] = (df_rent_jlck.租完归还_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))

In [181]:
# df_rent_jlck

In [182]:
#总体 "租完即送"占比（排除拒量）
df_rent  = pd.crosstab(df_j["下单日期"],df_j["租赁方案"],margins=True)
df_rent=df_rent.rename(columns={"租完即送":"租完即送进件","租完归还":"租完归还进件","All":"合计进件",})
df_rent_ck=pd.crosstab(dfck_fjl["下单日期"],dfck_fjl["租赁方案"],margins=True)
df_rent_ck=df_rent_ck.rename(columns={"租完即送":"租完即送出库","租完归还":"租完归还出库","All":"合计出库",})
df_r=pd.merge(df_rent,df_rent_ck,left_index=True, right_index=True)
df_r["租完即送进件占比"]=df_r["租完即送进件"]/df_r["合计进件"]
df_r["租完即送进件占比"]=df_r["租完即送进件占比"].apply(lambda x:format(x,".2%"))
df_r["租完归还进件占比"]=df_r["租完归还进件"]/df_r["合计进件"]
df_r["租完归还进件占比"]=df_r["租完归还进件占比"].apply(lambda x:format(x,".2%"))
df_r["租完即送出库占比"]=df_r["租完即送出库"]/df_r["合计出库"]
df_r["租完即送出库占比"]=df_r["租完即送出库占比"].apply(lambda x:format(x,".2%"))
df_r["租完归还出库占比"]=df_r["租完归还出库"]/df_r["合计出库"]
df_r["租完归还出库占比"]=df_r["租完归还出库占比"].apply(lambda x:format(x,".2%"))
df_r["租完即送转化率"]=df_r["租完即送出库"]/df_r["租完即送进件"]
df_r["租完即送转化率"]=df_r["租完即送转化率"].apply(lambda x:format(x,".2%"))
df_r["租完归还转化率"]=df_r["租完归还出库"]/df_r["租完归还进件"]
df_r["租完归还转化率"]=df_r["租完归还转化率"].apply(lambda x:format(x,".2%"))
df_r = df_r.iloc[:-1, :]
df_r = df_r.reset_index()
df_r.loc[:, '下单日期'] = pd.to_datetime(df_r['下单日期'])
df_r.set_index('下单日期', inplace=True)
# 拼接拒量出库
df_r1 = df_r.merge(df_rent_jlck, on='下单日期', how='left')

df_r1 = df_r1[["租完即送进件","租完归还进件","租完归还进件占比","租完即送进件占比","租完即送出库","租完归还出库", '租完即送_拒量', '租完归还_拒量', '租完即归还库占比_拒量', '租完即送出库占比_拒量', "租完归还出库占比","租完即送出库占比", '租完即送转化率','租完归还转化率']]#.iloc[:-1, :]
df_r1 = clean.continuous_dates(df_r1)
df_r_new = df_r1.reset_index(names=['下单日期'])


# 总体出库订单碎屏险购买数据

## 总体

In [183]:
def ck(dfck, name):
    # '服务订单状态：1、待支付；2、待确认；3、服务中；4、已失效；5、已取消；6、已退款',
    dfck[f'已取消'] = np.where(dfck["service_status"]==5,1,0)
    dfck[f'已退款'] = np.where(dfck["service_status"]==6,1,0)
    dfck[f'待支付'] = np.where(dfck["service_status"]==1,1,0)
    dfck[f'待确认'] = np.where(dfck["service_status"]==2,1,0)
    dfck[f'服务中'] = np.where(dfck["service_status"]==3,1,0)

    df_s3  =  dfck.groupby(["下单日期"]).agg({f'已取消':'sum',f'已退款':'sum', f'待支付':'sum', f'待确认':'sum',f'服务中':'sum'})
    
    df_s3[f"最终实际支付碎屏险出库_{name}"]=df_s3[f"待确认"]+df_s3[f"服务中"] +df_s3[f"待支付"]
    return df_s3

In [184]:
# 不包含京享租
dfck_fjl_fjd = dfck_fjl[dfck_fjl.归属渠道!='京东渠道']
dfck_jl_fjd = dfck_jl[dfck_jl.归属渠道!='京东渠道']
#总体出库订单碎屏险购买数据

df_bx_ck=pd.crosstab(dfck_fjl_fjd["下单日期"],dfck_fjl_fjd["buy_service_product"],margins=True)

if df_bx_ck.empty:
    # 获取当前日期
    current_date = datetime.now().date()
    
    # 计算15天前的日期
    start_date = current_date - timedelta(days=15)  # 15天不包括今天，所以是15天前
    
    # 生成最近15天的所有日期
    date_range = pd.date_range(start=start_date, end=current_date, freq='D')
    
    # 创建一个新的 DataFrame，列名与原始 df_rent_jlck 一致
    df_bx_ck = pd.DataFrame(index=date_range, columns=['不购买碎屏险出库', '购买碎屏险出库', 'All', '合计出库'])
    
    # 将所有数据填充为 0 或 '0%'
    df_bx_ck.fillna(0, inplace=True)
    
    
    # 设置索引名称
    df_bx_ck.index.name = '下单日期'
else:
    df_bx_ck=df_bx_ck.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
    
#总体出库订单碎屏险购买数据——拒量
df_bx_ck_jl=pd.crosstab(dfck_jl_fjd["下单日期"],dfck_jl_fjd["buy_service_product"],margins=True)

if df_bx_ck_jl.empty:
    # 获取当前日期
    current_date = datetime.now().date()
    
    # 计算15天前的日期
    start_date = current_date - timedelta(days=15)  # 15天不包括今天，所以是15天前
    
    # 生成最近15天的所有日期
    date_range = pd.date_range(start=start_date, end=current_date, freq='D')
    
    # 创建一个新的 DataFrame，列名与原始 df_rent_jlck 一致
    df_bx_ck_jl = pd.DataFrame(index=date_range, columns=['不购买碎屏险出库', '购买碎屏险出库', 'All', '合计出库_拒量'])
    
    # 将所有数据填充为 0 或 '0%'
    df_bx_ck_jl.fillna(0, inplace=True)
    
    
    # 设置索引名称
    df_bx_ck_jl.index.name = '下单日期'
else:

    df_bx_ck_jl=df_bx_ck_jl.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库_拒量",})

# 非拒量
df_s3 = ck(dfck_fjl_fjd, 'All')
# 拒量
df_s3_jl = ck(dfck_jl_fjd, '拒量')

# df_s3=pd.crosstab(dfck["下单日期"],dfck["服务订单状态"],margins=True)
# df_s3=df_s3.drop(columns={"All"})
df_s2merge=pd.merge(df_bx_ck,df_s3,left_index=True, right_index=True)
df_s3merge=pd.merge(df_bx_ck_jl,df_s3_jl,left_index=True, right_index=True)
df_s2merge = df_s2merge.merge(df_s3merge[['合计出库_拒量', '最终实际支付碎屏险出库_拒量']], left_index=True, right_index=True, how='left')
df_s2merge["实际支付碎屏险比例"]=df_s2merge["最终实际支付碎屏险出库_All"]/df_s2merge["合计出库"]
df_s2merge["实际支付碎屏险比例"]=df_s2merge["实际支付碎屏险比例"].apply(lambda x:format(x,".2%"))
df_s2merge.loc[:, '实际支付碎屏险比例_拒量'] = (df_s2merge.最终实际支付碎屏险出库_拒量/df_s2merge.合计出库_拒量).map(lambda x: format(x, '.2%'))

df_s2merge = df_s2merge[['不购买碎屏险出库','购买碎屏险出库','合计出库', '合计出库_拒量','已取消','已退款', '待支付', '待确认','服务中','最终实际支付碎屏险出库_拒量','最终实际支付碎屏险出库_All', '实际支付碎屏险比例_拒量',  '实际支付碎屏险比例']]
df_s2merge = clean.continuous_dates(df_s2merge)
df_s2merge = df_s2merge.reset_index(names=['下单日期'])
# df_s2merge
df_s2merge.tail()

,下单日期,不购买碎屏险出库,购买碎屏险出库,合计出库,合计出库_拒量,已取消,已退款,待支付,待确认,服务中,最终实际支付碎屏险出库_拒量,最终实际支付碎屏险出库_All,实际支付碎屏险比例_拒量,实际支付碎屏险比例
11,2025-11-20,2.0,147.0,149.0,0.0,2.0,0.0,0.0,36.0,109.0,0.0,145.0,nan%,97.32%
12,2025-11-21,4.0,130.0,134.0,0.0,1.0,1.0,0.0,111.0,17.0,0.0,128.0,nan%,95.52%
13,2025-11-22,2.0,122.0,124.0,0.0,2.0,0.0,0.0,117.0,3.0,0.0,120.0,nan%,96.77%
14,2025-11-23,1.0,88.0,89.0,0.0,0.0,1.0,0.0,87.0,0.0,0.0,87.0,nan%,97.75%
15,2025-11-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [185]:
# df_s2merge
# dfck[dfck["归属渠道"]=="芝麻租物"]

In [186]:
#芝麻租物出库订单碎屏险购买数据
dfckzw=dfck[dfck["归属渠道"]=="芝麻租物"]
df_zw3=pd.crosstab(dfckzw["下单日期"],dfckzw["buy_service_product"],margins=True)
df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})

dfckzw['已取消'] = np.where(dfckzw["service_status"]==5,1,0)
dfckzw['已退款'] = np.where(dfckzw["service_status"]==6,1,0)
dfckzw['待支付'] = np.where(dfckzw["service_status"]==1,1,0)
dfckzw['待确认'] = np.where(dfckzw["service_status"]==2,1,0)
dfckzw['服务中'] = np.where(dfckzw["service_status"]==3,1,0)

df_zws3  =   dfckzw.groupby(["下单日期"]).agg({'已取消':'sum','已退款':'sum', '待支付':'sum', '待确认':'sum','服务中':'sum'})  
# =pd.crosstab(dfckzw["下单日期"],dfckzw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})

df_zws2merge = pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
df_zws2merge["最终实际支付碎屏险出库"] = df_zws2merge["待确认"]+df_zws2merge["服务中"]+df_zws2merge["待支付"]
# 如果发现合计出库为0，或者没有芝麻租物的订单，则可能是每日各渠道订单统计导出中设置了出库渠道仅有搜索渠道
df_zws2merge["实际支付碎屏险比例"] = df_zws2merge["最终实际支付碎屏险出库"]/df_zws2merge["合计出库"]
df_zws2merge["实际支付碎屏险比例"] = df_zws2merge["实际支付碎屏险比例"].apply(lambda x:format(x,".2%"))


df_zws2merge = df_zws2merge[['不购买碎屏险出库','购买碎屏险出库','合计出库','已取消','已退款', '待支付', '待确认','服务中','最终实际支付碎屏险出库','实际支付碎屏险比例']]
df_zws2merge = clean.continuous_dates(df_zws2merge)
df_zws2merge_new = df_zws2merge.reset_index(names=['下单日期1'])
# df_zws2merge_new = df_zws2merge_new.rename(columns={'下单日期': '下单日期1'})
df_zws2merge_new.tail()

,下单日期1,不购买碎屏险出库,购买碎屏险出库,合计出库,已取消,已退款,待支付,待确认,服务中,最终实际支付碎屏险出库,实际支付碎屏险比例
11,2025-11-20,1.0,74.0,75.0,1.0,0.0,0.0,22.0,51.0,73.0,97.33%
12,2025-11-21,0.0,74.0,74.0,1.0,1.0,0.0,63.0,9.0,72.0,97.30%
13,2025-11-22,2.0,67.0,69.0,2.0,0.0,0.0,63.0,2.0,65.0,94.20%
14,2025-11-23,0.0,51.0,51.0,0.0,1.0,0.0,50.0,0.0,50.0,98.04%
15,2025-11-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


## 整合

In [187]:
df_s2merge_new = df_s2merge#.reset_index()
df_s2merge_new.loc[:, '碎屏险实际出库占比'] = (df_s2merge_new.最终实际支付碎屏险出库_All/df_s2merge_new.购买碎屏险出库).map(lambda x: format(x,".2%"))
df_s2merge_new.loc[:, '非芝麻实际支付碎屏险比例'] = ((df_s2merge_new.购买碎屏险出库-df_zws2merge_new.购买碎屏险出库)/(df_s2merge_new.合计出库-df_zws2merge_new.合计出库)).map(lambda x: format(x,".2%"))
df_s2merge_all_new = pd.concat([df_s2merge_new, df_zws2merge_new], axis=1)
# df_s2merge_all_new#.set_index('下单日期')

## 京享租

In [188]:
df_jd2_ck = df_jd2[df_jd2.是否出库==1]
df_jd2_sp=pd.crosstab(df_jd2_ck["下单日期"],df_jd2_ck["buy_service_product"],margins=True)
df_jd2_sp=df_jd2_sp.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
df_jdsp = ck(df_jd2_ck, 'All')
df_jd_merge=pd.merge(df_jd2_sp,df_jdsp,left_index=True, right_index=True)
df_jd_merge["实际支付碎屏险比例"]=(df_jd_merge["最终实际支付碎屏险出库_All"]/df_jd_merge["合计出库"]).map(lambda x: format(x, '.2%'))
df_jd_merge = df_jd_merge.reset_index()
df_jd_merge.loc[:, '下单日期'] = df_jd_merge.下单日期.astype('object')
df_jd_group_new = df_jd_group_new.merge(df_jd_merge[['下单日期', '合计出库', '最终实际支付碎屏险出库_All', '实际支付碎屏险比例']], on='下单日期', how='left')
df_jd_group_new.tail()

,下单日期,创建订单数,创建进件数,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率,合计出库,最终实际支付碎屏险出库_All,实际支付碎屏险比例
11,2025-11-20,1858,675,1350,728,53.93%,578,42.81%,441.0,76.30%,137.0,81.0,56.0,9.69%,8,10,0.0,0,38,39.0,6.57%,1.38%,14.01%,0.00%,1.73%,2.81%,38.0,38.0,100.00%
12,2025-11-21,1765,621,1287,733,56.95%,514,39.94%,394.0,76.65%,120.0,68.0,52.0,10.12%,7,6,0.0,3,36,37.0,7.00%,1.36%,13.23%,0.00%,1.17%,2.80%,36.0,36.0,100.00%
13,2025-11-22,1641,567,1208,687,56.87%,486,40.23%,381.0,78.40%,105.0,59.0,46.0,9.47%,6,4,0.0,11,25,25.0,5.14%,1.23%,12.14%,0.00%,0.82%,2.07%,25.0,25.0,100.00%
14,2025-11-23,1788,603,1310,763,58.24%,504,38.47%,406.0,80.56%,98.0,53.0,45.0,8.93%,3,3,0.0,23,16,20.0,3.17%,0.60%,10.52%,0.00%,0.60%,1.22%,16.0,16.0,100.00%
15,2025-11-24,260,82,182,115,63.19%,61,33.52%,56.0,91.80%,5.0,1.0,4.0,6.56%,0,0,0.0,4,0,0.0,0.00%,0.00%,1.64%,0.00%,0.00%,0.00%,NaN,NaN,NaN


## 新浪微博

In [189]:
# df_wb2_sp=pd.crosstab(df_wb2_ck["下单日期"],df_wb2_ck["buy_service_product"],margins=True)
# df_wb2_sp

In [190]:
# df_wb2_ck = df_wb2[df_wb2.是否出库==1]
# df_wb2_sp=pd.crosstab(df_wb2_ck["下单日期"],df_wb2_ck["buy_service_product"],margins=True)
# df_wb2_sp=df_wb2_sp.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
# df_wbsp = ck(df_wb2_ck, 'All')
# df_wb_merge=pd.merge(df_wb2_sp,df_wbsp,left_index=True, right_index=True)
# df_wb_merge["实际支付碎屏险比例"]=(df_wb_merge["最终实际支付碎屏险出库_All"]/df_wb_merge["合计出库"]).map(lambda x: format(x, '.2%'))
# df_wb_merge = df_wb_merge.reset_index()
# df_wb_merge.loc[:, '下单日期'] = df_wb_merge.下单日期.astype('object')
# df_wb_group_new = df_wb_group_new.merge(df_wb_merge[['下单日期', '合计出库', '最终实际支付碎屏险出库_All', '实际支付碎屏险比例']], on='下单日期', how='left')
# df_wb_group_new.tail()

# 拒量（廖志彧）

## 何薪华需求

In [191]:
# df.columns

In [192]:
# # 拒量的订单2024年12月到2025年6月正常还款的（租赁中）用户没有逾期的订单，需要订单号，用户下单的手机号码，审核人，还款期数（已还）
# df_jl = df[(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True)&df.status==4]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]
# # 


# df_name_new = df_name.sort_values('update_time', ascending=False).groupby('order_id').head(1)
# # 内连接，获取出库单分配人
# df_jl_name = df_jl_new[df_jl_new.是否出库==1].merge(df_name_new, on='order_id', how='inner')
# df_jl_name_new = df_jl_name[['下单日期', 'order_id', 'order_number', '策略命中等级', '分配人']]

## 拒量数据(转化数据、明细)

In [193]:
# # # 设置时间段
# now_day = datetime.now()
# seven_ago = now_day - pd.Timedelta(days=8)
# one_ago = now_day - pd.Timedelta(days=1)
# # 获取特定时间段的拒量订单的转化数据和明细

# df_jl = df[df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]
# df_jl_new = df_jl#[~df_jl.tips.str.contains('策略241205命中(1)', regex=False)]
# df_jl_new['策略命中等级'] = df_jl_new['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中\(\d+\)?|命中自有模型回捞策略|回捞策略250330命中?)')[0]
# # 进件数，出库数，出库率，风险等级
# df_jl_new_group = df_jl_new.groupby('下单日期').agg({'是否进件': 'sum', '是否出库': 'sum'})
# df_jl_new_group.rename(columns={'是否进件': '进件数', '是否出库': '出库'}, inplace=True)
# df_jl_new_group['进件出库率'] = (df_jl_new_group.出库/df_jl_new_group.进件数).map(lambda x: format(x, '.2%'))
# df_jl_new2 = df_jl_new[['下单日期', 'order_number', '策略命中等级']]
# df_jl_new2_group = df_jl_new2.groupby('策略命中等级').agg(数量=('order_number', 'count'))

# # 获取当前是星期几
# Monday = pd.to_datetime(datetime.now().strftime('%Y-%m-%d')).day_name()
# if Monday == 'Monday':
#     with pd.ExcelWriter(f'F:/需求/廖志彧需求/拒量数据_{Today_zfb}.xlsx', engine='openpyxl') as writer:
#         df_jl_new_group.to_excel(writer, sheet_name='转化数据')
#         df_jl_new2_group.to_excel(writer, sheet_name='拒量数据明细')
#         df_jl_new['order_number'].to_excel(writer, sheet_name='拒量明细', index=False)
# df_name['分配人'].unique()


## 出库单分配人

In [194]:
# # # 分配人名单：蔡斯静 林婉婷 胡春燕 何薪华 刘慧琪 钟桂洪
# name_list = ['蔡斯静', '林婉婷', '胡春燕', '何薪华', '刘慧琪', '钟桂洪', '王伟鸿']
# # 筛选出库单分配人
# df_name = df_name[df_name['分配人'].isin(name_list)]
# # df_name.分配人.unique()
# df_jl = df[df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]
# df_jl_new = df_jl#[~df_jl.tips.str.contains('策略241205命中(1)', regex=False)]
# df_jl_new['策略命中等级'] = df_jl_new['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中\(\d+\)?|命中自有模型回捞策略|回捞策略250330命中?)')[0]

# df_name_new = df_name.sort_values('update_time', ascending=False).groupby('order_id').head(1)
# # 内连接，获取出库单分配人
# df_jl_name = df_jl_new[df_jl_new.是否出库==1].merge(df_name_new, on='order_id', how='inner')
# df_jl_name_new = df_jl_name[['下单日期', 'order_id', 'order_number', '策略命中等级', '分配人']]
# with pd.ExcelWriter(f'F:/需求/廖志彧需求/分配数据_{t_date}.xlsx', engine='openpyxl') as writer:
#     df_jl_name_new.to_excel(writer, sheet_name='出库单分配人', index=False)
# # 

In [195]:
# df_jl_new.下单日期.unique()

## 首逾

In [196]:
# # 在没有其它变化的情况下该口径是OK的：2025.9.3
# now_day = datetime.now()

# df_jl = df[df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]

# name_list = ['蔡斯静', '林婉婷', '胡春燕', '何薪华', '刘慧琪', '钟桂洪','王伟鸿']
# df_name = df_name[df_name['分配人'].isin(name_list)]
# df_name_new = df_name.sort_values('update_time', ascending=False).groupby('order_id').head(1)
# # 首逾
# date1 = '2025-09'
# date2 = '2025-10'
# df_new = df_jl[df_jl.下单月份==date1]

# # df_new["出库"] = np.where(df_new["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)

# df_new = df_new[df_new.是否出库==1]

# df_stages_new = df_stages[df_stages['order_id'].notna()]
# df_stages_new['order_id'] = df_stages_new['order_id'].astype(int)
# df_stages_new.loc[:, '预计还款月份'] = df_stages_new.refund_date.dt.strftime('%Y-%m')
# df_new_merge = df_new.merge(df_stages_new, on='order_id', how='left')
# # df_jl_stages_new = df_new_merge[(df_new_merge.预计还款月份==date2)&(df_new_merge.sort==2)]
# df_jl_stages_new = df_new_merge[(df_new_merge.sort==2)]
# df_jl_stages_new.loc[:, 'now_day'] = pd.to_datetime(now_day.strftime('%Y-%m-%d'))
# # df_jl_stages_new = df_jl_stages_new[df_jl_stages_new.refund_date<=df_jl_stages_new.now_day]
# df_jl_stages_new.loc[:, 'overdue_day'] = np.where(df_jl_stages_new.reality_refund_date.notna(), 
#                                                 np.where(df_jl_stages_new.reality_refund_date>df_jl_stages_new.refund_date, (df_jl_stages_new.reality_refund_date-df_jl_stages_new.refund_date), 0)
#                                                 , np.where(df_jl_stages_new.refund_date>df_jl_stages_new.now_day, 0, (df_jl_stages_new.now_day-df_jl_stages_new.refund_date)))
# df_jl_stages_new.loc[:, 'overdue_day'] = df_jl_stages_new['overdue_day'].apply(lambda x: x.days)
# # df_jl_stages_new = df_jl_stages_new[df_jl_stages_new.overdue_day>0]

# # df_jl_stages_news = df_jl_stages_new[df_jl_stages_new.overdue_day>0][['order_id', 'order_number', '下单日期', 'refund_date', 'reality_refund_date',  'overdue_day']].rename(columns={'order_id': '订单id', 'order_number': '订单号', 'refund_date': '应还日期', 'reality_refund_date': '实还日期', 'overdue_day': '逾期天数'})
# df_jl_stages_news = df_jl_stages_new[['order_id', 'order_number', '下单日期', 'refund_date', 'reality_refund_date',  'overdue_day', 'status2']].rename(columns={'order_id': '订单id', 'order_number': '订单号', 'refund_date': '应还日期', 'reality_refund_date': '实还日期', 'overdue_day': '逾期天数'})
# df_jl_stages_news = df_jl_stages_news.merge(df_name_new[['order_id', '分配人']], left_on='订单id', right_on='order_id', how='inner')
# df_jl_stages_news.loc[:, '是否逾期'] = np.where((df_jl_stages_news.status2=='租赁中')&(df_jl_stages_news.实还日期.isna())&(df_jl_stages_news.逾期天数>0), 1, 0)
# df_jl_stages_news_g = df_jl_stages_news.groupby('分配人').agg({'order_id': 'count', '是否逾期': 'sum'}).rename(columns={'order_id': '出库'})
# df_jl_stages_news_g.loc[:, '逾期/出库'] = (df_jl_stages_news_g.是否逾期/df_jl_stages_news_g.出库).map(lambda x: format(x, '.2%'))
# with pd.ExcelWriter(f'F:/需求/廖志彧需求/首逾_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_jl_stages_news.to_excel(writer, sheet_name='出库订单', index=False)
#     df_jl_stages_news_g.to_excel(writer, sheet_name='首逾订单')
# # df_jl_stages_news

In [197]:
# df_jl.tail()

## wxz总体拒量程序

In [198]:
# 设置时间段
# now_day = datetime.now()
# seven_ago = now_day - pd.Timedelta(days=8)
# one_ago = now_day - pd.Timedelta(days=1)
# # 获取特定时间段的拒量订单的转化数据和明细
# # df_jl = df[(df.下单日期>=seven_ago)&(df.下单日期<=one_ago)&(df.tips.str.contains(r'策略2412|命中自有模型回捞策略')==True)] 多半无用
# df_jl = df[df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]
# df_jl_new = df_jl#[~df_jl.tips.str.contains('策略241205命中(1)', regex=False)]
# df_jl_new['策略命中等级'] = df_jl_new['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中\(\d+\)?|命中自有模型回捞策略|回捞策略250330命中?)')[0]
# # 进件数，出库数，出库率，风险等级
# df_jl_new_group = df_jl_new.groupby('下单日期').agg({'是否进件': 'sum', '是否出库': 'sum'})
# df_jl_new_group.rename(columns={'是否进件': '进件数', '是否出库': '出库'}, inplace=True)
# df_jl_new_group['进件出库率'] = (df_jl_new_group.出库/df_jl_new_group.进件数).map(lambda x: format(x, '.2%'))
# df_jl_new2 = df_jl_new[['下单日期', 'order_number', '策略命中等级']]
# df_jl_new2_group = df_jl_new2.groupby('策略命中等级').agg(数量=('order_number', 'count'))

# 获取当前是星期几
# Monday = pd.to_datetime(datetime.now().strftime('%Y-%m-%d')).day_name()
# if Monday == 'Monday':
#     with pd.ExcelWriter(f'F:/需求/廖志彧需求/拒量数据_{Today_zfb}.xlsx', engine='openpyxl') as writer:
#         df_jl_new_group.to_excel(writer, sheet_name='转化数据')
#         df_jl_new2_group.to_excel(writer, sheet_name='拒量数据明细')
#         df_jl_new['order_number'].to_excel(writer, sheet_name='拒量明细', index=False)


# df_name_new = df_name.sort_values('update_time', ascending=False).groupby('order_id').head(1)
# df_jl_name = df_jl_new[df_jl_new.是否出库==1].merge(df_name_new, on='order_id', how='left')
# df_jl_name_new = df_jl_name[['下单日期', 'order_id', 'order_number', '策略命中等级', '分配人']]
# with pd.ExcelWriter(f'F:/需求/廖志彧需求/分配数据_{Today_zfb}.xlsx', engine='openpyxl') as writer:
#     df_jl_name_new.to_excel(writer, sheet_name='出库单分配人', index=False)
# df_jl_name_new

# # 首逾
# date1 = '2025-06'
# date2 = '2025-07'
# df_new = df_jl[df_jl.下单月份==date1]
# df_new = df_new[df_new.是否出库==1]
# df_stages_new = df_stages[df_stages['order_id'].notna()]
# df_stages_new['order_id'] = df_stages_new['order_id'].astype(int)
# df_stages_new.loc[:, '预计还款月份'] = df_stages_new.refund_date.dt.strftime('%Y-%m')
# df_new_merge = df_new.merge(df_stages_new, on='order_id', how='left')
# df_jl_stages_new = df_new_merge[(df_new_merge.预计还款月份==date2)&(df_new_merge.sort==2)]
# df_jl_stages_new = df_new_merge[(df_new_merge.sort==2)]
# df_jl_stages_new.loc[:, 'now_day'] = pd.to_datetime(now_day.strftime('%Y-%m-%d'))
# # df_jl_stages_new = df_jl_stages_new[df_jl_stages_new.refund_date<=df_jl_stages_new.now_day]
# df_jl_stages_new.loc[:, 'overdue_day'] = np.where(df_jl_stages_new.reality_refund_date.notna(), 
#                                                 np.where(df_jl_stages_new.reality_refund_date>df_jl_stages_new.refund_date, (df_jl_stages_new.reality_refund_date-df_jl_stages_new.refund_date), 0)
#                                                 , np.where(df_jl_stages_new.refund_date>df_jl_stages_new.now_day, 0, (df_jl_stages_new.now_day-df_jl_stages_new.refund_date)))
# df_jl_stages_new.loc[:, 'overdue_day'] = df_jl_stages_new['overdue_day'].apply(lambda x: x.days)
# # df_jl_stages_new = df_jl_stages_new[df_jl_stages_new.overdue_day>0]

# # df_jl_stages_news = df_jl_stages_new[df_jl_stages_new.overdue_day>0][['order_id', 'order_number', '下单日期', 'refund_date', 'reality_refund_date',  'overdue_day']].rename(columns={'order_id': '订单id', 'order_number': '订单号', 'refund_date': '应还日期', 'reality_refund_date': '实还日期', 'overdue_day': '逾期天数'})
# df_jl_stages_news = df_jl_stages_new[['order_id', 'order_number', '下单日期', 'refund_date', 'reality_refund_date',  'overdue_day', 'status2']].rename(columns={'order_id': '订单id', 'order_number': '订单号', 'refund_date': '应还日期', 'reality_refund_date': '实还日期', 'overdue_day': '逾期天数'})
# df_jl_stages_news = df_jl_stages_news.merge(df_name_new[['order_id', '分配人']], left_on='订单id', right_on='order_id', how='left')
# df_jl_stages_news.loc[:, '是否逾期'] = np.where((df_jl_stages_news.status2=='租赁中')&(df_jl_stages_news.实还日期.isna())&(df_jl_stages_news.逾期天数>0), 1, 0)
# df_jl_stages_news_g = df_jl_stages_news.groupby('分配人').agg({'order_id': 'count', '是否逾期': 'sum'}).rename(columns={'order_id': '出库'})
# df_jl_stages_news_g.loc[:, '逾期/出库'] = (df_jl_stages_news_g.是否逾期/df_jl_stages_news_g.出库).map(lambda x: format(x, '.2%'))
# with pd.ExcelWriter(f'F:/需求/廖志彧需求/首逾_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_jl_stages_news.to_excel(writer, sheet_name='出库订单', index=False)
#     df_jl_stages_news_g.to_excel(writer, sheet_name='首逾订单')
# df_jl_stages_news

In [199]:
# df_jl_new[['下单日期', 'order_number', '策略命中等级', '是否出库']].to_excel(f'F:/需求/总需求/拒量回捞数据_{t_date}.xlsx', index=False)
# df_jl_stages_news_g

## 拒量日报

In [200]:
# df_jl
df[(df['tips'].str.contains(r'联合拒量订单|支付宝联合运营订单', na=False))].shape

(1974, 119)

In [201]:
df_jl = df[(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True)&(~df['tips'].str.contains(r'联合拒量订单|支付宝联合运营订单', na=False))]
# 剔除回捞被拒漏到商家的数据9.1
reject_merchants = ["小蚂蚁租机", "兴鑫兴通讯", "人人享租", "崇胜数码", "喜卓灵租机", "喜卓灵新租机"]
df_jl = df_jl[~df_jl['merchant_name'].isin(reject_merchants)]

df_jl_group = df_jl.groupby('下单日期').agg({'是否进件': 'sum', '人审拒绝': 'sum', '风控通过件': 'sum', '客户取消': 'sum', '无法联系': 'sum', '出库前风控强拒': 'sum', '待审核': 'sum', '是否出库': 'sum', '已退款': 'sum'}).rename(columns={'是否进件': '进件数', '是否出库': '出库'})
df_jl_group.loc[:, '客户取消'] = df_jl_group.客户取消+df_jl_group.已退款
df_jl_group.loc[:, '风控通过率'] = (df_jl_group.风控通过件/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '进件出库率'] = (df_jl_group.出库/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '取消比例'] = (df_jl_group.客户取消/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '电审拒绝率'] = (df_jl_group.人审拒绝/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '出库前强拒比例'] = (df_jl_group.出库前风控强拒/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '无法联系占比'] = (df_jl_group.无法联系/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group_new = df_jl_group[['进件数', '人审拒绝', '风控通过件', '风控通过率', '客户取消', '无法联系', '出库前风控强拒', '待审核', '出库', '进件出库率', '取消比例', '电审拒绝率', '出库前强拒比例', '无法联系占比']].reset_index()

df_jl_group_new#.tail()

,下单日期,进件数,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,进件出库率,取消比例,电审拒绝率,出库前强拒比例,无法联系占比
0,2025-11-09,2,2,0,0.00%,0,0,0,0,0,0.00%,0.00%,100.00%,0.00%,0.00%
1,2025-11-10,3,3,0,0.00%,0,0,0,0,0,0.00%,0.00%,100.00%,0.00%,0.00%
2,2025-11-11,2,2,0,0.00%,0,0,0,0,0,0.00%,0.00%,100.00%,0.00%,0.00%
3,2025-11-13,2,2,0,0.00%,0,0,0,0,0,0.00%,0.00%,100.00%,0.00%,0.00%
4,2025-11-14,2,1,1,50.00%,0,1,0,0,0,0.00%,0.00%,50.00%,0.00%,50.00%
5,2025-11-18,2,2,0,0.00%,0,0,0,0,0,0.00%,0.00%,100.00%,0.00%,0.00%
6,2025-11-20,3,2,1,33.33%,1,0,0,0,0,0.00%,33.33%,66.67%,0.00%,0.00%
7,2025-11-21,1,0,1,100.00%,0,1,0,0,0,0.00%,0.00%,0.00%,0.00%,100.00%
8,2025-11-22,1,0,1,100.00%,0,1,0,0,0,0.00%,0.00%,0.00%,0.00%,100.00%
9,2025-11-23,1,1,0,0.00%,0,0,0,0,0,0.00%,0.00%,100.00%,0.00%,0.00%


# 挽留

In [202]:
df_wl_merge = df_wl.merge(df[['order_number', '是否出库']], on='order_number', how='left')
df_wl_merge.loc[:, '下单日期'] = pd.to_datetime(df_wl_merge.下单日期)
df_wl_merge.loc[:, '挽留成功'] = np.where(df_wl_merge.task_status=='SUCCESS', 1, 0)
df_wl_g = df_wl_merge.groupby('下单日期').agg({'order_number': 'nunique', '挽留成功': 'sum', '是否出库': 'sum'}).rename(columns={'order_number': '挽留数', '是否出库': '出库'})
df_wl_g_merge = df_all[['去重订单数', '进件数', '客户取消']].merge(df_wl_g, on='下单日期', how='left')
df_wl_g_merge.loc[:, '进件出库率'] = (df_wl_g_merge.出库/df_wl_g_merge.进件数).map(lambda x: format(x, '.2%'))
df_wl_g_merge.loc[:, '订单出库率'] = (df_wl_g_merge.出库/df_wl_g_merge.去重订单数).map(lambda x: format(x, '.2%'))
df_wl_g_merge.loc[:, '挽留成功率'] = (df_wl_g_merge.挽留成功/df_wl_g_merge.挽留数).map(lambda x: format(x, '.2%'))
df_wl_g_merge.loc[:, '挽留出库率'] = (df_wl_g_merge.出库/df_wl_g_merge.挽留数).map(lambda x: format(x, '.2%'))
df_wl_g_merge = clean.continuous_dates(df_wl_g_merge)
df_wl_g_merge_new = df_wl_g_merge.reset_index(names=['下单日期'])
df_wl_g_merge_new.tail()

,下单日期,去重订单数,进件数,客户取消,挽留数,挽留成功,出库,进件出库率,订单出库率,挽留成功率,挽留出库率
11,2025-11-20,6996,1687,37,9.0,4.0,1.0,0.06%,0.01%,44.44%,11.11%
12,2025-11-21,5651,1551,43,6.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%
13,2025-11-22,5132,1485,26,5.0,1.0,1.0,0.07%,0.02%,20.00%,20.00%
14,2025-11-23,5025,1433,18,3.0,2.0,0.0,0.00%,0.00%,66.67%,0.00%
15,2025-11-24,736,160,0,0.0,0.0,0.0,nan%,nan%,nan%,nan%


# 获取日报文件名称并修改

In [203]:
now = datetime.now()
now_day = pd.to_datetime(now.strftime("%Y-%m-%d"))
now_date = now.strftime('%m%d')
before_date = (now-timedelta(days=1)).strftime('%m%d')
before_date_two = (now-timedelta(days=2)).strftime('%m%d')
before_date_four = (now-timedelta(days=4)).strftime('%m%d')
monday = pd.to_datetime(now.strftime('%Y-%m-%d')).day_name()

In [204]:
# 指定目录路径
directory = 'F:/日报/日报'

# 查找所有 .xlsx 和 .xls 文件
excel_files = glob.glob(os.path.join(directory, '*.xlsx')) + glob.glob(os.path.join(directory, '*.xls'))

for old_file_path in excel_files:
    # 提取文件名（不带路径）
    old_filename = os.path.basename(old_file_path)
    old_date = old_filename.split('.')[0][-4:]
    new_filename = old_filename.replace(old_date, before_date)
    # 将新的文件名添加到路径
    new_file_path = os.path.join(directory, new_filename)

    # 确保新文件名不存在，以避免覆盖文件
    if not os.path.exists(new_file_path):
        os.rename(old_file_path, new_file_path)
        print(f"文件 {old_filename} 已重命名为: {new_filename}")
    else:
        print(f"跳过文件 {old_filename}，因为新文件名已存在。")

文件 new商户订单统计表截至1120.xlsx 已重命名为: new商户订单统计表截至1123.xlsx
文件 new迪瓜租机订单日报截止1120.xlsx 已重命名为: new迪瓜租机订单日报截止1123.xlsx
文件 ~$new商户订单统计表截至1120.xlsx 已重命名为: ~$new商户订单统计表截至1123.xlsx
文件 ~$new迪瓜租机订单日报截止1120.xlsx 已重命名为: ~$new迪瓜租机订单日报截止1123.xlsx


# 读取日报

## 获取日期及星期

In [205]:

# 中文星期映射 获取星期几

# 设置需要打开的文件和密码
# file_path1 = f'F:/日报/日报/迪瓜租机订单日报截止{before_date}.xlsx'
file_path1 = f'F:/日报/日报/new迪瓜租机订单日报截止{before_date}.xlsx'

# file_path2 = f'F:/日报/日报/商户订单统计表截至{before_date}.xlsx'
file_path2 = f'F:/日报/日报/new商户订单统计表截至{before_date}.xlsx'
password = '20240101'

## 日报数据

In [206]:
def Open_Excel2(df_list, path, password, col_list, key_list, key):
    '''
    对日报数据进行自动更新
    :param df: 日报各个渠道的df
    :param path: 日报的excel文件路径
    :param password: excel文件的密码
    :param sheet_name: excel的sheet页名称
    :param col: 列的位置，如A列，B列等
    :param col_len: 每个模块之间的间隔宽度
    :param key: 判断是否有进行合计
    :return:
    '''
    # 定义文件路径和密码
    file_path = path
    password = password
    # 使用 xlwings 打开加密的 Excel 文件
    app = xw.App(visible=False)
    # 获取今天是星期几
    week = pd.to_datetime(datetime.now().strftime('%Y-%m-%d')).day_name()
    # 初始化数据列长
    col_len = 1
    try:
        wb = xw.Book(file_path, password=password)
        # 获取工作表
        sheet_names = wb.sheet_names
        num = 1 if key=='日报' else 0
        for idx, df_lis in enumerate(df_list):
            sheet = wb.sheets[sheet_names[idx+num]]
            # print(sheet_names[idx])
            col = col_list[idx]
            key = key_list[idx]
            for idx1, df in enumerate(df_lis):
                # 如果df没有数据则不执行操作
                # print(len(df))
                if len(df)!=0:
                    # print(len(df))
                    # 获取该工作表的最后一行，并插入一行
                    # col_len:判断是否是第一列第一个需要插入的数据；
                    # key：值等于1用来判断是否是免人审数据，免审订单转化，押金类型这三个数据页在最后一行的前三行插入； 值不等于1则在最后一行插入并且起始行数最后减4
                    # 获取最后一行的日期数据进行判断
                    last_row = sheet.range(col + str(sheet.cells.last_cell.row)).end('up').row
                    # 改进代码（查找特定列中的最后一个非空单元格）
                    # last_row = sheet.range(f'{col}{sheet.cells.last_cell.row}').end('up').row
                    # print(last_row)
                    # 更改
                    last_date = sheet.range(f'{col}{last_row}').value if key==0 else sheet.range(f'{col}{last_row - 4}').value
                    while last_date is None and last_row > 1: # 更改{
                        last_row -= 1
                        last_date = sheet.range(f'{col}{last_row}').value # 更改}
                    # 获取当前列的长度
                    col_len = sheet.range(f'{col}{1}').end('down').row
                    print(f"工作表: {sheet.name},current_key: {key}, col: {col}, last_row: {last_row}, last_date: {last_date}!")
                    # 获取前一天的日期
                    before_date = pd.to_datetime((datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d'))
                    if col_len == 1:
                        if before_date != last_date:
                            # print(f'before_date: {before_date},last_date:{last_date}')
                            diff = (before_date - last_date).days
                            if key==0:
                                # 判断最后一行日期是不是前一天的日期，如不是则差几天就插入几行，如是则不插入新的行
                                sheet.api.Rows(last_row).Insert() if diff == 1 else sheet.api.Rows(f'{last_row}:{last_row + diff - 1}').Insert()
                            else:
                                sheet.api.Rows(last_row - 3).Insert() if diff == 1 else sheet.api.Rows(f'{last_row - diff + 1}:{last_row}').Insert()

                    # 重新获取最后一行
                    last_row = sheet.used_range.last_cell.row
                    # 将 DataFrame 写入固定的位置
                    # 对应Excel中的行和列，起始行数
                    start_row = last_row - len(df) - 1 if key==0 else last_row - len(df) - 3
                    # print(start_row)
                    # 打印关键变量，便于调试
                    print(f"写入时，工作表: {sheet.name}, last_row: {last_row}, len(df): {len(df)}, current_key: {key}, start_row: {start_row}")
                    # 定位写入区域左上角单元格（行, 列）
                    target_range = sheet.range((start_row, col_len))
                    # 一次性写入整个DataFrame数据
                    target_range.options(index=False, header=False).value = df
                    # 获取上一个df的宽度并相加
                    col_len += 1 + len(df.columns)
            # 新的sheet页，数据列长重新赋值
            col_len = 1
        
        # 保存并重新加密文件
        # wb.save(password=password)
        # 确保关闭工作簿和应用程序实例
        wb.close()
        app.quit()
    finally:
        # 确保关闭工作簿和应用程序实例
        wb.close()
        app.quit()


## 创建新的sheet页

In [207]:



# # 自定义列名转数字的函数（替代xlwings.utils.col_name_to_num）
# def col_name_to_num(col_name):
#     """
#     将Excel列名（如'A', 'B', 'AA'）转换为数字（1, 2, 27）
#     """
#     num = 0
#     for c in col_name:
#         num = num * 26 + (ord(c.upper()) - ord('A') + 1)
#     return num

# def write_with_index_and_header(df_list, file_path, password, new_sheet_name, 
#                                start_col='A', start_row=1):
#     '''
#     创建新sheet页，同时写入行索引和列索引（表头），兼容所有xlwings版本
#     :param df_list: 待写入的DataFrame列表
#     :param file_path: Excel文件路径
#     :param password: Excel密码
#     :param new_sheet_name: 新sheet页名称
#     :param start_col: 起始列（默认'A'）
#     :param start_row: 起始行（默认1）
#     '''
#     app = xw.App(visible=False, add_book=False)
#     try:
#         # 打开加密文件
#         wb = xw.Book(file_path, password=password)
        
#         # 删除同名sheet（若存在）
#         for sheet in wb.sheets:
#             if sheet.name == new_sheet_name:
#                 sheet.delete()
        
#         # 创建新sheet
#         new_sheet = wb.sheets.add(name=new_sheet_name)
        
#         # 使用自定义函数将列名（如'A'）转换为数字（如1）
#         current_col = col_name_to_num(start_col)  # 替换原xlwings.utils的方法
        
#         for df in df_list:
#             if len(df) == 0:
#                 continue  # 跳过空数据
            
#             # 确定写入区域左上角（行=start_row，列=current_col）
#             target_range = new_sheet.range((start_row, current_col))
            
#             # 写入数据（保留行索引和列索引）
#             target_range.options(
#                 index=False,    # 不保留行索引
#                 header=True,   # 保留列索引（表头）
#                 chunksize=1000 # 大数据量优化
#             ).value = df
            
#             # 计算下一个数据的起始列（当前数据总列数 + 2列间隔）
#             total_cols = 1 + df.shape[1]  # 1（行索引） + 数据列数
#             current_col += total_cols + 2  # 加2列间隔
        
#         wb.save(password=password)
#         print(f"已成功写入新sheet《{new_sheet_name}》，包含行索引和列索引（表头）")
    
#     except Exception as e:
#         print(f"错误：{str(e)}")
#     finally:
#         wb.close()
#         app.quit()
        
# # 日报数据 - 在新sheet页写入
# # write_with_index_and_header([df_qudao_group[:-1]], file_path1, password, "拒收")

# # # 商家数据 - 在新sheet页写入
# # write_to_new_sheet(sj_df_list[0], file_path2, password, "商家数据汇总")

## 加入了写入空表的逻辑

In [208]:
def Open_Excel(df_list, path, password, col_list, key_list, key):
    '''
    对日报数据进行自动更新（增强版，处理空工作表）
    :param df_list: 日报各个渠道的df列表
    :param path: 日报的excel文件路径
    :param password: excel文件的密码
    :param col_list: 列的位置，如A列，B列等
    :param key_list: 每个sheet页的处理方式标志
    :param key: 判断是否有进行合计
    :return:
    '''
    # 定义文件路径和密码
    file_path = path
    password = password
    # 使用 xlwings 打开加密的 Excel 文件
    app = xw.App(visible=False)
    # 获取今天是星期几
    week = pd.to_datetime(datetime.now().strftime('%Y-%m-%d')).day_name()
    # 初始化数据列长
    col_len = 1
    try:
        wb = xw.Book(file_path, password=password)
        # 获取工作表
        sheet_names = wb.sheet_names
        num = 1 if key == '日报' else 0
        
        for idx, df_lis in enumerate(df_list):
            # 确保索引不越界
            if idx + num >= len(sheet_names):
                print(f"警告: 工作表索引 {idx+num} 超出范围，跳过")
                continue
                
            sheet = wb.sheets[sheet_names[idx+num]]
            col = col_list[idx]
            current_key = key_list[idx]  # 避免与函数参数key冲突
            
            # 检查工作表是否为空
            is_empty_sheet = sheet.used_range.last_cell.row == 1 and sheet.used_range.last_cell.column == 1
            
            for df in df_lis:
                if len(df) == 0:
                    continue
                
                if is_empty_sheet:
                    # 空工作表处理逻辑：直接写入数据，保留原格式
                    print(f"提示: 工作表 '{sheet.name}' 为空，直接写入数据")
                    start_row = 1  # 从第一行开始写入
                    print(f"工作表: {sheet.name}, len(df): {len(df)}, current_key: {current_key}, start_row: {start_row}")
                    # 写入数据（保留索引和表头）
                    target_range = sheet.range((start_row, col_len))
                    # 一次性写入整个DataFrame数据
                    target_range.options(index=False, header=True).value = df
                    
                    # 更新列位置
                    col_len += 1 + len(df.columns)
                    continue
                # 非空工作表的原有逻辑
                # 获取该工作表的最后一行，并插入一行
                    # col_len:判断是否是第一列第一个需要插入的数据；
                    # key：值等于1用来判断是否是免人审数据，免审订单转化，押金类型这三个数据页在最后一行的前三行插入； 值不等于1则在最后一行插入并且起始行数最后减4
                    # 获取最后一行的日期数据进行判断
                
                # 获取最后一行的日期数据进行判断
                last_row = sheet.range(col + str(sheet.cells.last_cell.row)).end('up').row
                last_date = sheet.range(f'{col}{last_row}').value if current_key==0 else sheet.range(f'{col}{last_row - 4}').value
                # 更改
                while last_date is None and sheet.name in ['拒收', '星晟数码', '蘑菇时间']: # 更改{
                    last_row -= 1                    
                    last_date = sheet.range(f'{col}{last_row}').value # 更改}
                print(f"工作表: {sheet.name},current_key: {current_key}, col:{col},last_row: {last_row}, last_date: {last_date}!")
                # if last_date is None:
                #     # 日期为空：插入一行并使用前一天作为默认日期 姑且看看
                #     if current_key == 0:
                # 非空工作表的原有逻辑
                # 获取最后一行
                # last_cell = sheet.range(col + str(sheet.cells.last_cell.row)).end('up')
                # last_row = last_cell.row无用
                
                # # 获取最后一行的日期
                # last_date_value = last_cell.value
                
                # # 处理日期值为空或无效的情况
                # if last_date_value is None:
                #     print(f"警告: 工作表 '{sheet.name}' 的日期列 {col} 为空，将插入新行")
                #     last_date = None
                # else:
                #     try:
                #         # 尝试将Excel日期值转换为pandas Timestamp
                #         if isinstance(last_date_value, str):
                #             # 处理字符串格式的日期
                #             last_date = pd.to_datetime(last_date_value).normalize()
                #         elif isinstance(last_date_value, (datetime, pd.Timestamp)):
                #             # 处理datetime或Timestamp对象
                #             last_date = pd.Timestamp(last_date_value).normalize()
                #         else:
                #             # 处理Excel数值格式的日期（浮点数）
                #             last_date = xw.utils.xl_to_datetime(last_date_value).normalize()
                #     except Exception as e:
                #         print(f"警告: 无法解析工作表 '{sheet.name}' 的日期值 '{last_date_value}': {str(e)}")
                #         last_date = None ！！！！！
                
                # 获取前一天的日期
                before_date = pd.to_datetime((datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d'))
                
                # 处理日期为空或无效的情况
                if col_len == 1:
                    if last_date is None:
                        # 日期为空：插入一行并使用前一天作为默认日期
                        if current_key == 0:
                            sheet.api.Rows(last_row).Insert()
                        
                        else:
                            sheet.api.Rows(last_row - 3).Insert()
                        # 写入默认日期
                        sheet.range(f"{col}{last_row}").value = before_date.strftime('%Y-%m-%d')
                        last_date = before_date  # 更新last_date避免后续错误
                    # 判断最后一行日期是不是前一天的日期，如不是则差几天就插入几行，如是则不插入新的行
                    elif before_date != last_date:
                        # 日期不匹配：计算差值并插入相应行数
                        diff = (before_date - last_date).days
                        if diff > 0:  # 确保差值为正数
                            if current_key == 0:
                                sheet.api.Rows(last_row).Insert() if diff == 1 else sheet.api.Rows(f'{last_row}:{last_row + diff - 1}').Insert()
                            else:
                                sheet.api.Rows(last_row - 3).Insert() if diff == 1 else sheet.api.Rows(f'{last_row - diff + 1}:{last_row}').Insert()
                
                # 重新获取最后一行
                last_row = sheet.used_range.last_cell.row
                
                # 计算起始行
                start_row = last_row - len(df) - 1 if current_key == 0 else last_row - len(df) - 3
                start_row = max(start_row, 3)  # 确保行号至少为3，避免0或负数
                # 打印关键变量，便于调试
                print(f"写入时，工作表: {sheet.name}, last_row: {last_row}, len(df): {len(df)}, current_key: {current_key}, start_row: {start_row}")
                # 写入数据（保留索引）
                target_range = sheet.range((start_row, col_len))
                target_range.options(index=False, header=False).value = df
                
                # 更新列位置# 获取上一个df的宽度并相加
                col_len += 1 + len(df.columns)
            
            # 新的sheet页重置列位置
            col_len = 1
        
        # 保存并重新加密文件
        print("已保存并重新加密文件")
        wb.save(password=password)
        # # 确保关闭工作簿和应用程序实例
        # wb.close()
        # app.quit()
    finally:
        # 确保关闭工作簿和应用程序实例
        wb.close()
        app.quit()

## 将日报数据写入excel

In [209]:
# 日报数据
df_list = [[df_all_new[:-1], df_ss_group_new[:-1],df_dr_group_new[:-1], df_zm_group_new[:-1], df_zw_group_new[:-1], df_tc[:-1], df_dy_group_new[:-1], df_zfb_group1[:-1]],
            [df_ms_new3[:-1]], [df_ms_o_group_new[:-1]],[df_qmy_group_new[:-1], df_fmy_group_new[:-1]], [df_r_new[:-1]], [df_s2merge_all_new[:-1]], [df_jl_group_new[:-1]], 
            [df_wl_g_merge_new[:-1]], [df_jd_group_new[:-1], df_jdsc_group_new[:-1]],[df_qudao_group[:]],[df_app_g_new[:-1]], [df_wb_group_new[:-1]]]# , [df_qudao_group[:-1]]
col_list = ['C', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A']
key_list = [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0]

Open_Excel(df_list, file_path1, password, col_list, key_list, key='日报')
del df_list
gc.collect()
print('内存回收成功')

# # 日报数据
# df_list = [[df_all_new[:-1], df_ss_group_new[:-1], df_dr_group_new[:-1], df_zm_group_new[:-1], df_zw_group_new[:-1], df_tc[:-1], df_dy_group_new[:-1], df_zfb_group1[:-1]], 
#             [df_ms_new3[:-1]], [df_ms_o_group_new[:-1]],[df_qmy_group_new[:-1], df_fmy_group_new[:-1]], [df_r_new[:-1]], [df_s2merge_all_new[:-1]], [df_jl_group_new[:-1]], 
#             [df_wl_g_merge_new[:-1]], [df_jd_group_new[:-1], df_jdsc_group_new[:-1]]]
# col_list = ['C', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'A']
# key_list = [0, 1, 1, 1, 0, 0, 0, 0, 0]
# Open_Excel2(df_list, file_path1, password, col_list, key_list, key='日报')
# 商家数据
sj_df_list = [[cxyz_new[:-1]], [hnw_new[:-1]], [zzy_new[:-1]], [qzsm_new[:-1]], [hkhz_new[:-1], hkhz_dy_new[:-1]], [xmy_new[:-1]], [rrxz_new[:-1]], [jgwl_new[:-1]], [xzl_new[:-1]],
                [xzl_news[:-1]], [xcsm_new[:-1], xcsm_zfb_new[:-1], xcsm_jd_new[:-1]], [mgsj_New[:-1]], [yqdy_New[:-1]], [aony_New[:-1]]]
sj_col_list = ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A']
sj_key_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Open_Excel(sj_df_list, file_path2, password, sj_col_list, sj_key_list, key='商家')
del sj_df_list
gc.collect()
print('内存回收成功')


工作表: 2025年,current_key: 0, col:C,last_row: 327, last_date: 2025-11-20 00:00:00!
写入时，工作表: 2025年, last_row: 332, len(df): 15, current_key: 0, start_row: 316
工作表: 2025年,current_key: 0, col:C,last_row: 330, last_date: 2025-11-23 00:00:00!
写入时，工作表: 2025年, last_row: 332, len(df): 15, current_key: 0, start_row: 316
工作表: 2025年,current_key: 0, col:C,last_row: 330, last_date: 2025-11-23 00:00:00!
写入时，工作表: 2025年, last_row: 332, len(df): 15, current_key: 0, start_row: 316
工作表: 2025年,current_key: 0, col:C,last_row: 330, last_date: 2025-11-23 00:00:00!
写入时，工作表: 2025年, last_row: 332, len(df): 15, current_key: 0, start_row: 316
工作表: 2025年,current_key: 0, col:C,last_row: 330, last_date: 2025-11-23 00:00:00!
写入时，工作表: 2025年, last_row: 332, len(df): 15, current_key: 0, start_row: 316
工作表: 2025年,current_key: 0, col:C,last_row: 330, last_date: 2025-11-23 00:00:00!
写入时，工作表: 2025年, last_row: 332, len(df): 15, current_key: 0, start_row: 316
工作表: 2025年,current_key: 0, col:C,last_row: 330, last_date: 2025-11-23 

In [210]:
# sheet_name = '2025年'
# # 自动写入日报数据 总体

# col_len = all_models.Open_Excel2(df_all_new[:-1], file_path1, password, sheet_name, 'C')
# # 搜索渠道
# col_len1 = all_models.Open_Excel2(df_ss_group_new[:-1], file_path1, password, sheet_name, 'C', col_len)
# # 单人会话
# df_dr_group_new = df_dr_group.reset_index()
# col_len2 = all_models.Open_Excel2(df_dr_group_new[:-1], file_path1, password, sheet_name, 'C', col_len1)
# # 芝麻租物
# col_len3 = all_models.Open_Excel2(df_zm_group_new[:-1], file_path1, password, sheet_name, 'C', col_len2)
# # 纯租物
# df_zw_group_new = df_zw_group.reset_index()
# col_len4 = all_models.Open_Excel2(df_zw_group_new[:-1], file_path1, password, sheet_name, 'C',  col_len3)
# # 总体剔除直播
# df_tc_new = df_tc.reset_index()
# col_len5 = all_models.Open_Excel2(df_tc_new[:-1], file_path1, password, sheet_name, 'C',  col_len4)
# # 抖音渠道
# df_dy_group_new = df_dy_group.reset_index()
# col_len6 = all_models.Open_Excel2(df_dy_group_new[:-1], file_path1, password, sheet_name, 'C', col_len5)
# #  支付宝直播
# df_zfb_group_new = df_zfb_group1.reset_index()
# col_len7 = all_models.Open_Excel2(df_zfb_group_new[:-1], file_path1, password, sheet_name, 'C',  col_len6)
# ## 免人审数据
# all_models.Open_Excel2(df_ms_new3[:-1], file_path1, password, '免人审数据', 'B',  key=1)
# ## 免审订单转化统计
# all_models.Open_Excel2(df_ms_o_group_new[:-1], file_path1, password, '免审订单转化统计', 'B',  key=1)
# # 全免押
# yj_col_len = all_models.Open_Excel2(df_qmy_group_new[:-1], file_path1, password, '押金类型', 'A',  key='押金')
# # # 非免押
# all_models.Open_Excel2(df_fmy_group_new[:-1], file_path1, password, '押金类型', 'A', col_len=yj_col_len, key='押金')
# ## 租完即送占比
# all_models.Open_Excel2(df_r_new[:-1], file_path1, password, '租完即送占比', 'A', key='租完即送')
# ## 碎屏险数据
# all_models.Open_Excel2(df_s2merge_all_new[:-1], file_path1, password, '碎屏险数据', 'A', key='碎屏险')
# ## 拒量
# all_models.Open_Excel2(df_jl_group_new[:-1], file_path1, password, '拒量', 'A', key='拒量')
# ## 挽留
# all_models.Open_Excel2(df_wl_g_merge_new[:-1], file_path1, password, '挽留', 'A', key='挽留')
# ## 京享租
# col_len = all_models.Open_Excel2(df_jd_group_new[:-1], file_path1, password, '京享租', 'A', key='京享租')
# # 京东商城
# all_models.Open_Excel2(df_jdsc_group_new[:-1], file_path1, password, '京享租', 'A', key='京享租', col_len=col_len)


## 商家数据

In [211]:
# def merchant(df, name):
#     df_max_date = df.下单日期.max()
#     if df_max_date<(now_day-timedelta(days=1)):
#         df.loc[len(df)] = np.nan
#         # return df
#         all_models.Open_Excel2(df=df, path=file_path2, password=password, sheet_name=name, col='A', key='商家')
#     else:
#         all_models.Open_Excel2(df=df, path=file_path2, password=password, sheet_name=name, col='A', key='商家')
#         # return df

In [212]:
# # 澄心优租
# merchant(cxyz_new[:-1], '澄心优租')
# # 北京海鸟窝科技有限公司
# merchant(hnw_new[:-1], '海鸟窝')
# # 租着用
# merchant(zzy_new[:-1], '租着用')
# # 趣智数码
# merchant(qzsm_new[:-1], '趣智数码')
# # 汇客好租
# merchant(hkhz_new[:-1], '汇客好租')
# # 小蚂蚁租机
# merchant(xmy_new[:-1], '小蚂蚁租机')
# # 乙辉数码
# # merchant(yhsm_new, '乙辉数码')
# # 兴鑫兴通讯
# # merchant(xxx_new, '兴鑫兴通讯')

# # 南京聚格网络科技
# merchant(jgwl_new[:-1], '南京聚格网络科技')
# # 呱子笔记本电脑
# # merchant(gz_new[:-1], '呱子笔记本电脑')
# # 人人享租
# merchant(rrxz_new[:-1], '人人享租')
# # 崇胜数码
# # merchant(cssm_new[:-1], '崇胜数码')
# # 喜卓灵租机
# merchant(xzl_new[:-1], '喜卓灵租机')